In [ ]:
{
cells: [
{
cell_type: "markdown",
metadata: { },
source: [
"# Samsung Stock Prediction ",
" ",
"## Library"
]
},
{
cell_type: "code",
execution_count: 82,
metadata: { },
outputs: [ ],
source: [
"import numpy as np ",
"import pandas as pd ",
"import matplotlib.pyplot as plt ",
"import os ",
"from matplotlib.pylab import rcParams ",
"from statsmodels.tsa.stattools import adfuller ",
"from statsmodels.tsa.seasonal import seasonal_decompose ",
"from statsmodels.graphics.tsaplots import plot_acf, plot_pacf ",
"from statsmodels.tsa.arima_model import ARIMA ",
"from sklearn.metrics import mean_squared_error, mean_absolute_error ",
"import math"
]
},
{
cell_type: "markdown",
metadata: { },
source: [
"## Data Import"
]
},
{
cell_type: "code",
execution_count: 84,
metadata: { },
outputs: [
{
name: "stdout",
output_type: "stream",
text: [
"Date ",
"2000-01-04 6110.0 ",
"2000-01-05 5580.0 ",
"2000-01-06 5620.0 ",
"2000-01-07 5540.0 ",
"2000-01-10 5770.0 ",
"Name: Close, dtype: float64 ",
"(5187,) "
]
}
],
source: [
"filepath = os.getenv('HOME') + '/aiffel/stock_prediction/data/005930.KS.csv' ",
"df = pd.read_csv(filepath, index_col='Date', parse_dates=True) ",
"ts = df['Close'] ",
"print(ts.head()) ",
"print(ts.shape)"
]
},
{
cell_type: "markdown",
metadata: { },
source: [
"## Check the data ",
"### Visualization"
]
},
{
cell_type: "code",
execution_count: 85,
metadata: { },
outputs: [
{
data: {
text/plain: [
"[<matplotlib.lines.Line2D at 0x7fcca945f290>]"
]
},
execution_count: 85,
metadata: { },
output_type: "execute_result"
},
{
data: {
image/png: "... ",
text/plain: [
"<Figure size 1080x432 with 1 Axes>"
]
},
metadata: {
needs_background: "light"
},
output_type: "display_data"
}
],
source: [
"# 증가하는 추세 ",
"rcParams['figure.figsize'] = 15,6 ",
"plt.plot(ts)"
]
},
{
cell_type: "markdown",
metadata: { },
source: [
"### Missing Value"
]
},
{
cell_type: "code",
execution_count: 86,
metadata: { },
outputs: [
{
name: "stdout",
output_type: "stream",
text: [
"Date ",
"2000-12-25 NaN ",
"2001-01-23 NaN ",
"2001-01-24 NaN ",
"2001-01-25 NaN ",
"2001-03-14 NaN ",
"2005-01-03 NaN ",
"2017-11-16 NaN ",
"2017-11-23 NaN ",
"2018-01-02 NaN ",
"2018-11-15 NaN ",
"2019-11-14 NaN ",
"Name: Close, dtype: float64 "
]
}
],
source: [
"# check the missing value ",
"print(ts[ts.isna()]) ",
"# interpolation ",
"ts = ts.interpolate(method='time')"
]
},
{
cell_type: "markdown",
metadata: { },
source: [
"### Rolling Statistics"
]
},
{
cell_type: "code",
execution_count: 31,
metadata: { },
outputs: [ ],
source: [
"# Rolling Statistics ",
"def plot_rolling_statistics(timeseries, window=12): ",
" ",
" rolmean = timeseries.rolling(window=window).mean() # 이동평균 시계열 ",
" rolstd = timeseries.rolling(window=window).std() # 이동표준편차 시계열 ",
" ",
" # 원본시계열, 이동평균, 이동표준편차를 plot으로 시각화해 본다. ",
" orig = plt.plot(timeseries, color='blue',label='Original') ",
" mean = plt.plot(rolmean, color='red', label='Rolling Mean') ",
" std = plt.plot(rolstd, color='black', label = 'Rolling Std') ",
" plt.legend(loc='best') ",
" plt.title('Rolling Mean & Standard Deviation') ",
" plt.show(block=False)"
]
},
{
cell_type: "code",
execution_count: 87,
metadata: { },
outputs: [
{
data: {
image/png: "... ",
text/plain: [
"<Figure size 1080x432 with 1 Axes>"
]
},
metadata: {
needs_background: "light"
},
output_type: "display_data"
}
],
source: [
"# 시간에 따라 평균 분석 패턴 변화하는 편. 딱히 패턴은 안보임. ",
"plot_rolling_statistics(ts, window=12)"
]
},
{
cell_type: "markdown",
metadata: { },
source: [
"### Stationary 체크"
]
},
{
cell_type: "code",
execution_count: 37,
metadata: { },
outputs: [ ],
source: [
"def augmented_dickey_fuller_test(timeseries): ",
" # statsmodels 패키지에서 제공하는 adfuller 메소드를 호출합니다. ",
" dftest = adfuller(timeseries, autolag='AIC') ",
" ",
" # adfuller 메소드가 리턴한 결과를 정리하여 출력합니다. ",
" print('Results of Dickey-Fuller Test:') ",
" dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used']) ",
" for key,value in dftest[4].items(): ",
" dfoutput['Critical Value (%s)' % key] = value ",
" print(dfoutput)"
]
},
{
cell_type: "code",
execution_count: 88,
metadata: { },
outputs: [
{
name: "stdout",
output_type: "stream",
text: [
"Results of Dickey-Fuller Test: ",
"Test Statistic 0.569039 ",
"p-value 0.986827 ",
"#Lags Used 27.000000 ",
"Number of Observations Used 5159.000000 ",
"Critical Value (1%) -3.431618 ",
"Critical Value (5%) -2.862100 ",
"Critical Value (10%) -2.567068 ",
"dtype: float64 "
]
}
],
source: [
"# p-value를 보면 안정적이지 못한 시계열 (거의 1에 가까움) ",
"augmented_dickey_fuller_test(ts)"
]
},
{
cell_type: "markdown",
metadata: { },
source: [
"### Stationary로 변환"
]
},
{
cell_type: "code",
execution_count: 89,
metadata: { },
outputs: [
{
data: {
text/plain: [
"[<matplotlib.lines.Line2D at 0x7fcca87ac450>]"
]
},
execution_count: 89,
metadata: { },
output_type: "execute_result"
},
{
data: {
image/png: "... ",
text/plain: [
"<Figure size 1080x432 with 1 Axes>"
]
},
metadata: {
needs_background: "light"
},
output_type: "display_data"
}
],
source: [
"ts_log = np.log(ts) ",
"plt.plot(ts_log)"
]
},
{
cell_type: "code",
execution_count: 90,
metadata: { },
outputs: [
{
name: "stdout",
output_type: "stream",
text: [
"Results of Dickey-Fuller Test: ",
"Test Statistic -0.687281 ",
"p-value 0.850017 ",
"#Lags Used 4.000000 ",
"Number of Observations Used 5182.000000 ",
"Critical Value (1%) -3.431613 ",
"Critical Value (5%) -2.862098 ",
"Critical Value (10%) -2.567067 ",
"dtype: float64 "
]
}
],
source: [
"# augmented dickey-fuller test로 확인. ",
"# p-value 살짝 줄어듦. ",
"augmented_dickey_fuller_test(ts_log)"
]
},
{
cell_type: "markdown",
metadata: { },
source: [
"### Time Series Decomposition"
]
},
{
cell_type: "code",
execution_count: 91,
metadata: { },
outputs: [
{
data: {
image/png: "... ",
text/plain: [
"<Figure size 1080x432 with 4 Axes>"
]
},
metadata: {
needs_background: "light"
},
output_type: "display_data"
}
],
source: [
"decomposition = seasonal_decompose(ts_log, model='multiplicative',period=30) ",
" ",
"trend = decomposition.trend ",
"seasonal = decomposition.seasonal ",
"residual = decomposition.resid ",
" ",
"plt.subplot(411) ",
"plt.plot(ts_log, label='Original') ",
"plt.legend(loc='best') ",
"plt.subplot(412) ",
"plt.plot(trend, label='Trend') ",
"plt.legend(loc='best') ",
"plt.subplot(413) ",
"plt.plot(seasonal,label='Seasonality') ",
"plt.legend(loc='best') ",
"plt.subplot(414) ",
"# trend와 seasonality를 제거하고 난 나머지 ",
"# Trend+Seasonality+Residual = Original ",
"plt.plot(residual, label='Residuals') ",
"plt.legend(loc='best') ",
"plt.tight_layout()"
]
},
{
cell_type: "code",
execution_count: 92,
metadata: { },
outputs: [
{
data: {
image/png: "... ",
text/plain: [
"<Figure size 1080x432 with 1 Axes>"
]
},
metadata: {
needs_background: "light"
},
output_type: "display_data"
}
],
source: [
"# Residual 안정성 확인 ",
"plot_rolling_statistics(residual)"
]
},
{
cell_type: "code",
execution_count: 93,
metadata: { },
outputs: [
{
name: "stdout",
output_type: "stream",
text: [
"Results of Dickey-Fuller Test: ",
"Test Statistic -1.728531e+01 ",
"p-value 5.748537e-30 ",
"#Lags Used 3.300000e+01 ",
"Number of Observations Used 5.123000e+03 ",
"Critical Value (1%) -3.431627e+00 ",
"Critical Value (5%) -2.862104e+00 ",
"Critical Value (10%) -2.567070e+00 ",
"dtype: float64 "
]
}
],
source: [
"# 이전에 직접 해준것보다 훨씬 유의미하게 나옴. ",
"residual.dropna(inplace=True) ",
"augmented_dickey_fuller_test(residual)"
]
},
{
cell_type: "markdown",
metadata: { },
source: [
"### ARIMA"
]
},
{
cell_type: "code",
execution_count: 94,
metadata: { },
outputs: [
{
data: {
image/png: "iVBORw0KGgoAAAANSUhEUgAAA3IAAAF1CAYAAACtTdhwAAAABHNCSVQICAgIfAhkiAAAAAlwSFlzAAALEgAACxIB0t1+/AAAADh0RVh0U29mdHdhcmUAbWF0cGxvdGxpYiB2ZXJzaW9uMy4yLjIsIGh0dHA6Ly9tYXRwbG90bGliLm9yZy+WH4yJAAAgAElEQVR4nO3dfZRtZ30f9u/vzpWEBAIZS2DQi0UdSsFeRXZuIa7tRAnBlqgbEq/EAbu8LahCl8my06xVMHZt3GYlpCmu7ZpYUbFiwDaYBBIrLBns0FLqFqgkKgFCEQjxoisJXSGh1/syc855+sc5Z+6ZuTP3bc6d2XvO57PWrLP3Pvs8+znaa9+Zr37Pfna11gIAAEB/7NnpDgAAAHBqBDkAAICeEeQAAAB6RpADAADoGUEOAACgZwQ5AACAnhHkAGBOqur1VfXnW/j8n1TV6+bZJwB2J0EOgG1VVZ+squ9U1Tmn8JlWVX/hTPZru1XVO6rq92e3tdaubq29d6f6BEB/CHIAbJuqujzJjyVpSf7GjnbmBKpq78lsA4CdIMgBsJ1em+QzSX4vyeoQwkmV7k0z66tDFKvqU5PNt1XVE1X1dyfb/+uququqHq6qG6rquTOf//6q+rPJew9U1dsn28+pqt+oqvsmP78xrQxW1ZVVtb+q3lpV30ryLydVs39dVb9fVY8leX1VPaOqfreq7q+qe6vqH1XV0kZftqp+s6ruqarHquqWqvqxyfarkrw9yd+dfKfb1v93qKo9VfXLVfWNqjpQVe+rqmdM3rt8UqV8XVV9s6q+XVW/tOWzA0BvCHIAbKfXJvmDyc9PVNWzT/SB1tpfniy+uLX2tNbaH1XVX0vyT5L8dJLnJPlGkg8mSVWdn+TfJ/lYkucm+QtJPjFp45eS/KUkVyR5cZKXJPnlmcN9T5JnJvneJNdMtr0yyb9OcsGk3+9NMpi0+4NJfjzJm7KxmybHemaSP0zyr6rqKa21jyX5x0n+aPKdXrzBZ18/+fmrSf6jJE9L8tvr9vnRJC9I8rIkv1JVL9ykHwDsMoIcANuiqn4044D0odbaLUm+muRnTrO5n01yfWvtc621I0l+MckPT4Zu/mSSb7XW3tVaO9xae7y19tmZz/0PrbUDrbUHk/xaktfMtDtK8quttSOttUOTbZ9urf3b1tooydOTXJ3kF1prT7bWDiT5X5K8aqNOttZ+v7X2UGtt0Fp7V5JzMg5eJ/sdf721dndr7YnJd3zVuuGdv9ZaO9Rauy3JbRmHUwAWgCAHwHZ5XZI/ba19e7L+h5kZXnmKnptxFS5JMgk6DyW5OMmlGYfEE35usvzcmfUHW2uH133mnpnl701yVpL7q+qRqnokyb9I8qyNDlZV/7Cq7qiqRyf7PiPJhSf6csfp694ks1XMb80sH8y4agfAAnDTNgBnXFWdm/EwyKXJ/WfJuDp1QVW9OMmTSc6b+cj3nKDJ+zIOVdP2n5rku5Pcm3HwevUJPnf7ZP2yybaptsFnZrfdk+RIkgtba4PjdXByP9xbMx72eHtrbVRV30lSxznWRn2duizjIZ0PJLnkBJ8FYJdTkQNgO/zNJMMkL8r4nrErkrwwyf+V8X1ztyb5qao6b/KYgTeu+/wDGd8nNvWHSd5QVVdMJiv5x0k+21r7epKPJvmeqvqFyeQm51fVSyef+0CSX66qi6rqwiS/kmTNIwCOp7V2f5I/TfKuqnr6ZEKS76uqv7LB7udnHLweTLK3qn4l46GZs9/p8qra7HfxB5L8g6p6XlU9LUfvqTtugARgMQhyAGyH1yX5l621b7bWvjX9yXjyjp/N+D6z5YzDzXsznlRk1juSvHcynPGnW2ufSPLfJ/lwkvuTfF8m96m11h5P8vIk/2XGQw+/kvGEIUnyj5LcnOTzSb6Q5HOTbafitUnOTvKlJN/JeCKU52yw38eT/EmSL2c8LPJw1g7T/FeT14eq6nMbfP76JO9P8qkkX5t8/u+fYl8B2KWqtRON7AAAAKBLVOQAAAB6RpADAADoGUEOAACgZwQ5AACAnhHkAAAAeqbTDwS/8MIL2+WXX77T3QAAANgRt9xyy7dbaxet397pIHf55Zfn5ptv3uluAAAA7Iiq+sZG2w2tBAAA6BlBDgAAoGcEOQAAgJ4R5AAAAHpGkAMAAOgZQQ4AAKBnBDkAAICeEeQAAAB6RpADAADombkEuaq6vqoOVNUXN3m/quq3ququqvp8Vf3QPI673Yajlk/c8UB+6xNfySfueCDDUdvpLgEAAAto75za+b0kv53kfZu8f3WS509+XprkdyavvTEctbzmdz+bW+95JIeWhzn37KVccekFef8bX5qlPXVa7X3yzgO5/b7H8v3PfXqufMGzTqudRW0PAAAW2VyCXGvtU1V1+XF2eWWS97XWWpLPVNUFVfWc1tr98zj+dvjknQdy6z2P5ODyMElycHmYW+95JJ+880Be9sJnn1JbZyIULlp7XQ6ZXW5PoAYA2B3mVZE7kYuT3DOzvn+y7ZggV1XXJLkmSS677LJt6dzJuP2+x3JoEuKmDi0P86X7HjvlIDfPULho7fUhZHa1vXn3bdpmV0Nr19vrct8AgO7briC30V8TG95g1lq7Lsl1SbJv377O3IT2/c99es49e2k1jCTJuWcv5UXPffoptzXPULho7XU5ZHa9vXn3rcuhtevtdblvs212NWh2uW9noj0A2Mh2Bbn9SS6dWb8kyX3bdOy5uPIFz8oVl16QT3/5/rQ9e3PeOWfliksvyJUveNYptzXPULho7XU5ZHa9vXn3rcuhtevtdblvSbeDZpf7dibam7YpBHejPYAu2a4gd0OSt1TVBzOe5OTRPt0flyRLeyrvf+NL88M/9cYsP/VZedcv/4PT/oUwz1C4aO11OWR2vb15963LobXr7XW5b0m3g2aX+3Ym2hOCu9Vel0PmorUHzCnIVdUHklyZ5MKq2p/kV5OclSSttWuT3JjkFUnuSnIwyRvmcdzttrSnct4jd+e8R+4+rV/Is+3MKxQuWntdDpldb2/efetyaO16e13uW9LtoNnlvp2J9oTgbrTXh5C5aO11OWR2vT12j3nNWvnqE7zfkvzcPI61W8wrFC5ae10OmV1vb95963Jo7Xp7Xe5b0u2g2eW+nYn2hOButNflkLlo7fUhZHa9vS6HzK631zXbNbQS5qarIbMP7c27ra6G1q631+W+Jd0Oml3u25loTwjuRntdDpmL1l6XQ2bX2+tDyOxye120Z6c7APTXNBhecO9n8rIXPnvL/zAuUntd79v73/jSXPSVf5cL9v/f+V9f/YNb+sU3z/a63Lcz0d40GNZwOWmjnDf5Q2QrIXgebS1ae9NQOGseIVN78w3Vp2OR2psNhS1rQ+HpWLT2ukiQA+AYXQ+aXe3bvNsTgrvRXpdD5qK11+WQ2fX2uhwy+9BeFwlyANBhQvDOt9flkLlo7XU5ZHa9vS6HzD6010WCHADACXQ1ZC5ae10OmV1vr8shsw/tdZHJTgAA6I0uTyrW5fa6PtFW19vrIkEOAAAWQFdDZl/a6xpDKwEAAHpGkAMAAOgZQQ4AAKBnBDkAAICeEeQAAAB6RpADAADoGUEOAACgZwQ5AACAnhHkAAAAekaQAwAA6BlBDgAAoGcEOQAAgJ4R5AAAAHpGkAMAAOgZQQ4AAKBnBDkAAICeEeQAAAB6RpADAADoGUEOAACgZwQ5AACAnhHkAAAAekaQAwAA6BlBDgAAoGcEOQAAgJ4R5AAAAHpGkAMAAOgZQQ4AAKBnBDkAAICeEeQAAAB6RpADAADoGUEOAACgZwQ5AACAnplLkKuqq6rqzqq6q6retsH7z6iqf1dVt1XV7VX1hnkcFwAAYBFtOchV1VKSdye5OsmLkry6ql60brefS/Kl1tqLk1yZ5F1VdfZWjw0AALCI5lGRe0mSu1prd7fWlpN8MMkr1+3TkpxfVZXkaUkeTjKYw7EBAAAWzjyC3MVJ7plZ3z/ZNuu3k7wwyX1JvpDk51trozkcGwAAYOHMI8jVBtvauvWfSHJrkucmuSLJb1fV0zdsrOqaqrq5qm5+8MEH59A9AACA3WUeQW5/kktn1i/JuPI26w1JPtLG7krytST/yUaNtdaua63ta63tu+iii+bQPQAAgN1lHkHupiTPr6rnTSYweVWSG9bt880kL0uSqnp2khckuXsOxwYAAFg4e7faQGttUFVvSfLxJEtJrm+t3V5Vb568f22S/zHJ71XVFzIeivnW1tq3t3psAACARbTlIJckrbUbk9y4btu1M8v3JfnxeRwLAABg0c3lgeAAAABsH0EOAACgZwQ5AACAnhHkAAAAekaQAwAA6BlBDgAAoGcEOQAAgJ4R5AAAAHpGkAMAAOgZQQ4AAKBnBDkAAICeEeQAAAB6RpADAADoGUEOAACgZwQ5AACAnhHkAAAAekaQAwAA6BlBDgAAoGcEOQAAgJ4R5AAAAHpGkAMAAOgZQQ4AAKBnBDkAAICeEeQAAAB6RpADAADoGUEOAACgZwQ5AACAnhHkAAAAekaQAwAA6BlBDgAAoGcEOQAAgJ4R5AAAAHpGkAMAAOgZQQ4AAKBnBDkAAICeEeQAAAB6RpADAADoGUEOAACgZwQ5AACAnplLkKuqq6rqzqq6q6retsk+V1bVrVV1e1X9n/M4LgAAwCLau9UGqmopybuTvDzJ/iQ3VdUNrbUvzexzQZJ/nuSq1to3q+pZWz0uAADAoppHRe4lSe5qrd3dWltO8sEkr1y3z88k+Uhr7ZtJ0lo7MIfjAgAALKR5BLmLk9wzs75/sm3Wf5zku6rqk1V1S1W9dg7HBQAAWEhbHlqZpDbY1jY4zl9M8rIk5yb5dFV9prX25WMaq7omyTVJctlll82hewAAALvLPCpy+5NcOrN+SZL7NtjnY621J1tr307yqSQv3qix1tp1rbV9rbV9F1100Ry6BwAAsLvMI8jdlOT5VfW8qjo7yauS3LBunz9O8mNVtbeqzkvy0iR3zOHYAAAAC2fLQytba4OqekuSjydZSnJ9a+32qnrz5P1rW2t3VNXHknw+ySjJe1prX9zqsQEAABbRPO6RS2vtxiQ3rtt27br1f5bkn83jeAAAAItsLg8EBwAAYPsIcgAAAD0jyAEAAPSMIAcAANAzghwAAEDPCHIAAAA9I8gBAAD0jCAHAADQM4IcAABAzwhyAAAAPSPIAQAA9IwgBwAA0DOCHAAAQM8IcgAAAD0jyAEAAPSMIAcAANAzghwAAEDPCHIAAAA9I8gBAAD0jCAHAADQM4IcAABAzwhyAAAAPSPIAQAA9IwgBwAA0DOCHAAAQM8IcgAAAD0jyAEAAPSMIAcAANAzghwAAEDPCHIAAAA9I8gBAAD0jCAHAADQM4IcAABAzwhyAAAAPSPIAQAA9IwgBwAA0DOCHAAAQM8IcgAAAD0jyAEAAPTMXIJcVV1VVXdW1V1V9bbj7PefVdWwqv72PI4LAACwiLYc5KpqKcm7k1yd5EVJXl1VL9pkv3+a5ONbPSYAAMAim0dF7iVJ7mqt3d1aW07ywSSv3GC/v5/kw0kOzOGYAAAAC2seQe7iJPfMrO+fbFtVVRcn+VtJrp3D8QAAABbaPIJcbbCtrVv/jSRvba0NT9hY1TVVdXNV3fzggw/OoXsAAAC7y945tLE/yaUz65ckuW/dPvuSfLCqkuTCJK+oqkFr7d+ub6y1dl2S65Jk37596wMhAADAwptHkLspyfOr6nlJ7k3yqiQ/M7tDa+150+Wq+r0kH90oxAEAAHBiWw5yrbVBVb0l49kol5Jc31q7varePHnffXEAAABzNI+KXFprNya5cd22DQNca+318zgmAADAoprLA8EBAADYPoIcAABAzwhyAAAAPSPIAQAA9IwgBwAA0DOCHAAAQM8IcgAAAD0jyAEAAPSMIAcAANAzghwAAEDPCHIAAAA9I8gBAAD0jCAHAADQM4IcAABAzwhyAAAAPSPIAQAA9IwgBwAA0DOCHAAAQM8IcgAAAD0jyAEAAPSMIAcAANAzghwAAEDPCHIAAAA9I8gBAAD0jCAHAADQM4IcAABAzwhyAAAAPSPIAQAA9IwgBwAA0DOCHAAAQM8IcgAAAD0jyAEAAPSMIAcAANAzghwAAEDPCHIAAAA9s3enOwAAALDeaNTSkoxay6i1tJbxT1pGLWnt6Gtr4/2m+6clo5YsD0ZpSe55+OC4zXX7Ttv4rvPOzkXnn7OD3/bUCXIAAMAJTUPPcNRWw9Vw1JIkjxxcXn1vdb/WxmFsujxZH02C1HDde621PH54kJaWT3/1obn0+fDKMEmy/zuHjrvfU/YuzeV420mQAwCAHmuTUDQNSKPZ9UnoGk4qUbMVqtlANhvQ2rqwdrSdY4/95JFBkuSO+x+f23fh5AhyAABwBmxUwZqtWm0UvjYKUQeXh2lp+cL+R2dCWctwdHSo4Mk62QoV3SfIAQCwcNYHqSR5/PDKavXp6FDAjYcBTitU64cZjlpWhwd+5u6H59LXwXCUJHliUv2CZE5BrqquSvKbSZaSvKe19s517/9skrdOVp9I8t+01m6bx7EBANh9pmFqNkBNK1krk2DzwGOHjwapaeA6QciaDWFTTxweB6Qv3vvYXPpueCDbYctBrqqWkrw7ycuT7E9yU1Xd0Fr70sxuX0vyV1pr36mqq5Ncl+SlWz02AAA7YxqSjoalybaZe7Jmq1nTIX13HXhibbjaJIAdz6HlcVt3P/jkGf+e0FXzqMi9JMldrbW7k6SqPpjklUlWg1xr7f+Z2f8zSS6Zw3EBANjEdOjgdJKL6fL6atVsRWu26rXRfVknG7Q2sjwYV9EefPzIvL8qLKR5BLmLk9wzs74/x6+2vTHJn2z2ZlVdk+SaJLnsssvm0D0AgO4aDEdrq1eTwHRMCBsdO0Rwur218f1TrSU3ff3hTWcYPNV+Je7Lgq6aR5CrDbZt+E9HVf3VjIPcj27WWGvtuoyHXmbfvn0GGAMAO24allpaDi4PVocEDtcMLWyntP2xwytJS276+nfm0sfRJLkNhv58gkUwjyC3P8mlM+uXJLlv/U5V9Z8meU+Sq1tr83nCHwDAjNkJMlZnGkzy8JPLa7Ztdn/X+u2z07s/fnglSXLbPY/Op7PyFrAF8whyNyV5flU9L8m9SV6V5Gdmd6iqy5J8JMlrWmtfnsMxAYAem71/a7ZSNRi2tLQceOzw0UC2rpJ1osA1a/qw4ju/NZ+HFQN0xZaDXGttUFVvSfLxjB8/cH1r7faqevPk/WuT/EqS707yz6sqSQattX1bPTYAsD2m4Wk4W/EaHZ0G/v5HD50wdA1GR+/92uz+rYPL4+D1VbMRAhzXXJ4j11q7McmN67ZdO7P8piRvmsexAIDja5PQtKZqNUoGo1FWhqO0ltz7yKE1E2qMRht8ZlLpOt4MhdNp4L/+7YPb9fUAyJyCHABw+maHGa5WsCbBa9halgejtCTffOjghtWu2TB2vGpXcjR4ffMhwQugzwQ5ADgNo3XBa3XYYGurVa97Hj64dkhhG98DNlvxGq8f/1jTBynf+8ihbfhmAPSBIAfAwtgsfK0MxxWv+x45dMx9YBtVvQajYyfVmDWteu3/juAFwJkhyAHQeePQNVodbjgOUi0HHj98bNjaoPJ1dMKNjdufBq9vGG4IQE8IcgCcMbMTaEyf5/XQE0eOqXZNK2ODdWFs9XPrAtjByZTyXz1gZkMAFpMgB8CGWktaWg4uD8YBa3g0XK0NXqMMpxNzrAtnswFs+jyvLz/wxA59IwDYPQQ5gF1oOhRxGqjWh7Cj6+NhioPh2irYYNjy+OGVJMlt9zy6w98GAFhPkAPomNlngA3WVLhGqyFr9r3p9ieODNJa8pm7HzruRBwAQP8JcgBnwGA4OiaMrd92aGWY1pIv3vvoMaHtdIwmnxPiAGD3E+QANjBaV/EaDMfr0wczf+OhJ7MybMcOYdxkco6NrAxGSZLHDw/O7JcBAHYdQQ7YtVprqxN2PH54ZTWMrQ9nw8lzxE6mKjZ9MPN9jxzezq8CALCGIAd03vrgNZiErpXJpB0rk4rYNIxNK2XD0dEJO75472M7/C0AAOZHkAO2zWg0Dl2zgWwwOrq8Mmw5uDxMay233vPIeEbF4eYPcQYAWFSCHHBaVoaj1dkTW0sOPH54HNCGk+rZugrZYDg6qUA2GI7vGzu0PDzD3wAAoL8EOWAmcI3D2WrVbGZ5ZVo9m7xOJ/OYPuT5qwee3MFvAACwWAQ52GVaa1kerg1iqwFtOMqh5WFGk6GL60MZAAD9IMhBx41GLaPJ7IuPHFzOyvDYytnKcDRTMTt+KlsxdBEAoPcEOdgB04k9VkajrAxGq+FsZbh2eRrMnpg8Z+yO+x/f4Z4DANAFghzMyWoQG4yDWGtHHxo9raCNhzye3KQfAACwGUEOjmM0Goev2UrZ8mC8vjwJbRuFs+mwRQ+NBgDgTBDkWEjTgHZkGsoGo4xay1ceeHwS3I5W0QAAoGsEOXaV1sbPLFuerZwN1ga25Q0C2uGVcQXt208s70S3AQDglAhy9EZrLUcmQWxlMpTxa99+cjWwTcOaqfQBANjtBDk6Y7Z6Nn4drq6vD2nTe9C+9ah70AAAWDyCHNuitfGz0EYtOfD44XWBbfw6NJUjAACcFEGOuRmOWg6vDHN4ZZgjg9FkeVxZOzIYrT4L7asHntzhngIAQL8JcpySI4OZcLYyWhPaVszwCAAA20KQ4xjLg1EOrQxXp+T/D996bBzeVoYeZA0AAB0gyC2waWA7tDzMweXB6vK0sjadkv87T67sZDcBAIB1BLkFcKLABgAA9Isgt4u0lgxby7cePSywAQDALibI9dB0ZsiDy8PVsHZoZZjHD4+HQH7t22aFBACA3UyQ66jWWg6vjIdEHlwe5PDKMIeWx+uetwYAAItNkOuAJ48M1lTWDi2PK27yGgAAsBFBbpsMhpMJR2YC2xNHBhm1ls/vf3SnuwcAAPSIIHeGPH54JQ89sbx6H9vyYHTMPiMlNwAA4DQIcmfIIwdXcv+jh3e6GwAAwC60Z6c7AAAAwKkR5AAAAHpmLkGuqq6qqjur6q6qetsG71dV/dbk/c9X1Q/N47gAAACLaMtBrqqWkrw7ydVJXpTk1VX1onW7XZ3k+ZOfa5L8zlaPCwAAsKiqta3NnFhVP5zkHa21n5is/2KStNb+ycw+/yLJJ1trH5is35nkytba/cdr+5nf+8L28rdfv6X+zdutt92aJLnixVccd78jg1GODIbH3ecrX/pikuT5L/qBufRtkdrrct+63l6X+7Zo7XW5b11vr8t963p7Xe5b19vrct8Wrb0u963r7XW5bzvZ3jl7l3LO3m7edfahN//nt7TW9q3fPo8g97eTXNVae9Nk/TVJXtpae8vMPh9N8s7W2p9P1j+R5K2ttZs3aO+ajKt2edpzvu8vvuJX37+l/u2UkwlyAADAzutjkJvH4wdqg23r0+HJ7DPe2Np1Sa5Lkn379rU/+ns/vLXe7ZB7Hj6Y/d85tNPdAAAATuDiC87NZd993k53Y0MfevPG2+cRO/cnuXRm/ZIk953GPgAAAJyEeQS5m5I8v6qeV1VnJ3lVkhvW7XNDktdOZq/8S0kePdH9cQAAAGxsy0MrW2uDqnpLko8nWUpyfWvt9qp68+T9a5PcmOQVSe5KcjDJG7Z6XAAAgEU1j3vk0lq7MeOwNrvt2pnlluTn5nEsAACARdfNqVl2gaectZSnnLUntdE0LwAAAFswl4ocx7ro/HNy0fnnZDRqOTwY5uDyMIeWhzm8Ml4+vDLMaGtPfgAAABaUIHeG7dlTOe/svTnv7LX/qVtrOTIYjQPeyjjkHZosDyU8AADgOAS5HVJVk+GXS8e8d2RwNNQdWj5awVsZCngAAIAg10njJ8sv5YJ121eGo9VQNw14h1aGWR6MdqSfAADAzhDkeuSspT15xrl78oxzz1qzfTAcjat3K8McXh7l4MogB5eHObIi4AEAwG4kyO0Ce5f25PylPTn/KWsD3nDUcmhlmIPLAwEPAAB2EUFuF1vaU3naOXvztHPWnmYBDwAA+k2QW0CnEvAOLY/vwTORJgAAdIcgx6rNAt70UQlHBqMcWRnm8MooRwbj18ODYQZm0wQAgG0lyHFCax6VsG6ilWQ82crhwSiHV4Y5MnmdLi8PRmlyHgAAzJUgx5btXdqTpy3tOaaSlySj0bSad7SStzyt7g1GWRkKegAAcKoEOc6oPXsq5569lHPPPvbB58naYZvLg1GWh+Phm+PX8bqhmwAAsJYgx45aM2xzE4PhaE2wG78eHbppMhYAABaNIEfn7V3ak71Le3Le2ZvvszIcD9NcnqnsTV9XBi3Lw2FWhs0wTgAAdgVBjl3hrKU9OesEYa+1djTgDUZZGbZJ2BtmeTB+b8VQTgAAekCQY2FUVc7Zu5Rz9m4+jDMZT9AyDXUrw7Za6ZtdPzIYZTA0pBMAgJ0hyME6e/ZUnrLn+PftTU2HdK4MZsPfdJjnOPQNRiPDOgEAmCtBDrZgOqQzxxnSmYyHdQ5GbTX0rYxGxywPhpMwaPIWAABOQJCDbVBVOWupTir0JclwEvqmj19Ymbl/bzrEc1rp8yw+AIDFI8hBBy3tqSyd5PDOZPyIhtWK37BlMBxlZTR5Xd02qf6p+AEA9J4gB7vA+BENOengN634DVbD3rjCN634TUPheL9mYhcAgI4R5GABTSt+p+JE4W819E2GfA5H4x8AAOZPkANOyumEv9FoPJxztrI3HLXVYZ/jUHg0CJrhEwDg5AhywBmzZ0/lnFMMf8nR6t9wEvQGo0nomwmAw3XDP1UAAYBFIsgBnXM61b9kXAEcjGaC3yQEHg2ELcM1703Wh809gABArwhywK6xZ0/l7D2Vs7PnlD87Hdo5XK38CYEAQHcJcgA5/SpgcrQSuD4MrhkiOn1/eOx+7gkEAE6VIAewRdNK4Njp3RO4JgAONw6GawOjiiAALDJBDmCHbaUamGxcERyuCXxHXzcLh6qCANAvghxAz221IpjkhCFwNFMFHI6y4b7CIABsH0EOgC1XBZO1lcFhmw4RHY2XJ/cHjtra6uBsEBxN9hMIAeDEBIpAlCwAAAjNSURBVDkA5mJtZfD0bVQdHM4ExNlAuCY8TgPkSCAEYPcT5ADolHlUB5OsBrrNqoDTquE4/I2HjG4UCD1oHoAuEuQA2JXGgXDrFcLkaJVwNLk/cDTKuDo4s3w0FLYNQ6ShowDMkyAHACewtkq4tWrhhiFvgyrgtDI4fW86pPRoKIxqIcACE+QAYBvNs1LY2mzgy4YVwc2C4ThATiqMgiFA7whyANBTVZW9SzXXX+bHhMCZYaPrQ+CwnSAwjjywHuBMEeQAgFXzrBgmm1cNR5tUCDcKh4aUAhxrS0Guqp6Z5I+SXJ7k60l+urX2nXX7XJrkfUm+J8koyXWttd/cynEBgH44E1XDZOPK4Wh0bGgcbRIEN/qcfAj0yVb/XX1bkk+01t5ZVW+brL913T6DJP+wtfa5qjo/yS1V9WettS9t8dgAwIKad+UwGVcPR23zoDds41lHN6sUjtrm281WCszbVoPcK5NcOVl+b5JPZl2Qa63dn+T+yfLjVXVHkouTCHIAQGdUVZYqcw+ISdYOIZ0ZSjpqxw4xXQ2LbXYIao4Jl9PQCSymrQa5Z0+CWlpr91fVs463c1VdnuQHk3z2OPtck+SaJLnsssu22D0AgJ23Z09lTypnbf1Z98eYrRAeLySOWtZVF9cONZ2tJE5DokoidNcJg1xV/fuM729b75dO5UBV9bQkH07yC621xzbbr7V2XZLrkmTfvn3++QAAOI4zMcx06mgInLnvcN19hbMT07SWmcrixlXGaYA05BS25oRBrrX21zd7r6oeqKrnTKpxz0lyYJP9zso4xP1Ba+0jp91bAAC2zbSSeKYcM6T0OEFxGiZnK42z9zQe3S+rbRh5ym621aGVNyR5XZJ3Tl7/eP0OVVVJfjfJHa21X9/i8QAA2CXO5JDT5OjjL1bD3UZDTzeY5fTo0NK19ygafkqXbDXIvTPJh6rqjUm+meTvJElVPTfJe1prr0jyI0lek+QLVXXr5HNvb63duMVjAwDApqaPvziTNg2Lk3sSp4GxzVYPZ4Lk7FDTDYenqiyyiS0FudbaQ0letsH2+5K8YrL858kZrMkDAMAO2Y6wmBx7v+JqEFx37+LscNT1w1Vnh6Ju9J77Fvtl3s/nBAAA5uxM3684daIq4onemx12Olq/X5utVJ7xr7LrCXIAAECSo4HxTIeE1javHrZR0rK2SthaS8vR9aPbJ8vZeN/RpK3j7TtqSfVw/KAgBwAAbKuqylIlS2dwspvdbs9OdwAAAIBTI8gBAAD0jCAHAADQM4IcAABAzwhyAAAAPSPIAQAA9IwgBwAA0DOCHAAAQM8IcgAAAD0jyAEAAPSMIAcAANAzghwAAEDPCHIAAAA9I8gBAAD0TLXWdroPm6qqB5N8Y6f7sYELk3x7pzuB89AhzkV3OBfd4Vx0h3PRHc5FNzgP3XEy5+J7W2sXrd/Y6SDXVVV1c2tt3073Y9E5D93hXHSHc9EdzkV3OBfd4Vx0g/PQHVs5F4ZWAgAA9IwgBwAA0DOC3Om5bqc7QBLnoUuci+5wLrrDuegO56I7nItucB6647TPhXvkAAAAekZFDgAAoGcEuVNQVVdV1Z1VdVdVvW2n+7PIqurrVfWFqrq1qm7e6f4skqq6vqoOVNUXZ7Y9s6r+rKq+Mnn9rp3s46LY5Fy8o6runVwbt1bVK3ayj4ugqi6tqv+jqu6oqtur6ucn210X2+w458J1sc2q6ilV9f9W1W2Tc/Frk+2ui212nHPhutgBVbVUVf9fVX10sn7a14ShlSepqpaSfDnJy5PsT3JTkle31r60ox1bUFX19ST7WmuegbLNquovJ3kiyftaaz8w2fY/JXm4tfbOyf/k+K7W2lt3sp+LYJNz8Y4kT7TW/ued7NsiqarnJHlOa+1zVXV+kluS/M0kr4/rYlsd51z8dFwX26qqKslTW2tPVNVZSf48yc8n+am4LrbVcc7FVXFdbLuq+m+T7Evy9NbaT27lbygVuZP3kiR3tdbubq0tJ/lgklfucJ9g27XWPpXk4XWbX5nkvZPl92b8hxNn2Cbngm3WWru/tfa5yfLjSe5IcnFcF9vuOOeCbdbGnpisnjX5aXFdbLvjnAu2WVVdkuS/SPKemc2nfU0Icifv4iT3zKzvj18OO6kl+dOquqWqrtnpzpBnt9buT8Z/SCV51g73Z9G9pao+Pxl6adjSNqqqy5P8YJLPxnWxo9adi8R1se0mQ8huTXIgyZ+11lwXO2STc5G4LrbbbyT575KMZrad9jUhyJ282mCb/5uxc36ktfZDSa5O8nOTIWZA8jtJvi/JFUnuT/Kune3O4qiqpyX5cJJfaK09ttP9WWQbnAvXxQ5orQ1ba1ckuSTJS6rqB3a6T4tqk3PhuthGVfWTSQ601m6ZV5uC3Mnbn+TSmfVLkty3Q31ZeK21+yavB5L8m4yHvrJzHpjcmzK9R+XADvdnYbXWHpj8wh4l+d/i2tgWk/tOPpzkD1prH5lsdl3sgI3OhetiZ7XWHknyyYzvyXJd7KDZc+G62HY/kuRvTOZ5+GCSv1ZVv58tXBOC3Mm7Kcnzq+p5VXV2klcluWGH+7SQquqpk5vYU1VPTfLjSb54/E9xht2Q5HWT5dcl+eMd7MtCm/4ymPhbcW2ccZOJBH43yR2ttV+fect1sc02Oxeui+1XVRdV1QWT5XOT/PUk/yGui2232blwXWyv1tovttYuaa1dnnGO+N9ba/9VtnBN7J17L3ep1tqgqt6S5ONJlpJc31q7fYe7taieneTfjH9fZ2+SP2ytfWxnu7Q4quoDSa5McmFV7U/yq0nemeRDVfXGJN9M8nd2roeLY5NzcWVVXZHx0O+vJ/l7O9bBxfEjSV6T5AuTe1CS5O1xXeyEzc7Fq10X2+45Sd47mfV7T5IPtdY+WlWfjutiu212Lt7vuuiE0/5d4fEDAAAAPWNoJQAAQM8IcgAAAD0jyAEAAPSMIAcAANAzghwAAEDPCHIAAAA9I8gBAAD0jCAHAADQM/8/t0BvPnkGkdsAAAAASUVORK5CYII= ",
text/plain: [
"<Figure size 1080x432 with 1 Axes>"
]
},
metadata: {
needs_background: "light"
},
output_type: "display_data"
},
{
data: {
image/png: "iVBORw0KGgoAAAANSUhEUgAAA2oAAAF1CAYAAACZNBlsAAAABHNCSVQICAgIfAhkiAAAAAlwSFlzAAALEgAACxIB0t1+/AAAADh0RVh0U29mdHdhcmUAbWF0cGxvdGxpYiB2ZXJzaW9uMy4yLjIsIGh0dHA6Ly9tYXRwbG90bGliLm9yZy+WH4yJAAAgAElEQVR4nO3dfbRdZ30f+O9PV5aRX4QAy45fFOyA67FhBYVR7dI0jRuGxCYvpqxMasjwtpxxPIPbJJO1CqGUkDZlMtOhoRQaL09wACfEoYGmDstAUhiXJk0Yy454sT3GisFIyFiysWJsy5Z17zN/3HPh+vpKupL2veeRz+ez1l33nn322ed3zrOfffb37GfvW621AAAA0I9V4y4AAACApxLUAAAAOiOoAQAAdEZQAwAA6IygBgAA0BlBDQAAoDOCGgCDq6pHqur7ljDf2VXVqmr1StTVq6p6Y1X92VE8/pNV9YYhawJgvAQ1gAlUVV+rqr2jQHV/Vf1OVZ10hMu6uap+bv601tpJrbV7hqn2O8/xUFUdf5iPa1X1wqHq6EFVvbOqfnf+tNbapa21D42rJgCGJ6gBTK6fbK2dlOSlSf52krcfzoNr1rJ/jlTV2Ul+KElL8lPL/XxHa7Gjg5N+xBCAwyeoAUy41to3knwyyYur6jlV9Ymq2j06gvWJqjprbt7Rka1/VVV/nuSxJNdnNkS9b3R07n2j+b5zJKuqfryq/qqqHq6q7VX1zsMs8fVJ/jLJB5M8ZXjfwqN584cQVtXnRpO/MKrtH42m/89Vta2qvlVVN1bVGfMe/6Kq+tPRffdX1dtG04+vqvdU1c7Rz3vmju5V1cVVtaOq3lJV30zyO6OjXn9YVb9bVQ8neWNVPbuqPlBV91XVN6rq16tqarEXXFX/dvRePVxVt1bVD42mX5LkbUn+0eg1fWHh+1BVq6rq7VV1b1XtqqoPV9WzR/fNDTV9Q1V9vaoeqKp/dpjtAcAKENQAJlxVbUzyyiR/ldnPhd9J8vwk35tkb5L3LXjI65JcmeTkJG9M8l+TXD0a7nj1Ik/xaGbD1vokP57kf6mqVx1Gia9P8nujnx+rqtOW8qDW2t8f/fmSUW1/UFU/kuR/T/IzSU5Pcm+SG5Kkqk5O8p+TfCrJGUlemOQzo2X8syR/J8mmJC9JcmGeegTye5I8N7Pv25WjaZcl+cPR6/69JB9Ksn+03B9I8qNJnjJkdJ5bRs/13CQfSfIfqupZrbVPJXlXkj8YvaaXLPLYN45+/kGS70tyUp7ehn8vyXlJXp7kHVV1/gHqAGBMBDWAyfVHVbUnyZ8l+S9J3tVae7C19rHW2mOttW8n+VdJfnjB4z7YWru9tba/tfbkoZ6ktXZza+1LrbWZ1toXk/z+IstcVFX9vcyGn4+21m5N8tdJXrv0l/g0P5vkutbaba21J5L8SpKXjYZX/kSSb7bW3t1ae7y19u3W2ufnPe5ftNZ2tdZ2J/m1zAbWOTNJfrW19kRrbe9o2l+01v6otTaTZF2SS5P8Ymvt0dbariS/meTyxYpsrf3uqC32t9beneT4zAarpb7Gf9Nau6e19sjoNV6+YPjlr7XW9rbWvpDkC5kNnwB0xJh5gMn1qtbaf54/oapOyGyAuCTJc0aTT66qqdba9Oj29sN5kqq6KMlvJHlxkjWZDR3/YYkPf0OSP2mtPTC6/ZHRtN88nBrmOSPJbXM3WmuPVNWDSc5MsjGzQfBAj7t33u17R9Pm7G6tPb7gMfPfp+cnOS7JfVU1N21VDvBeVtUvZ/Zo2xmZPTdvXZJTDviqDl3r6iTzj0R+c97fj2X2qBsAHXFEDYD5fjmzR24uaq2tSzI3fLDmzdMWPGbh7YU+kuTGJBtba89Ocs2C5S2qqtZmdojiD1fVN0fnf/1SkpdU1dwRoEeTnDDvYd9ziMXuzGxomnuOE5M8L8k3MhuaXrCUx2V2WOjOebcXew/mT9ue5Ikkp7TW1o9+1rXWXrTwQaPz0d6S2df+nNba+iR/k+++Z4d6vxerdX+S+w/xOAA6IqgBMN/JmT0vbU9VPTfJry7hMfdn9lyogy3zW621x6vqwix96OKrkkwnuSCz52ttSnJ+Zs+Je/1onq1JXl1VJ4wuXnLFIWr7SJI3VdWm0cVA3pXk8621ryX5RJLvqapfHF085OTR0cBkdrjm26tqQ1WdkuQdSZ5yifyDaa3dl+RPkry7qtaNLvjxgqpabAjoyZkNVruTrK6qd2T2iNr813T2Qa64+ftJfqmqzqnZf7kwd07b/qXWC8D4CWoAzPeeJGuTPJDZKy1+agmP+bdJfnp0lcj3LnL//5rkX1TVtzMbcD66xFrekOR3Wmtfb619c+4nsxfG+NnROVe/mWRfZsPLhzJ70Y753pnkQ1W1p6p+prX2mST/PMnHktyX2SNolyfJ6Jy8VyT5ycwODbw7sxfkSJJfT7IlyReTfCmzwyd/fYmvY87rMzv0844kD2X2QiOnLzLfpzN7Fc6vZHbY4uN56hDJuWGjD1bVbXm66zJ7Nc7PJfnq6PH/+DBrBWDMqrVDjaAAAABgJTmiBgAA0BlBDQAAoDOCGgAAQGcENQAAgM4IagAAAJ1ZPa4nPuWUU9rZZ589rqcHAAAYq1tvvfWB1tqGxe4bW1A7++yzs2XLlnE9PQAAwFhV1b0Hus/QRwAAgM4IagAAAJ0R1AAAADojqAEAAHRGUAMAAOiMoAYAANAZQQ0AAKAzghoAAEBnBDUAAIDOHDKoVdV1VbWrqr58gPurqt5bVduq6otV9dLhy1x+0zMtn7nz/rz3M3fnM3fen+mZNu6SAACACbV6CfN8MMn7knz4APdfmuTc0c9FSX5r9PuYMT3T8roPfD5bt+/J3n3TWbtmKps2rs/1V1yUqVV1xMu8+a5duX3nw3nRGety8XmnHvGyAACAyXLIoNZa+1xVnX2QWS5L8uHWWkvyl1W1vqpOb63dN1CNy+7mu3Zl6/Y9eWzfdJLksX3T2bp9T26+a1defv5ph7285Qh+AADA5BjiHLUzk2yfd3vHaNrTVNWVVbWlqrbs3r17gKcexu07H87eUUibs3ffdO7Y+fARLW9+8Gt5avADAAA4lCGC2mKHiBY9wau1dm1rbXNrbfOGDRsGeOphvOiMdVm7Zuop09aumcoFZ6w7ouUNHfwAAIDJMkRQ25Fk47zbZyXZOcByV8zF552aTRvXp6b3JW0mJ4yGKl583qlHtLyhgx8AADBZhghqNyZ5/ejqj38nyd8cS+enJcnUqsr1V1yUDXf/cdbv+PP8u9f8wFGdTzZ08AMAACbLIS8mUlW/n+TiJKdU1Y4kv5rkuCRprV2T5KYkr0yyLcljSd60XMUup6lVlRP23JMT9txzRBcQWbis66+4KC979RXZd+Kpeffbf8lVHwEAgCVbylUfX3OI+1uSNw9W0TPEkMEPAACYLEMMfQQAAGBAghoAAEBnBDUAAIDOCGoAAACdEdQAAAA6I6gBAAB0RlADAADojKAGAADQGUENAACgM4IaAABAZwQ1AACAzghqAAAAnRHUAAAAOiOoAQAAdEZQAwAA6IygBgAA0BlBDQAAoDOCGgAAQGcENQAAgM4IagAAAJ0R1AAAADojqAEAAHRGUAMAAOiMoAYAANAZQQ0AAKAzghoAAEBnBDUAAIDOCGoAAACdEdQAAAA6I6gBAAB0RlADAADojKAGAADQGUENAACgM4IaAABAZwQ1AACAzghqAAAAnRHUAAAAOiOoAQAAdEZQAwAA6IygBgAA0BlBDQAAoDOCGgAAQGcENQAAgM4IagAAAJ0R1AAAADojqAEAAHRGUAMAAOjMkoJaVV1SVXdV1baqeusi9z+7qv64qr5QVbdX1ZuGLxUAAGAyHDKoVdVUkvcnuTTJBUleU1UXLJjtzUnuaK29JMnFSd5dVWsGrhUAAGAiLOWI2oVJtrXW7mmt7UtyQ5LLFszTkpxcVZXkpCTfSrJ/0EoBAAAmxFKC2plJts+7vWM0bb73JTk/yc4kX0ryC621mUEqBAAAmDBLCWq1yLS24PaPJdma5Iwkm5K8r6rWPW1BVVdW1Zaq2rJ79+7DLhYAAGASLCWo7Uiycd7tszJ75Gy+NyX5eJu1LclXk/x3CxfUWru2tba5tbZ5w4YNR1ozAADAM9pSgtotSc6tqnNGFwi5PMmNC+b5epKXJ0lVnZbkvCT3DFkoAADApFh9qBlaa/ur6uokn04yleS61trtVXXV6P5rkvzLJB+sqi9ldqjkW1prDyxj3QAAAM9YhwxqSdJauynJTQumXTPv751JfnTY0gAAACbTkv7hNQAAACtHUAMAAOiMoAYAANAZQQ0AAKAzghoAAEBnBDUAAIDOCGoAAACdEdQAAAA6I6gBAAB0RlADAADojKAGAADQGUENAACgM4IaAABAZwQ1AACAzghqAAAAnRHUAAAAOiOoAQAAdEZQAwAA6IygBgAA0BlBDQAAoDOCGgAAQGcENQAAgM4IagAAAJ0R1AAAADojqAEAAHRGUAMAAOiMoAYAANAZQQ0AAKAzghoAAEBnBDUAAIDOCGoAAACdEdQAAAA6I6gBAAB0RlADAADojKAGAADQGUENAACgM4IaAABAZwQ1AACAzghqAAAAnRHUAAAAOiOoAQAAdEZQAwAA6IygBgAA0BlBDQAAoDOCGgAAQGcENQAAgM4IagAAAJ0R1AAAADqzpKBWVZdU1V1Vta2q3nqAeS6uqq1VdXtV/ZdhywQAAJgcqw81Q1VNJXl/klck2ZHklqq6sbV2x7x51if590kuaa19vapOXa6CAQAAnumWckTtwiTbWmv3tNb2JbkhyWUL5nltko+31r6eJK21XcOWCQAAMDmWEtTOTLJ93u0do2nz/a0kz6mqm6vq1qp6/WILqqorq2pLVW3ZvXv3kVUMAADwDLeUoFaLTGsLbq9O8t8n+fEkP5bkn1fV33rag1q7trW2ubW2ecOGDYddLAAAwCQ45DlqmT2CtnHe7bOS7Fxkngdaa48mebSqPpfkJUm+MkiVAAAAE2QpR9RuSXJuVZ1TVWuSXJ7kxgXz/KckP1RVq6vqhCQXJblz2FIBAAAmwyGPqLXW9lfV1Uk+nWQqyXWttdur6qrR/de01u6sqk8l+WKSmSS/3Vr78nIWDgAA8Ey1lKGPaa3dlOSmBdOuWXD7Xyf518OVBgAAMJmW9A+vAQAAWDmCGgAAQGcENQAAgM4IagAAAJ0R1AAAADojqAEAAHRGUAMAAOiMoAYAANAZQQ0AAKAzghoAAEBnBDUAAIDOCGoAAACdEdQAAAA6I6gBAAB0RlADAADojKAGAADQGUENAACgM4IaAABAZwQ1AACAzghqAAAAnRHUAAAAOiOoAQAAdEZQAwAA6IygBgAA0BlBDQAAoDOCGgAAQGcENQAAgM4IagAAAJ0R1AAAADojqAEAAHRGUAMAAOiMoAYAANAZQQ0AAKAzghoAAEBnBDUAAIDOCGoAAACdEdQAAAA6I6gBAAB0RlADAADojKAGAADQGUENAACgM4IaAABAZwQ1AACAzghqAAAAnRHUAAAAOiOoAQAAdEZQAwAA6MySglpVXVJVd1XVtqp660Hm+9tVNV1VPz1ciQAAAJPlkEGtqqaSvD/JpUkuSPKaqrrgAPP9H0k+PXSRAAAAk2QpR9QuTLKttXZPa21fkhuSXLbIfP84yceS7BqwPgAAgImzlKB2ZpLt827vGE37jqo6M8k/THLNcKUBAABMpqUEtVpkWltw+z1J3tJamz7ogqqurKotVbVl9+7dS60RAABgoqxewjw7kmycd/usJDsXzLM5yQ1VlSSnJHllVe1vrf3R/Jlaa9cmuTZJNm/evDDsAQAAkKUFtVuSnFtV5yT5RpLLk7x2/gyttXPm/q6qDyb5xMKQBgAAwNIcMqi11vZX1dWZvZrjVJLrWmu3V9VVo/udlwYAADCgpRxRS2vtpiQ3LZi2aEBrrb3x6MsCAACYXEv6h9cAAACsHEENAACgM4IaAABAZwQ1AACAzghqAAAAnRHUAAAAOiOoAQAAdEZQAwAA6IygBgAA0BlBDQAAoDOCGgAAQGcENQAAgM4IagAAAJ0R1AAAADojqAEAAHRGUAMAAOiMoAYAANAZQQ0AAKAzghoAAEBnBDUAAIDOCGoAAACdEdQAAAA6I6gBAAB0RlADAADojKAGAADQGUENAACgM4IaAABAZwQ1AACAzghqAAAAnRHUAAAAOiOoAQAAdEZQAwAA6IygBgAA0BlBDQAAoDOCGgAAQGcENQAAgM4IagAAAJ0R1AAAADojqAEAAHRGUAMAAOiMoAYAANAZQQ0AAKAzghoAAEBnBDUAAIDOCGoAAACdEdQAAAA6I6gBAAB0RlADAADozJKCWlVdUlV3VdW2qnrrIvf/bFV9cfTz36rqJcOXCgAAMBkOGdSqairJ+5NcmuSCJK+pqgsWzPbVJD/cWvv+JP8yybVDFwoAADAplnJE7cIk21pr97TW9iW5Icll82dorf231tpDo5t/meSsYcsEAACYHEsJamcm2T7v9o7RtAO5IsknF7ujqq6sqi1VtWX37t1LrxIAAGCCLCWo1SLT2qIzVv2DzAa1tyx2f2vt2tba5tba5g0bNiy9SgAAgAmyegnz7Eiycd7ts5LsXDhTVX1/kt9Ocmlr7cFhygMAAJg8SzmidkuSc6vqnKpak+TyJDfOn6GqvjfJx5O8rrX2leHLBAAAmByHPKLWWttfVVcn+XSSqSTXtdZur6qrRvdfk+QdSZ6X5N9XVZLsb61tXr6yAQAAnrmWMvQxrbWbkty0YNo18/7+uSQ/N2xpAAAAk2lJ//AaAACAlSOoAQAAdEZQAwAA6IygBgAA0BlBDQAAoDOCGgAAQGcENQAAgM4IagAAAJ0R1AAAADojqAEAAHRGUAMAAOiMoAYAANAZQQ0AAKAzghoAAEBnBDUAAIDOCGoAAACdEdQAAAA6I6gBAAB0RlADAADojKAGAADQGUENAACgM4IaAABAZwQ1AACAzghqAAAAnRHUAAAAOiOoAQAAdEZQAwAA6IygBgAA0BlBDQAAoDOCGgAAQGcENQAAgM4IagAAAJ0R1AAAADojqAEAAHRm9bgLYDymZ1puvmtXbt/5cF50xrpcfN6pmVpV4y4LAACIoDaRpmdaXveBz2fr9j3Zu286a9dMZdPG9bn+iou6CGtCJAAAk05Qm0A337UrW7fvyWP7ppMkj+2bztbte3LzXbvy8vNPG2ttvYdIAABYCc5Rm0C373w4e0chbc7efdO5Y+fDY6rou+aHyJanhkgAAJgUgtoEetEZ67J2zdRTpq1dM5ULzlg3poq+q+cQCQAAK0VQm0AXn3dqNm1cn5rel7SZnDAaXnjxeaeOu7SuQyQAAKwUQW0CTa2qXH/FRdlw9x9n/Y4/z797zQ90cw5YzyESAABWiouJTKipVZUT9tyTE/bcc9QXEBnyKo1zIfJlr74i+048Ne9++y+56iMAABNHUOOoLMdVGocMkTAu/s0EHFv0WZbCesJKEtSO0F/89YOHnOfhvU8ued5xGKK+2+59KLfe+1Ce2D+TZPYqjbfe+1Cuufmv89LnP2estcG4zMy0vOuTd2bbrkeyb/9M1qxelReeelLedun5WeUDHbqjz7IU1pNj28te8Lxxl3DYnKPGUfnag49m3yikzdm3fyZfe/DRMVW0/GZmWm6796F8/LYdue3ehzIz08Zd0nf0XNsk2bp9T7bteiRP7J9JS/LE/pls2/VItm7fM+7S6Ezvfbb3+oaiz7IU1hNWmiNqHJWzn3di1qxe9Z0jakmyZvWqnP28E8dY1fLp+du0nms7FszMtGzdvidfe/DRnP28E7Np4/ojft8O9gXG0Rxp5pml9z7be31D0mdZCusJK80RNY7Kpo3r88JTT0r2z16l8fjRB/mmjevHXdqy6PnbtOWobVK+TZ/bIX3vZ+/OH966I+/97N151yfvPOLXO/cFxnzP5C8wODI9b0+S/usbkj7LUkziejIp+wG9WlJQq6pLququqtpWVW9d5P6qqveO7v9iVb10+FLp0apVlbdden5OuuOPsvar/zX/5EfO7e7b1iE3Mj0P9Ry6tqHDS8+G3iFdji8wfFg+8/S8PUn6r29I+ixLMWnrySTtB/TqkEMfq2oqyfuTvCLJjiS3VNWNrbU75s12aZJzRz8XJfmt0W8mwKpVlTUPbkse3JaXPv8t4y7nKYYeujP0UM8hh9sNXdv88JI8Nbw804Z4DD2cZe4LjJ//hV/O9Emn5eqrrjyqth16PR5yvePI9T50vPf6htR7n6UPx8J6MuT2fZL2A3q1lHPULkyyrbV2T5JU1Q1JLksyP6hdluTDrbWW5C+ran1Vnd5au2/wiuEwDL2Rmfs27favP5BMrc7xx60+4m/Tht5AD1lbsjxj8XsNCMuxQzrkFxhDrseTuAPZ63o3dJ8dWu/1Dd2uvfZZ+tLzejL09r33c/J63bYPqWaz1UFmqPrpJJe01n5udPt1SS5qrV09b55PJPmN1tqfjW5/JslbWmtbDrTc5z7//PaKt103wEsYztYvbE2SbHrJpkPO+/DjTx5ynrvv+HKS5NwLXnx0hS2TIesb+rUOtbzd334iDzyy72nTN5y0JqecfPwRLbO1lq9suyeZWpMzzjg9Jx0/larD3zB8+/H9+caevZnfBauSM9evzcnPOrLr/AxV23LU11rL17+1N3ufnE5rs8tae9xUvve5a4+4xqHM1fbYE08mqdSqGqS2Htfj5VrvHnliOo8/OZ1nHTd1VOvd0Hpe7+bqG6rPzi1vyLbotb7latce++ycIdu25z57LOh1PRl6+74cnxdDOZJtwLpnHbfCVS7NR6/6u7e21jYvdt9S3uXFXu3CdLeUeVJVVya5MklOOv0FS3jqlbWUgHY4hg5oQ4ehIesb+rUOtbxnHTeVqjxtI3P8cVNHvMyqynnnHv36+/ho4zJfa8kTT04f8QZwqNqS5KTjp7L2uKmnbQRPOv7I3rtHnpj+zrKS2de698npPPLEkb3eIXcgqyrf+9y1eeSJNXniyekcP9COS4/r8dDr3XKE3CHbduj1bs5Q2+Mh++xytEWv9S1Xu/bYZ5Nh37vev5hajuUN/YVDr+vJ0Nv3ofcDZusZpi2WaxvQm6W8kh1JNs67fVaSnUcwT1pr1ya5Nkk2b97c/uDnX3ZYxfZkHP+I+c0f+ZUkyTv+6Y0r/tzHqp6Hed1270N572fvfspwu+NXr8ob/+45XQwpSIYdVvDx23bkD2/d8dSJLXnZ9z0vr37pWYdd17s+eWdmnrU+mVqd3d9+Is9e20e7Loch1+Oh17u55aVmr03VWjI90/KT33/mUQ3dGapth1zv5utxezx0WwxtyPqWq12HMvRnz5Dv3XKtJ0P3iaGW1/PnxXKtJ0PuVwy5HzBkWxzJNqDXf3j90asOfN9SgtotSc6tqnOSfCPJ5Uleu2CeG5NcPTp/7aIkf+P8NHowd+Jvj2OY587/WLiB7uX8j2T2/Xvp858zyE7ekOeBzY3rz+o1SZ75538MuR4Pvd4NfQ7D0G07SRfE6P18kiHr671dh/7sGfK9W67zj/c974WZPum03HbvQ918ziZ9f14MvZ4sx37FkPsBQ7ZF79uAoRwyqLXW9lfV1Uk+nWQqyXWttdur6qrR/dckuSnJK5NsS/JYkjctX8lweIbcyAyp5xC5HIb8AOl9h3Q5DLUeD73eDf1hOXTbHgtfiAyl9x2XIes7Ftq11y+6luPqxe/65J155IJXJVOr897P3t3NyJWk/8+LIdeT3vcrhmyLY2EbMIQlDeJsrd2U2TA2f9o18/5uSd48bGnwzNdriFwOQ36A9L5D2rsh17uhPyyHbtved1yG1PuOy5D1TVK7JsO+d0OvJz0fsUom7/Oi5/2KIdtiUrYBz5yz7YDuDfUB0vsO6SSZtKE7Pet9x2Xo+ialXZNh37ueh2XOGXIopc+LfgzdFpOwDTjk5fmXy+bNm9uWLQe8en/3xnIxkdf+VJLk/R/p5+R1GJdJ+P8pk6r3tp2ZaYP9w1s41h3oAhb/5EfOPaoLCi38/329/BNojs4426LXi4lU1VFdnp9FjKOx1609bmzPDT36wXNPGXcJLJNe23Z6puV1H/h8Hn3Rq9JWrc77b96WTRvX5/orLsqUHT8m0IXnPDd//tcPZOv2Pdm7bzpr10xl08b1ueriFxxRn/jMnffnqw88+pShlF994NE8vn86Lz//tCOus9dtyiTSFksnqAHAEt18165s3b4nbWp2J/KxfdPZun1Pbr5r11HtRMKxampV5forLsrNd+3KHTsfzgVnrMvF5516xF9c3L7z4ezdN/2UaXv3TeeOnQ/rY0wcQQ0AlshOJDzd1KrKy88/bZA+8KIz1mXtmqk8Nq+frV0zlQvOWHfUy4ZjzapxFwAAx4q5ncj57ETCcC4+79Rs2rg+J6yZSiU5YTSU8uLzTh13abDiHFE7RkzPtDy2/vuy78TT8pk77z+qYQUAHJm5nciF5+PYiYRhDD2UEo5lrvp4DJg7ef0vvnJf2qrVOeH445y8DjAm0zPNTiQAg3DVx2Ock9cB+jHk+TgAcCDOUTsGHOzkdQAA4JlHUDsGOHkdAAAmi6B2DHAFJAAAmCzOUTsGuAISAABMFkHtGOHkdQAAmByGPgIAAHRGUAMAAOiMoAYAANAZQQ0AAKAzghoAAEBnBDUAAIDOCGoAAACdEdQAAAA6I6gBAAB0RlADAADoTLXWxvPEVbuT3DuWJz+4U5I8MO4iSKIteqIt+qEt+qEt+qAd+qEt+qEt+nGotnh+a23DYneMLaj1qqq2tNY2j7sOtEVPtEU/tEU/tEUftEM/tEU/tEU/jqYtDH0EAADojKAGAADQGUHt6a4ddwF8h7boh7boh7boh7bog3boh7boh7boxxG3hXPUAAAAOuOIGgAAQGcEtXmq6pKququqtlXVW8ddzySrqq9V1ZeqamtVbRl3PZOkqq6rql1V9eV5055bVX9aVXePfj9nnDVOggO0wzur6hujfrG1ql45zhonRVVtrKr/p6rurKrbq+oXRtP1ixV2kLbQN+VcxuYAAAO8SURBVFZYVT2rqv7fqvrCqC1+bTRdv1hBB2kHfWJMqmqqqv6qqj4xun3EfcLQx5GqmkrylSSvSLIjyS1JXtNau2OshU2oqvpaks2tNf8DZIVV1d9P8kiSD7fWXjya9n8m+VZr7TdGX2I8p7X2lnHW+Ux3gHZ4Z5JHWmv/1zhrmzRVdXqS01trt1XVyUluTfKqJG+MfrGiDtIWPxN9Y0VVVSU5sbX2SFUdl+TPkvxCkldHv1gxB2mHS6JPjEVV/W9JNidZ11r7iaPZh3JE7bsuTLKttXZPa21fkhuSXDbmmmDFtdY+l+RbCyZfluRDo78/lNkdI5bRAdqBMWit3ddau23097eT3JnkzOgXK+4gbcEKa7MeGd08bvTTol+sqIO0A2NQVWcl+fEkvz1v8hH3CUHtu85Msn3e7R2x8R+nluRPqurWqrpy3MWQ01pr9yWzO0pJTh1zPZPs6qr64mhopCFFK6yqzk7yA0k+H/1irBa0RaJvrLjREK+tSXYl+dPWmn4xBgdoh0SfGIf3JPmnSWbmTTviPiGofVctMs03EuPzg621lya5NMmbR8PAYNL9VpIXJNmU5L4k7x5vOZOlqk5K8rEkv9hae3jc9UyyRdpC3xiD1tp0a21TkrOSXFhVLx53TZPoAO2gT6ywqvqJJLtaa7cOtUxB7bt2JNk47/ZZSXaOqZaJ11rbOfq9K8l/zOzQVMbn/tG5IXPniOwacz0TqbV2/+gDeSbJ/x39YsWMzv34WJLfa619fDRZvxiDxdpC3xiv1tqeJDdn9rwo/WJM5reDPjEWP5jkp0bXWbghyY9U1e/mKPqEoPZdtyQ5t6rOqao1SS5PcuOYa5pIVXXi6CTxVNWJSX40yZcP/iiW2Y1J3jD6+w1J/tMYa5lYcxv6kX8Y/WJFjE7W/0CSO1tr/2beXfrFCjtQW+gbK6+qNlTV+tHfa5P8D0n+v+gXK+pA7aBPrLzW2q+01s5qrZ2d2Rzx2dba/5Sj6BOrB6/yGNVa219VVyf5dJKpJNe11m4fc1mT6rQk/3H28zirk3yktfap8ZY0Oarq95NcnOSUqtqR5FeT/EaSj1bVFUm+nuR/HF+Fk+EA7XBxVW3K7LDsryX5+bEVOFl+MMnrknxpdB5Ikrwt+sU4HKgtXqNvrLjTk3xodNXsVUk+2lr7RFX9RfSLlXSgdrhen+jGEX9WuDw/AABAZwx9BAAA6IygBgAA0BlBDQAAoDOCGgAAQGcENQAAgM4IagAAAJ0R1AAAADojqAEAAHTm/wfjaMI2k3x0/AAAAABJRU5ErkJggg== ",
text/plain: [
"<Figure size 1080x432 with 1 Axes>"
]
},
metadata: {
needs_background: "light"
},
output_type: "display_data"
}
],
source: [
"# ACF 그래프 ",
"# q에 적합한 값이 없어보여 q=0 ",
"plot_acf(ts_log) ",
"# PACF 그래프: p=1이 가장 적합해보임 ",
"# 2 이상인 구간에서는 PACF가 거의 0에 가까워지고 있어서 데이터 상관도가 0이라고 보면 됨. ",
"plot_pacf(ts_log) ",
"plt.show()"
]
},
{
cell_type: "code",
execution_count: 75,
metadata: { },
outputs: [
{
name: "stdout",
output_type: "stream",
text: [
"Results of Dickey-Fuller Test: ",
"Test Statistic -39.817874 ",
"p-value 0.000000 ",
"#Lags Used 3.000000 ",
"Number of Observations Used 5182.000000 ",
"Critical Value (1%) -3.431613 ",
"Critical Value (5%) -2.862098 ",
"Critical Value (10%) -2.567067 ",
"dtype: float64 "
]
},
{
data: {
image/png: "... ",
text/plain: [
"<Figure size 1080x432 with 1 Axes>"
]
},
metadata: {
needs_background: "light"
},
output_type: "display_data"
}
],
source: [
"# d차 차분을 보고 시계열이 안정된 형태인지 확인 ",
"diff_1 = ts_log.diff(periods=1).iloc[1:] ",
"diff_1.plot(title='Difference 1st') ",
" ",
"augmented_dickey_fuller_test(diff_1)"
]
},
{
cell_type: "code",
execution_count: 76,
metadata: { },
outputs: [
{
data: {
text/plain: [
"<matplotlib.legend.Legend at 0x7fcca32a0150>"
]
},
execution_count: 76,
metadata: { },
output_type: "execute_result"
},
{
data: {
image/png: "... ",
text/plain: [
"<Figure size 720x432 with 1 Axes>"
]
},
metadata: {
needs_background: "light"
},
output_type: "display_data"
}
],
source: [
"# 학습 테스트 9:1 ",
"train_data, test_data = ts_log[:int(len(ts_log)*0.9)], ts_log[int(len(ts_log)*0.9):] ",
"plt.figure(figsize=(10,6)) ",
"plt.grid(True) ",
"plt.plot(ts_log, c='r', label='training dataset') # train_data를 적용하면 그래프가 끊어져 보이므로 자연스러운 연출을 위해 ts_log를 선택 ",
"plt.plot(test_data, c='b', label='test dataset') ",
"plt.legend()"
]
},
{
cell_type: "code",
execution_count: 77,
metadata: { },
outputs: [
{
name: "stdout",
output_type: "stream",
text: [
"Date ",
"2000-01-04 8.717682 ",
"2000-01-05 8.626944 ",
"Name: Close, dtype: float64 ",
"(4668,) ",
"(519,) "
]
}
],
source: [
"print(ts_log[:2]) ",
"print(train_data.shape) ",
"print(test_data.shape)"
]
},
{
cell_type: "code",
execution_count: 78,
metadata: { },
outputs: [
{
name: "stdout",
output_type: "stream",
text: [
" ARIMA Model Results ",
"============================================================================== ",
"Dep. Variable: D.Close No. Observations: 4667 ",
"Model: ARIMA(1, 1, 0) Log Likelihood 10983.285 ",
"Method: css-mle S.D. of innovations 0.023 ",
"Date: Thu, 13 Aug 2020 AIC -21960.569 ",
"Time: 16:27:24 BIC -21941.224 ",
"Sample: 1 HQIC -21953.765 ",
" ",
"================================================================================= ",
" coef std err z P>|z| [0.025 0.975] ",
"--------------------------------------------------------------------------------- ",
"const 0.0004 0.000 1.205 0.228 -0.000 0.001 ",
"ar.L1.D.Close 0.0478 0.015 3.262 0.001 0.019 0.076 ",
" Roots ",
"============================================================================= ",
" Real Imaginary Modulus Frequency ",
"----------------------------------------------------------------------------- ",
"AR.1 20.9337 +0.0000j 20.9337 0.0000 ",
"----------------------------------------------------------------------------- "
]
},
{
name: "stderr",
output_type: "stream",
text: [
"/home/aiffel0036/anaconda3/envs/aiffel/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:218: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting. ",
" ' ignored when e.g. forecasting.', ValueWarning) ",
"/home/aiffel0036/anaconda3/envs/aiffel/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:218: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting. ",
" ' ignored when e.g. forecasting.', ValueWarning) "
]
}
],
source: [
"# Build Model ",
"model = ARIMA(train_data, order=(1, 1, 0)) ",
"fitted_m = model.fit(disp=-1) ",
"print(fitted_m.summary())"
]
},
{
cell_type: "code",
execution_count: 79,
metadata: { },
outputs: [
{
data: {
image/png: "... ",
text/plain: [
"<Figure size 1080x432 with 1 Axes>"
]
},
execution_count: 79,
metadata: { },
output_type: "execute_result"
},
{
data: {
image/png: "... ",
text/plain: [
"<Figure size 1080x432 with 1 Axes>"
]
},
metadata: {
needs_background: "light"
},
output_type: "display_data"
}
],
source: [
"fitted_m.plot_predict()"
]
},
{
cell_type: "code",
execution_count: 80,
metadata: { },
outputs: [
{
data: {
image/png: "... ",
text/plain: [
"<Figure size 1000x500 with 1 Axes>"
]
},
metadata: {
needs_background: "light"
},
output_type: "display_data"
}
],
source: [
"# Forecast : 결과가 fc에 담깁니다. ",
"fc, se, conf = fitted_m.forecast(len(test_data), alpha=0.05) # 95% conf ",
" ",
"# Make as pandas series ",
"fc_series = pd.Series(fc, index=test_data.index) # 예측결과 ",
"lower_series = pd.Series(conf[:, 0], index=test_data.index) # 예측결과의 하한 바운드 ",
"upper_series = pd.Series(conf[:, 1], index=test_data.index) # 예측결과의 상한 바운드 ",
" ",
"# Plot ",
"plt.figure(figsize=(10,5), dpi=100) ",
"plt.plot(train_data, label='training') ",
"plt.plot(test_data, c='b', label='actual price') ",
"plt.plot(fc_series, c='r',label='predicted price') ",
"plt.fill_between(lower_series.index, lower_series, upper_series, color='k', alpha=.10) ",
"plt.legend() ",
"plt.show()"
]
},
{
cell_type: "code",
execution_count: 83,
metadata: { },
outputs: [
{
name: "stdout",
output_type: "stream",
text: [
"MSE: 18517858.63179965 ",
"MAE: 3590.190001880987 ",
"RMSE: 4303.238156528134 ",
"MAPE: 7.68% "
]
}
],
source: [
"# 최종으로 오차 계산 ",
"mse = mean_squared_error(np.exp(test_data), np.exp(fc)) ",
"print('MSE: ', mse) ",
" ",
"mae = mean_absolute_error(np.exp(test_data), np.exp(fc)) ",
"print('MAE: ', mae) ",
" ",
"rmse = math.sqrt(mean_squared_error(np.exp(test_data), np.exp(fc))) ",
"print('RMSE: ', rmse) ",
" ",
"mape = np.mean(np.abs(np.exp(fc) - np.exp(test_data))/np.abs(np.exp(test_data))) ",
"print('MAPE: {:.2f}%'.format(mape*100))"
]
},
{
cell_type: "markdown",
metadata: { },
source: [
"# Kakao Stock Prediction"
]
},
{
cell_type: "code",
execution_count: 102,
metadata: { },
outputs: [
{
name: "stdout",
output_type: "stream",
text: [
"Date ",
"2000-02-11 273000.0 ",
"2000-02-14 247000.0 ",
"2000-02-15 217500.0 ",
"2000-02-16 243500.0 ",
"2000-02-17 243500.0 ",
"Name: Close, dtype: float64 ",
"(5159,) "
]
}
],
source: [
"filepath2 = os.getenv('HOME')+'/aiffel/stock_prediction/data/035720.KS.csv' ",
"df2 = pd.read_csv(filepath2, index_col='Date', parse_dates=True) ",
"ts2 = df2['Close'] ",
"print(ts2.head()) ",
"print(ts2.shape)"
]
},
{
cell_type: "code",
execution_count: 103,
metadata: { },
outputs: [
{
data: {
text/plain: [
"[<matplotlib.lines.Line2D at 0x7fccbbe8cd10>]"
]
},
execution_count: 103,
metadata: { },
output_type: "execute_result"
},
{
data: {
image/png: "... ",
text/plain: [
"<Figure size 1080x432 with 1 Axes>"
]
},
metadata: {
needs_background: "light"
},
output_type: "display_data"
}
],
source: [
"# 데이터 확인 ",
"rcParams['figure.figsize']=15,6 ",
" ",
"plt.plot(ts2)"
]
},
{
cell_type: "code",
execution_count: 104,
metadata: { },
outputs: [
{
name: "stdout",
output_type: "stream",
text: [
"Date ",
"2000-12-25 NaN ",
"2000-12-27 NaN ",
"2000-12-28 NaN ",
"2000-12-29 NaN ",
"2001-01-01 NaN ",
" .. ",
"2017-11-16 NaN ",
"2017-11-23 NaN ",
"2018-01-02 NaN ",
"2018-11-15 NaN ",
"2019-11-14 NaN ",
"Name: Close, Length: 114, dtype: float64 "
]
}
],
source: [
"# check the missing value ",
"print(ts2[ts2.isna()]) ",
"# interpolation ",
"ts2 = ts2.interpolate(method='time')"
]
},
{
cell_type: "code",
execution_count: 105,
metadata: { },
outputs: [
{
data: {
image/png: "... ",
text/plain: [
"<Figure size 1080x432 with 1 Axes>"
]
},
metadata: {
needs_background: "light"
},
output_type: "display_data"
}
],
source: [
"# 오르는 추세인데 딱히 규칙적 패턴은 없어보임. ",
"plot_rolling_statistics(ts2,window=12)"
]
},
{
cell_type: "code",
execution_count: 106,
metadata: { },
outputs: [
{
data: {
text/plain: [
"[<matplotlib.lines.Line2D at 0x7fccbbd5c890>]"
]
},
execution_count: 106,
metadata: { },
output_type: "execute_result"
},
{
data: {
image/png: "... ",
text/plain: [
"<Figure size 1080x432 with 1 Axes>"
]
},
metadata: {
needs_background: "light"
},
output_type: "display_data"
}
],
source: [
"# 로그 함수로 변환 ",
"ts_log2 = np.log(ts2) ",
"plt.plot(ts_log2)"
]
},
{
cell_type: "code",
execution_count: 107,
metadata: { },
outputs: [
{
name: "stdout",
output_type: "stream",
text: [
"Results of Dickey-Fuller Test: ",
"Test Statistic -1.671834 ",
"p-value 0.445680 ",
"#Lags Used 5.000000 ",
"Number of Observations Used 5153.000000 ",
"Critical Value (1%) -3.431620 ",
"Critical Value (5%) -2.862101 ",
"Critical Value (10%) -2.567069 ",
"dtype: float64 "
]
}
],
source: [
"# 제대로 변환되었는지 aumented dickey-fuller test 시행 ",
"# 아직 p-value보면 덜 변환됨. ",
"augmented_dickey_fuller_test(ts_log2)"
]
},
{
cell_type: "code",
execution_count: 110,
metadata: { },
outputs: [
{
data: {
image/png: "... ",
text/plain: [
"<Figure size 1080x432 with 4 Axes>"
]
},
metadata: {
needs_background: "light"
},
output_type: "display_data"
}
],
source: [
"# Time Series Decomposition ",
"decomposition2 = seasonal_decompose(ts_log2, model='multiplicative',period=30) ",
" ",
"trend2 = decomposition2.trend ",
"seasonal2 = decomposition2.seasonal ",
"residual2 = decomposition2.resid ",
" ",
"plt.subplot(411) ",
"plt.plot(ts_log2, label='Original') ",
"plt.legend(loc='best') ",
"plt.subplot(412) ",
"plt.plot(trend2, label='Trend') ",
"plt.legend(loc='best') ",
"plt.subplot(413) ",
"plt.plot(seasonal2,label='Seasonality') ",
"plt.legend(loc='best') ",
"plt.subplot(414) ",
"# trend와 seasonality를 제거하고 난 나머지 ",
"# Trend+Seasonality+Residual = Original ",
"plt.plot(residual2, label='Residuals') ",
"plt.legend(loc='best') ",
"plt.tight_layout()"
]
},
{
cell_type: "code",
execution_count: 111,
metadata: { },
outputs: [
{
data: {
image/png: "... ",
text/plain: [
"<Figure size 1080x432 with 1 Axes>"
]
},
metadata: {
needs_background: "light"
},
output_type: "display_data"
}
],
source: [
"plot_rolling_statistics(residual2)"
]
},
{
cell_type: "code",
execution_count: 112,
metadata: { },
outputs: [
{
name: "stdout",
output_type: "stream",
text: [
"Results of Dickey-Fuller Test: ",
"Test Statistic -1.770622e+01 ",
"p-value 3.518811e-30 ",
"#Lags Used 3.300000e+01 ",
"Number of Observations Used 5.095000e+03 ",
"Critical Value (1%) -3.431634e+00 ",
"Critical Value (5%) -2.862107e+00 ",
"Critical Value (10%) -2.567072e+00 ",
"dtype: float64 "
]
}
],
source: [
"# 이전에 직접 해준것보다 훨씬 유의미하게 나옴. ",
"residual2.dropna(inplace=True) ",
"augmented_dickey_fuller_test(residual2)"
]
},
{
cell_type: "code",
execution_count: 113,
metadata: { },
outputs: [
{
data: {
image/png: "iVBORw0KGgoAAAANSUhEUgAAA3IAAAF1CAYAAACtTdhwAAAABHNCSVQICAgIfAhkiAAAAAlwSFlzAAALEgAACxIB0t1+/AAAADh0RVh0U29mdHdhcmUAbWF0cGxvdGxpYiB2ZXJzaW9uMy4yLjIsIGh0dHA6Ly9tYXRwbG90bGliLm9yZy+WH4yJAAAgAElEQVR4nO3de5RsV30f+O/v3isJhACBJbDQw2IcwoC9Btm5A/HYTpQQbIk4UeIVO2APrwWjMMuw7EzWCsR2/JjJSpjJ4NgeE2sUUAzYRiaBxAojGxPNMIxnDEjCEiBkQBYgXUlIAknocR/dVbXnj6rqW13qvq+u7qrT9fms1avrnDq1zy4dDvd+72/vfaq1FgAAALpjz7w7AAAAwMkR5AAAADpGkAMAAOgYQQ4AAKBjBDkAAICOEeQAAAA6RpADgBmpqtdX1R9v4fN/UFWvm2WfANidBDkAdlRVfbyqHq6qM07iM62q/sJ29munVdUvVdVvT+5rrV3eWnvvvPoEQHcIcgDsmKq6OMkPJmlJ/vZcO3McVbXvRPYBwDwIcgDspNcm+WSS30qyNoRwVKV708T22hDFqvrEaPetVfV4Vf390f7/rqruqKqHquq6qnrexOe/q6o+Nnrv/qr62dH+M6rqV6vq3tHPr44rg1V1aVUdqKq3VdXXk/zbUdXs31fVb1fVo0leX1XPrKr3VNV9VXVPVf2zqtq70Zetql+rqrur6tGqurmqfnC0/7IkP5vk74++063T/x2qak9V/XxVfa2qHqiq91XVM0fvXTyqUr6uqu6qqm9U1c9t+eoA0BmCHAA76bVJfmf088NV9dzjfaC19ldGL1/SWjurtfZ7VfXXk/yLJD+e5LwkX0tybZJU1dOT/Ockf5jkeUn+QpIbRm38XJK/nOSSJC9J8tIkPz9xum9P8uwk35HkytG+K5L8+yRnj/r93iS9Ubvfk+SHkrwpG7txdK5nJ/ndJP+uqp7SWvvDJP88ye+NvtNLNvjs60c/fy3Jf5HkrCS/MXXMDyR5YZKXJ/mFqnrRJv0AYJcR5ADYEVX1AxkGpA+21m5O8udJfuIUm/vJJNe01j7TWjuS5J8k+b7R0M0fSfL11to7W2uHW2uPtdY+NfG5/7G19kBr7cEkv5zkNRPtDpL8YmvtSGvt0Gjfn7TW/mNrbZDkGUkuT/IzrbUnWmsPJPlXSV61USdba7/dWvtma63XWntnkjMyDF4n+h1/pbV2Z2vt8dF3fNXU8M5fbq0daq3dmuTWDMMpAEtAkANgp7wuyR+11r4x2v7dTAyvPEnPy7AKlyQZBZ1vJjk/yYUZhsTjfm70+nkT2w+21g5PfebuidffkeS0JPdV1SNV9UiS/z3JczY6WVX9o6q6vaq+NTr2mUnOOd6XO0Zf9yWZrGJ+feL1wQyrdgAsAZO2Adh2VfXUDIdB7h3NP0uG1amzq+olSZ5IcubER779OE3em2GoGrf/tCTfluSeDIPXq4/zudtG2xeN9o21DT4zue/uJEeSnNNa6x2rg6P5cG/LcNjjba21QVU9nKSOca6N+jp2UYZDOu9PcsFxPgvALqciB8BO+DtJ+klenOGcsUuSvCjJ/5PhvLlbkvxoVZ05eszAG6c+f3+G88TGfjfJG6rqktFiJf88yadaa19N8pEk315VPzNa3OTpVfWy0ec+kOTnq+rcqjonyS8kWfcIgGNprd2X5I+SvLOqnjFakOQ7q+qvbnD40zMMXg8m2VdVv5Dh0MzJ73RxVW32Z/EHkvzDqnp+VZ2Vo3PqjhkgAVgOghwAO+F1Sf5ta+2u1trXxz8ZLt7xkxnOM1vJMNy8N8NFRSb9UpL3joYz/nhr7YYk/zTJh5Lcl+Q7M5qn1lp7LMkrkvytDIcefjnDBUOS5J8luSnJZ5N8LslnRvtOxmuTnJ7kC0keznAhlPM2OO6jSf4gyZcyHBZ5OOuHaf670e9vVtVnNvj8NUnen+QTSb4y+vxbT7KvAOxS1drxRnYAAACwSFTkAAAAOkaQAwAA6BhBDgAAoGMEOQAAgI4R5AAAADpmoR8Ifs4557SLL7543t0AAACYi5tvvvkbrbVzp/cvdJC7+OKLc9NNN827GwAAAHNRVV/baL+hlQAAAB0jyAEAAHSMIAcAANAxghwAAEDHCHIAAAAdI8gBAAB0jCAHAADQMYIcAABAxwhyAAAAHTOTIFdV11TVA1X1+U3er6r69aq6o6o+W1XfO4vz7rT+oOWG2+/Pr9/w5dxw+/3pD9q8uwQAACyhfTNq57eS/EaS923y/uVJXjD6eVmS3xz97oz+oOU17/lUbrn7kRxa6eepp+/NJReenfe/8WXZu6dOqb2Pf/GB3Hbvo/mu5z0jl77wOafUDgAAsHxmEuRaa5+oqouPccgVSd7XWmtJPllVZ1fVea21+2Zx/p3w8S8+kFvufiQHV/pJkoMr/dxy9yP5+BcfyMtf9NyTamvWoXDcpmAIAADLYVYVueM5P8ndE9sHRvueFOSq6sokVybJRRddtCOdOxG33ftoDo1C3NihlX6+cO+jJx3kZhkKk24EQ0ETAABmZ6eC3EZ/Y99wgllr7eokVyfJ/v37F2YS2nc97xl56ul718JXkjz19L158fOecdJtzTIUJosfDA1LBQCA2dqpIHcgyYUT2xckuXeHzj0Tl77wObnkwrPzJ1+6L23Pvpx5xmm55MKzc+kLn3PSbc0yFCaLHwyXbViqoAkAwHbbqSB3XZK3VNW1GS5y8q0uzY9Lkr17Ku9/48vyfT/6xqw87Tl558//w1P+C/osQ2Gy+MFwmYaldiFoAgDQfTMJclX1gSSXJjmnqg4k+cUkpyVJa+2qJNcneWWSO5IcTPKGWZx3p+3dUznzkTtz5iN3nlJomGxnVqEwWfxguEzDUrsQNFUfAQC6b1arVr76OO+3JD81i3PtFrMKheO2FjkYLtOw1EUOmqqPAAC7x04NrWSbLXIwXKZhqYscNJet+jhuUzAEAHYjQY4NzTIYzrK9Zao+Jos9LHWRq4/J4gdDIRMA2ApBjs5ZlupjstjDUhe5+pgsdjBc9JAJACw+QY6lt6jVx3FbizosdZGrj8liB8NFDpmTbVoYBwAWlyAHC25Rh6UucvUxWexguMghM1n8hXGEQgAQ5GCpLEv1MVnsYLjIITNZ7IVxVB8BYEiQAxbGsgTDRQ6ZyWIvjLNs1cdxm4IhANMEOWDXWtRguMghM1nshXGWqfqYLP6wVCETYH4EOYATNOtguIghM1nshXGWqfqYLPawVNVHgPkS5AB2gUUOhou8+uoiVx+TxR6WuujVx3Gbnv0I7FaCHABPsqgL4yxT9TFZ7GGpi1x9TJbv2Y+CJiwfQQ6ATlmW6mOy2MNSF7n6mCzXsx+7EDSB2dsz7w4AwDyNg+HZ93wyL3/Rc7f8l9VZtjcOhud++T/l7AP/b/63V3/PKf/lfBwKq7+StEHOHP1lf6vVx1m1Nw6Gk7YrGM6zrWR9MGxZHwwXob1xMHzrB/40/+pjX8pbP/Cnec17PpX+oJ1Se+M2b7j9/vz6DV/ODbffvzBtQZepyAHAAlvUYamLXH1MluvZj4tczUwMc1XNZLsIcgCwJBZ17uO4rUUNhoscMrejvUUOhoscMrejvXGbFu1hI4IcALAQFjUYLnLI3I72FjkYLnLI3I72VDMFzWMR5ACAXWlZnv246EHTMNfFCJqLHDK3o71xm7s5GFrsBABghy3LIjvJbBfGWfRFdizas7sX7Vk0ghwAAOssajBc5JC5He3NMhgucsjcjvZmHQwXkSAHAMC2mnUwXMSQuR3tqWYuTtBcRIIcAABLa1mGuS5yyNyO9mYdDBeRIAcAAAtKNXMxguYismolAABw0pbp2ZSLSJADAAB2nVkHzUVjaCUAAEDHCHIAAAAdI8gBAAB0jCAHAADQMYIcAABAxwhyAAAAHSPIAQAAdIwgBwAA0DGCHAAAQMcIcgAAAB0jyAEAAHSMIAcAANAxghwAAEDHCHIAAAAdI8gBAAB0zEyCXFVdVlVfrKo7qurtG7z/zKr6T1V1a1XdVlVvmMV5AQAAltGWg1xV7U3yriSXJ3lxkldX1YunDvupJF9orb0kyaVJ3llVp2/13AAAAMtoFhW5lya5o7V2Z2ttJcm1Sa6YOqYleXpVVZKzkjyUpDeDcwMAACydWQS585PcPbF9YLRv0m8keVGSe5N8LslPt9YGMzg3AADA0plFkKsN9rWp7R9OckuS5yW5JMlvVNUzNmys6sqquqmqbnrwwQdn0D0AAIDdZRZB7kCSCye2L8iw8jbpDUk+3IbuSPKVJP/lRo211q5ure1vre0/99xzZ9A9AACA3WUWQe7GJC+oquePFjB5VZLrpo65K8nLk6SqnpvkhUnunMG5AQAAls6+rTbQWutV1VuSfDTJ3iTXtNZuq6o3j96/Ksn/lOS3qupzGQ7FfFtr7RtbPTcAAMAy2nKQS5LW2vVJrp/ad9XE63uT/NAszgUAALDsZvJAcAAAAHaOIAcAANAxghwAAEDHCHIAAAAdI8gBAAB0jCAHAADQMYIcAABAxwhyAAAAHSPIAQAAdIwgBwAA0DGCHAAAQMcIcgAAAB0jyAEAAHSMIAcAANAxghwAAEDHCHIAAAAdI8gBAAB0jCAHAADQMYIcAABAxwhyAAAAHSPIAQAAdIwgBwAA0DGCHAAAQMcIcgAAAB0jyAEAAHSMIAcAANAxghwAAEDHCHIAAAAdI8gBAAB0jCAHAADQMYIcAABAxwhyAAAAHSPIAQAAdIwgBwAA0DGCHAAAQMcIcgAAAB0jyAEAAHSMIAcAANAxghwAAEDHCHIAAAAdM5MgV1WXVdUXq+qOqnr7JsdcWlW3VNVtVfV/z+K8AAAAy2jfVhuoqr1J3pXkFUkOJLmxqq5rrX1h4pizk/zrJJe11u6qquds9bwAAADLahYVuZcmuaO1dmdrbSXJtUmumDrmJ5J8uLV2V5K01h6YwXkBAACW0iyC3PlJ7p7YPjDaN+kvJnlWVX28qm6uqtdu1lhVXVlVN1XVTQ8++OAMugcAALC7zCLI1Qb72tT2viR/KcnfTPLDSf5pVf3FjRprrV3dWtvfWtt/7rnnzqB7AAAAu8uW58hlWIG7cGL7giT3bnDMN1prTyR5oqo+keQlSb40g/MDAAAslVlU5G5M8oKqen5VnZ7kVUmumzrm95P8YFXtq6ozk7wsye0zODcAAMDS2XJFrrXWq6q3JPlokr1Jrmmt3VZVbx69f1Vr7faq+sMkn00ySPLu1trnt3puAACAZTSLoZVprV2f5PqpfVdNbf/LJP9yFucDAABYZjN5IDgAAAA7R5ADAADoGEEOAACgYwQ5AACAjhHkAAAAOkaQAwAA6BhBDgAAoGMEOQAAgI4R5AAAADpGkAMAAOgYQQ4AAKBjBDkAAICOEeQAAAA6RpADAADoGEEOAACgYwQ5AACAjhHkAAAAOkaQAwAA6BhBDgAAoGMEOQAAgI4R5AAAADpGkAMAAOgYQQ4AAKBjBDkAAICOEeQAAAA6RpADAADoGEEOAACgYwQ5AACAjhHkAAAAOkaQAwAA6BhBDgAAoGMEOQAAgI4R5AAAADpGkAMAAOgYQQ4AAKBjBDkAAICOEeQAAAA6RpADAADoGEEOAACgY2YS5Krqsqr6YlXdUVVvP8Zx/3VV9avq783ivAAAAMtoy0GuqvYmeVeSy5O8OMmrq+rFmxz3Pyf56FbPCQAAsMxmUZF7aZI7Wmt3ttZWklyb5IoNjntrkg8leWAG5wQAAFhaswhy5ye5e2L7wGjfmqo6P8nfTXLVDM4HAACw1GYR5GqDfW1q+1eTvK211j9uY1VXVtVNVXXTgw8+OIPuAQAA7C77ZtDGgSQXTmxfkOTeqWP2J7m2qpLknCSvrKpea+0/TjfWWrs6ydVJsn///ulACAAAsPRmEeRuTPKCqnp+knuSvCrJT0we0Fp7/vh1Vf1Wko9sFOIAAAA4vi0HudZar6rekuFqlHuTXNNau62q3jx637w4AACAGZpFRS6tteuTXD+1b8MA11p7/SzOCQAAsKxm8kBwAAAAdo4gBwAA0DGCHAAAQMcIcgAAAB0jyAEAAHSMIAcAANAxghwAAEDHCHIAAAAdI8gBAAB0jCAHAADQMYIcAABAxwhyAAAAHSPIAQAAdIwgBwAA0DGCHAAAQMcIcgAAAB0jyAEAAHSMIAcAANAxghwAAEDHCHIAAAAdI8gBAAB0jCAHAADQMYIcAABAxwhyAAAAHSPIAQAAdIwgBwAA0DGCHAAAQMcIcgAAAB0jyAEAAHSMIAcAANAxghwAAEDHCHIAAAAdI8gBAAB0jCAHAADQMYIcAABAxwhyAAAAHSPIAQAAdMy+eXcAAABgM621tJYMWktL1l4no31T700ev9ofJEkeePRw2vTxbXR8Wp7xlNPyrKedPrfveCoEOQAA4JhaaxmMAtCgtQwGw9f9wTBAPXJwJf3B8JjNjh209e8Pjz8arAat5bHDvSTJp7/y0DCQZRi2TtWhlX6S5M8ffOKYx9XZJcgBAADbazBo6bf1QWg6SPUHbV2o6g9ajqwO0tJy54OPrwWrjcLWsN3JdjbuxxNHhsHr9vsem8n3aqMT9QdbSG9LYiZBrqouS/JrSfYmeXdr7R1T7/9kkreNNh9P8t+31m6dxbkBAGDextWlwUS4euzw6jAUDdYHpLXX0/sHG1eyHj/SS2vJjV99aO2YU3WkN6xQ3f/okRl9c+Zly0GuqvYmeVeSVyQ5kOTGqrqutfaFicO+kuSvttYerqrLk1yd5GVbPTcAAGxkuhLVWo5WsAZHXw9ay0pvOI/q7ocOHq1uTVWkxuGsv0n4mvT4aHjg5+95dCbfZTA6Qa+vSsVRs6jIvTTJHa21O5Okqq5NckWStSDXWvv/Jo7/ZJILZnBeAAA6ZlyF6k8Ho4mhfcd6b9BaDq3005Lcdu+3psLVZKXrxPt0eHVYpTrw8KHt+dKwDWYR5M5PcvfE9oEcu9r2xiR/sNmbVXVlkiuT5KKLLppB9wAAOJ6NAlaSfOvg6tr+Jy1yMVW96g82roANBsNFLFpaPvWVh7bc1/FKhI8e6m25LeiqWQS52mDfhv8GUlV/LcMg9wObNdZauzrDoZfZv3+/+jEAsPQ2Cln9qSGCq/1BWkvueeTQ0eOn5lytBayJCtY4fE0vZjFexOIL981meGCbPgGwJbMIcgeSXDixfUGSe6cPqqr/Ksm7k1zeWvvmDM4LALAQ+uuC08S8qokANlnJGrSWw6vD4YFfvv+x9E8yZG1kvMz6Xd88uL1fFlgIswhyNyZ5QVU9P8k9SV6V5CcmD6iqi5J8OMlrWmtfmsE5AQBO2ORS7YNRFWu8NPtadWtc6ZoOY6NjnljpJS255e5H1oYL9k9yLtak8QIb33h8ZYbfFFgWWw5yrbVeVb0lyUczfPzANa2126rqzaP3r0ryC0m+Lcm/rqok6bXW9m/13ADA7jEYVaRaWg6u9NZVtfoTqwP214Woo5Wr9dtHHy48q3lZ/dGKgePKF8A8zeQ5cq2165NcP7XvqonXb0ryplmcCwCYr8mq1XhBjEcOrqybjzV9zGSFa3r/5PDBxw6vJkluvftbM+mreVnAbjWTIAcALJ5xJWsyOPUGLWnJg48d2TRwbVbdGu+fNF4Q4/b7HpvHVwRYWoIcACyAydDVnxhK2B88eTXCde+Pg9bEvK/J1QmnHRwFrzseeHyHvyEAsyTIAcBJmlwgYzJAjX9W+oOkJXc/dHBdRas3OLnQNWY1QgCmCXIALIXJoHW0ijUMV5PBa6Nq1/rhh08eXjjt8Ch4HXj40E58NQCWkCAHwMLq9QdrVa/eYLD2ezzU8MDDByeqW4N1c7t6gxOveAleAHSNIAfATI2rXJNhanqeV7//5GpXf4MK2GbGQw3vfkjwAmA5CXIAJHlyABuvbvjAY4efVBFbO6Z/9PjjPRzZPC8AmB1BDmAX6PWH4SpJHj28umHFq7fu9dFQ1t8kgI1XN/zzB57Y6a8DAByHIAcwZ+NgtVHoGle6htuDJ1XBxse3ljx+eBi8brvn0Tl/IwBguwlyAFvU6w/WBa/JYDYdunr9o+8/driXlpZPf+WheX8FAKBjBDlg6Y0rXr1RxavfXx/IehPbT6z0kpZ85q6H11XDTkU71Q8CAEtPkAN2hY1CV68/yEpvkJbkK994Iv3BIKv9ycrZYFQxO4nz9IcHH1kdbM8XAQA4AYIcsDBaG1a3Wtragh2rE0MUp+eJnUgYO7w6XCnx6986vIPfBABgewlywMy11tYqX6trVbLB2vyw1f5kGDv6uj9oeezwahILdgAAHIsgBxzT+uB1dGjieN+hlX5aks8d+NZaaDvWg5wBANg6QQ6WyPhZY60lDz+xsm7Y4jiYrfbXV9COl8lW+8O5Yo+PnjkGAMD2E+Sgo8bDF3ujBTx6E0FsvD39/mDiWWN/9vXH5vwNAAA4VYIcLJBhCBsGr3FlbLU/yOHVfgYt+fw931ob4tjbwrL3AAB0myAH26i/ViE7GspWBy2rvanANqqabRbMVnrD4YuPHTZ8EQAAQQ5OWmstK+MQNgpkKxOhbKU3yONHemkt+fRXHpp3dwEA2IUEORhpLTm40stqbxzUjv6s9Nq6YY/HM7BqIwAA20iQY9frD1pWesOq2cq4gjaxvdIf5NHDq0lLbr37W/PuLgAAHJcgR6cd6fVH4WwU1qYC2nhu2nEpoAEA0CGCHAtrtT/IkXE46w3WQtuR3iCPHe6lpeUzX3tk3t0EAIAdJ8gxF4PBcB7akdVBjvT7ObI6WNseV9T6x5hn1qy7DwDAEhPk2BattRzpDYPZ4V4/h1f7ObTSz6C13PTVh05owRAAAGBjghynrD9oObw6DGlHeoPR6+EQyCO9wZOeibbaH4x+C3EAALAVghzHNGgtjx5eHYa1UUg7vDoMbQIZAADMhyBHkqwNfTy02s/BleHrxw730lrLbfc8Ou/uAQAAEwS5JTMeCnlwZfgzfr3RwiIWFAEAgMUkyO1SJxPYAACAbhHkOm5ySOSh1X4Gg5ZPf+UhgQ0AAHYxQa4DWms5vDrIwZVeDk1U2g6vrn/W2mpvuCqkEAcAALubILdABoO2Vllbq7KNfpuuBgAAjAlyc9AfBbbV/iCDQfJnX380h1Y2fvYaAADANEFuG/X6gxxc7efwaMGR8dL+K6MhkIdW+kmSh59YnWc3AQCAjhHktsk9jxzKXd88OO9uAAAAu9CeeXdgtxpYcAQAANgmghwAAEDHzCTIVdVlVfXFqrqjqt6+wftVVb8+ev+zVfW9szgvAADAMtpykKuqvUneleTyJC9O8uqqevHUYZcnecHo58okv7nV8wIAACyrWVTkXprkjtbana21lSTXJrli6pgrkryvDX0yydlVdd4Mzg0AALB0qm3xwWVV9feSXNZae9No+zVJXtZae8vEMR9J8o7W2h+Ptm9I8rbW2k3HavvZ3/Gi9oqfvWZL/Zu1W269JUlyyUsuOeZxR3qDHOn1j3nMl7/w+STJC1783TPp2zK1t8h9W/T2Frlvy9beIvdt0dtb5L4tenuL3LdFb2+R+7Zs7S1y3xa9vUXu2zzbO2Pf3pyxbzGXD/ngm/+bm1tr+6f3zyLI/ViSH54Kci9trb114pj/I8m/mApy/7i1dvMG7V2Z4fDLnHXed/6lV/7i+7fUv3k5kSAHAADMXxeD3CyeI3cgyYUT2xckufcUjkmStNauTnJ1kuzfv7/93j/4vhl0cefd/dDBHHj40Ly7AQAAHMf5Zz81F33bmfPuxoY++OaN988idt6Y5AVV9fyqOj3Jq5JcN3XMdUleO1q98i8n+VZr7b4ZnBsAAGDpbLki11rrVdVbknw0yd4k17TWbquqN4/evyrJ9UlemeSOJAeTvGGr5wUAAFhWsxhamdba9RmGtcl9V028bkl+ahbnAgAAWHaLOaMPAACATc2kIseTfdtZp6cqObzaz6GVQQ6t9tMfbG2FUAAAgESQ2zZnnr4vZ56+/j/vMNT1c2h19DN63esLeAAAwIkT5HbQU07bm6ectjfPmtp/pNfP4VHV7uBKby3krQp4AADABgS5BTB8AOHePDOnrdu/2h8crdyt9HNwVMFb6Q3m1FMAAGARCHIL7LS9e3La3j15xlPWB7zeOOBNDtVc6edIb5CmiAcAALueINdB+/buydP37snTpwLeYNBGwzOPBryDKz0BDwAAdhlBbhfZs6fytDP25WlnrL+sAh4AAOwugtwSONGAd3C1Z4gmAAB0gCC3xI4V8A6O5t0dXu0PV9VcHeTwqpU0AQBgEQhyPMmePZWzztiXs8548v88ev1BjvQGo4A3/H14dZAjPZU8AADYKYIcJ2Xf3j3Zt3fPk6p4SdJay5HeIEdWBznc6z8p7PUHUh4AAMyCIMfMVNXaQ8+nn4mXDJ+LNwx6/az0B1npDbeHv/tZ6Ql6AABwIgQ5dsz4uXgbDdlMhnPzVsZhr9dfF/TGr1X1AABAkGOB7NlTecqeYUUvG1T0kqNz9FZ6g3VVvdX+0X09C7IAALDLCXJ0ytE5epsfM1nZWwt4o9eT+xT3AADoKkGOXWd9ZW9zq/31QW+lP8hqv63tWx1tG84JAMCiEeRYWuM5e2eefuzj+oNRuOsPstqbCHujILjaa2uvPX4BAICdIMjBcezdU9l7AhW+5GiVbzLc9frD173BcP/qYBgIFfoAADhVghzM0LjKl+NU+ZLhwi290Xy+3qjKNx7O2esP1u3vDZpqHwAAawQ5mJPhwi05oUpfa20Y8AZHg15v0LLSG/7ujQPgxPsqfgAAu5cgBx1QVTl9X+X07Dnhz4zD3nh45+pEyBuHvl6/rQVBVT8AgO4Q5GCXOpmKXzKs+g1D3TD09SbD3sS+1f7wwezj6p/wBwCw8wQ5IMmw6nfa3sppe5On5sTCX3K08jc5xHO80uc48K0FxLV9AiAAwFYIcsCWjCt/J+tkAmBv0NIfVQTN/QMAEOSAOTnVADgOecPQ146GvicFwbY2PFQVEADYbQQ5oFPGz/U7FdNBbzIUrqv8jbb7ExVDlUAAYJEIcsDS2EoIHGwU/gZHw1q9gH4AAAk5SURBVF6/v8l+Q0IBgG0gyAGcgD17KqfvqdHWyYfB8aqgxw9+Tw6L4xBpaCgAMCbIAeyAyVVBT9VkwOtPhb/JwDceHtqf+hEGAWD3EOQAOmIrQ0PHJoeI9ttEZbC1tbmCg9Y2DYKD0XECIQDMlyAHsETWDxE9dePq4GCQdb/HAXEyEK4Lj6P3DRcFgK0R5AA4aeurg6deJRxMVfomg99433RFcBwSVQkBWGaCHABzM6sKYbJ5KOxPBcLJSmF/KhgOfw8rjgCwyAQ5AHaFWYbC1iaHgWZtPmF/XeB78nBRwRCAnSLIAcCUqsq+vTWzPyRbGz5LcDyfcC0QToXA4b7RcRMhcNCmAuPAswkBlp0gBwDbrKqyt7LlVUcnbVg1PEY43KiaqHII0F2CHAB00KyrhmMbhbwTDYKDtvl+i9EAzJYgBwCsGa5IOpu5hpPWVQgnKoWDtnEFcaP9rWXdENNBU0UElpcgBwBsuz17KntSOW12o0uTHJ1/uFY93GA4aWvHD4PrPjvRpkoisKi2FOSq6tlJfi/JxUm+muTHW2sPTx1zYZL3Jfn2JIMkV7fWfm0r5wUASCbnH86+ipgcrSRODjEdTC06s9Ew1MFENXEwNezUkFNgFrZakXt7khtaa++oqrePtt82dUwvyT9qrX2mqp6e5Oaq+lhr7QtbPDcAwLYaVxK3y2A81LStH1LaNqgoDtrE8VOVw8HU5wcTIVNYhN1pq0HuiiSXjl6/N8nHMxXkWmv3Jblv9Pqxqro9yflJBDkAYKmNg+J2znWZripOh8XBRPA7GgLXVxwnP/uk9zwOA+Ziq/+/8dxRUEtr7b6qes6xDq6qi5N8T5JPHeOYK5NcmSQXXXTRFrsHALDctruqmGw8V/HoPMMNQuGTqosCI5ys4wa5qvrPGc5vm/ZzJ3OiqjoryYeS/Exr7dHNjmutXZ3k6iTZv3+/2xUAYMFt91zFsXFgHIe81japME4FwnWVyKnQ2TYIoEIjXXDcINda+xubvVdV91fVeaNq3HlJHtjkuNMyDHG/01r78Cn3FgCApbUWGLdhBdRpk6Fxs2B4NBxuEC5HYXBtjuPEAjfjwNksfMMWbHVo5XVJXpfkHaPfvz99QFVVkvckub219itbPB8AAGy7ydC4EzasFI4D3+RCNpsEx8mhrBtVLVUcd5+tBrl3JPlgVb0xyV1JfixJqup5Sd7dWntlku9P8pokn6uqW0af+9nW2vVbPDcAAOwKOzU8dWy64rhRJXF6fuLksZMrp24UQtcFR3Mct8WWglxr7ZtJXr7B/nuTvHL0+o+THfqnDAAA4Lh2uuI4NhhMBbyJ121iyGrL8HVahmExR0Njaxn+TO7L0ffWjh8Mf5/I8dXBtLKdq90CAACs2YlVVJfFnnl3AAAAgJMjyAEAAHSMIAcAANAxghwAAEDHCHIAAAAdI8gBAAB0jCAHAADQMYIcAABAxwhyAAAAHSPIAQAAdIwgBwAA0DGCHAAAQMcIcgAAAB0jyAEAAHRMtdbm3YdNVdWDSb42735s4Jwk35h3J3AdFohrsThci8XhWiwO12JxuBaLwXVYHCdyLb6jtXbu9M6FDnKLqqpuaq3tn3c/lp3rsDhci8XhWiwO12JxuBaLw7VYDK7D4tjKtTC0EgAAoGMEOQAAgI4R5E7N1fPuAElch0XiWiwO12JxuBaLw7VYHK7FYnAdFscpXwtz5AAAADpGRQ4AAKBjBLmTUFWXVdUXq+qOqnr7vPuzzKrqq1X1uaq6papumnd/lklVXVNVD1TV5yf2PbuqPlZVXx79ftY8+7gsNrkWv1RV94zujVuq6pXz7OMyqKoLq+r/qqrbq+q2qvrp0X73xQ47xrVwX+ywqnpKVX26qm4dXYtfHu13X+ywY1wL98UcVNXeqvrTqvrIaPuU7wlDK09QVe1N8qUkr0hyIMmNSV7dWvvCXDu2pKrqq0n2t9Y8A2WHVdVfSfJ4kve11r57tO9/SfJQa+0do3/keFZr7W3z7Ocy2ORa/FKSx1tr/+s8+7ZMquq8JOe11j5TVU9PcnOSv5Pk9XFf7KhjXIsfj/tiR1VVJXlaa+3xqjotyR8n+ekkPxr3xY46xrW4LO6LHVdV/0OS/Ume0Vr7ka38HUpF7sS9NMkdrbU7W2srSa5NcsWc+wQ7rrX2iSQPTe2+Isl7R6/fm+FfnNhmm1wLdlhr7b7W2mdGrx9LcnuS8+O+2HHHuBbssDb0+GjztNFPi/tixx3jWrDDquqCJH8zybsndp/yPSHInbjzk9w9sX0g/nCYp5bkj6rq5qq6ct6dIc9trd2XDP8ileQ5c+7PsntLVX12NPTSsKUdVFUXJ/meJJ+K+2Kupq5F4r7YcaMhZLckeSDJx1pr7os52eRaJO6LnfarSf5xksHEvlO+JwS5E1cb7POvGfPz/a21701yeZKfGg0xA5LfTPKdSS5Jcl+Sd863O8ujqs5K8qEkP9Nae3Te/VlmG1wL98UctNb6rbVLklyQ5KVV9d3z7tOy2uRauC92UFX9SJIHWms3z6pNQe7EHUhy4cT2BUnunVNfll5r7d7R7weS/IcMh74yP/eP5qaM56g8MOf+LK3W2v2jP7AHSf5N3Bs7YjTv5ENJfqe19uHRbvfFHGx0LdwX89VaeyTJxzOck+W+mKPJa+G+2HHfn+Rvj9Z5uDbJX6+q384W7glB7sTdmOQFVfX8qjo9yauSXDfnPi2lqnraaBJ7quppSX4oyeeP/Sm22XVJXjd6/bokvz/Hviy18R8GI3837o1tN1pI4D1Jbm+t/crEW+6LHbbZtXBf7LyqOreqzh69fmqSv5Hkz+K+2HGbXQv3xc5qrf2T1toFrbWLM8wR/2dr7b/NFu6JfTPv5S7VWutV1VuSfDTJ3iTXtNZum3O3ltVzk/yH4Z/X2Zfkd1trfzjfLi2PqvpAkkuTnFNVB5L8YpJ3JPlgVb0xyV1Jfmx+PVwem1yLS6vqkgyHfn81yT+YWweXx/cneU2Sz43moCTJz8Z9MQ+bXYtXuy923HlJ3jta9XtPkg+21j5SVX8S98VO2+xavN99sRBO+c8Kjx8AAADoGEMrAQAAOkaQAwAA6BhBDgAAoGMEOQAAgI4R5AAAADpGkAMAAOgYQQ4AAKBjBDkAAICO+f8By3ldxOCm3u0AAAAASUVORK5CYII= ",
text/plain: [
"<Figure size 1080x432 with 1 Axes>"
]
},
metadata: {
needs_background: "light"
},
output_type: "display_data"
},
{
data: {
image/png: "iVBORw0KGgoAAAANSUhEUgAAA2oAAAF1CAYAAACZNBlsAAAABHNCSVQICAgIfAhkiAAAAAlwSFlzAAALEgAACxIB0t1+/AAAADh0RVh0U29mdHdhcmUAbWF0cGxvdGxpYiB2ZXJzaW9uMy4yLjIsIGh0dHA6Ly9tYXRwbG90bGliLm9yZy+WH4yJAAAgAElEQVR4nO3df5SldX0n+Penq2lsfqVRGsKPRoiyLOjGjtsD4ySZMHHNgEkGJyebwWT9dcgSd2U2yeac0ThZY2YyrruzToyrEw4biT8SQ5zoZBgXNRldxklGXX6k/QEs0iFit402IB1EWpqu/u4f95YWRTVdXf1U3S91X69z6lQ9z33ucz/3fu/3Pt93PT9utdYCAABAP9ZNugAAAACeSFADAADojKAGAADQGUENAACgM4IaAABAZwQ1AACAzghqAAyuqh6pqu9bwnLnVFWrqvWrUVevqurVVfXnR3H/j1bVq4asCYDJEtQAplBVfbmq9o0D1der6veq6oRlruumqvr5+fNaaye01u4ZptrvPMZDVXXsEd6vVdVzh6qjB1X15qr6/fnzWmuXtdbeO6maABieoAYwvX6ytXZCkhcm+VtJfu1I7lwjK74dqapzkvxwkpbkH6z04x2txfYOTvseQwCOnKAGMOVaa19N8tEkz6+qk6vqI1V1/3gP1keq6qy5Zcd7tv5FVf1FkkeTvD+jEPXO8d65d46X+86erKr68ar6y6p6uKp2VtWbj7DEVyb5TJL3JHnC4X0L9+bNP4Swqj41nv25cW3/aDz/v6+qHVX1jaq6oarOmHf/51XVn41v+3pVvXE8/9iqentV7R7/vH1u715VXVJVu6rq9VX1tSS/N97r9cdV9ftV9XCSV1fV91TVu6vqvqr6alX9ZlXNLPaEq+q3x6/Vw1V1a1X98Hj+pUnemOQfjZ/T5xa+DlW1rqp+rarurao9VfW+qvqe8W1zh5q+qqq+UlUPVNU/PcL2AGAVCGoAU66qtiR5aZK/zGi78HtJnp3k7CT7krxzwV1ekeSqJCcmeXWS/5Tk6vHhjlcv8hDfyihsbUry40n+h6p62RGU+MokfzD++ftVddpS7tRa+7vjP18wru2PqupHk/yvSX4myelJ7k1yfZJU1YlJ/kOSjyU5I8lzk3xivI5/muRvJ9ma5AVJLsoT90B+b5JnZvS6XTWed3mSPx4/7z9I8t4kB8br/YEkP5bkCYeMznPz+LGemeQDSf5NVT2jtfaxJG9J8kfj5/SCRe776vHP30vyfUlOyJPb8IeSnJ/kxUneVFUXHKIOACZEUAOYXn9SVXuT/HmS/5jkLa21B1trH2qtPdpa+2aSf5HkRxbc7z2ttdtbawdaa48f7kFaaze11r7QWjvYWvt8kj9cZJ2Lqqofyij8fLC1dmuSv0rys0t/ik/yc0mua63d1lp7LMmvJnnR+PDKn0jytdba21pr326tfbO19tl59/tnrbU9rbX7k/xGRoF1zsEkv95ae6y1tm8879OttT9prR1MclKSy5L8UmvtW621PUl+K8kVixXZWvv9cVscaK29LcmxGQWrpT7Hf9Vau6e19sj4OV6x4PDL32it7WutfS7J5zIKnwB0xDHzANPrZa21/zB/RlUdl1GAuDTJyePZJ1bVTGttdjy980gepKouTvLWJM9PsiGj0PFvlnj3VyX509baA+PpD4zn/daR1DDPGUlum5torT1SVQ8mOTPJloyC4KHud++86XvH8+bc31r79oL7zH+dnp3kmCT3VdXcvHU5xGtZVb+S0d62MzI6N++kJKcc8lkdvtb1SebvifzavL8fzWivGwAdsUcNgPl+JaM9Nxe31k5KMnf4YM1bpi24z8LphT6Q5IYkW1pr35PkmgXrW1RVbczoEMUfqaqvjc//+uUkL6iquT1A30py3Ly7fe9hVrs7o9A09xjHJ3lWkq9mFJqes5T7ZXRY6O5504u9BvPn7UzyWJJTWmubxj8ntdaet/BO4/PRXp/Rcz+5tbYpyd/ku6/Z4V7vxWo9kOTrh7kfAB0R1ACY78SMzkvbW1XPTPLrS7jP1zM6F+qp1vmN1tq3q+qiLP3QxZclmU1yYUbna21NckFG58S9crzM9iQ/VVXHjS9ecuVhavtAktdU1dbxxUDekuSzrbUvJ/lIku+tql8aXzzkxPHewGR0uOavVdXmqjolyZuSPOES+U+ltXZfkj9N8raqOml8wY/nVNVih4CemFGwuj/J+qp6U0Z71OY/p3Oe4oqbf5jkl6vq3Bp95cLcOW0HllovAJMnqAEw39uTbEzyQEZXWvzYEu7z20l+enyVyHcscvv/mOSfVdU3Mwo4H1xiLa9K8nutta+01r4295PRhTF+bnzO1W8l2Z9ReHlvRhftmO/NSd5bVXur6mdaa59I8r8k+VCS+zLag3ZFkozPyXtJkp/M6NDAuzO6IEeS/GaSW5J8PskXMjp88jeX+DzmvDKjQz/vSPJQRhcaOX2R5T6e0VU4v5TRYYvfzhMPkZw7bPTBqrotT3ZdRlfj/FSSvx7f/x8fYa0ATFi1drgjKAAAAFhN9qgBAAB0RlADAADojKAGAADQGUENAACgM4IaAABAZ9ZP6oFPOeWUds4550zq4QEAACbq1ltvfaC1tnmx2yYW1M4555zccsstk3p4AACAiaqqew91m0MfAQAAOiOoAQAAdEZQAwAA6IygBgAA0BlBDQAAoDOCGgAAQGcENQAAgM4IagAAAJ0R1AAAADpz2KBWVddV1Z6q+uIhbq+qekdV7aiqz1fVC4cvc+XNHmz5xJ1fzzs+cXc+cefXM3uwTbokAABgSq1fwjLvSfLOJO87xO2XJTlv/HNxkt8Z/37amD3Y8op3fzbbd+7Nvv2z2bhhJlu3bMr7r7w4M+tq2eu86a49uX33w3neGSflkvNPXfa6AACA6XLYoNZa+1RVnfMUi1ye5H2ttZbkM1W1qapOb63dN1CNK+6mu/Zk+869eXT/bJLk0f2z2b5zb266a09efMFpR7y+lQh+AADA9BjiHLUzk+ycN71rPO9Jquqqqrqlqm65//77B3joYdy+++HsG4e0Ofv2z+aO3Q8va33zg1/LE4MfAADA4QwR1BbbRbToCV6ttWtba9taa9s2b948wEMP43lnnJSNG2aeMG/jhplceMZJy1rf0MEPAACYLkMEtV1JtsybPivJ7gHWu2ouOf/UbN2yKTW7P2kHc9z4UMVLzj91WesbOvgBAADTZYigdkOSV46v/vi3k/zN0+n8tCSZWVd5/5UXZ/Pd/z6bdv1F/s+X/8BRnU82dPADAACmy2EvJlJVf5jkkiSnVNWuJL+e5Jgkaa1dk+TGJC9NsiPJo0les1LFrqSZdZXj9t6T4/bes6wLiCxc1/uvvDgv+qkrs//4U/O2X/tlV30EAACWbClXfXz5YW5vSV43WEVrxJDBDwAAmC5DHPoIAADAgAQ1AACAzghqAAAAnRHUAAAAOiOoAQAAdEZQAwAA6IygBgAA0BlBDQAAoDOCGgAAQGcENQAAgM4IagAAAJ0R1AAAADojqAEAAHRGUAMAAOiMoAYAANAZQQ0AAKAzghoAAEBnBDUAAIDOCGoAAACdEdQAAAA6I6gBAAB0RlADAADojKAGAADQGUENAACgM4IaAABAZwQ1AACAzghqAAAAnRHUAAAAOiOoAQAAdEZQAwAA6IygBgAA0BlBDQAAoDOCGgAAQGcENQAAgM4IagAAAJ0R1AAAADojqAEAAHRGUAMAAOiMoAYAANAZQQ0AAKAzghoAAEBnBDUAAIDOCGoAAACdEdQAAAA6I6gBAAB0RlADAADojKAGAADQGUENAACgM0sKalV1aVXdVVU7quoNi9z+PVX176vqc1V1e1W9ZvhSAQAApsNhg1pVzSR5V5LLklyY5OVVdeGCxV6X5I7W2guSXJLkbVW1YeBaAQAApsJS9qhdlGRHa+2e1tr+JNcnuXzBMi3JiVVVSU5I8o0kBwatFAAAYEosJaidmWTnvOld43nzvTPJBUl2J/lCkl9srR0cpEIAAIAps5SgVovMawum/36S7UnOSLI1yTur6qQnrajqqqq6papuuf/++4+4WAAAgGmwlKC2K8mWedNnZbTnbL7XJPlwG9mR5K+T/JcLV9Rau7a1tq21tm3z5s3LrRkAAGBNW0pQuznJeVV17vgCIVckuWHBMl9J8uIkqarTkpyf5J4hCwUAAJgW6w+3QGvtQFVdneTjSWaSXNdau72qXju+/Zok/zzJe6rqCxkdKvn61toDK1g3AADAmnXYoJYkrbUbk9y4YN418/7eneTHhi0NAABgOi3pC68BAABYPYIaAABAZwQ1AACAzghqAAAAnRHUAAAAOiOoAQAAdEZQAwAA6IygBgAA0BlBDQAAoDOCGgAAQGcENQAAgM4IagAAAJ0R1AAAADojqAEAAHRGUAMAAOiMoAYAANAZQQ0AAKAzghoAAEBnBDUAAIDOCGoAAACdEdQAAAA6I6gBAAB0RlADAADojKAGAADQGUENAACgM4IaAABAZwQ1AACAzghqAAAAnRHUAAAAOiOoAQAAdEZQAwAA6IygBgAA0BlBDQAAoDOCGgAAQGcENQAAgM4IagAAAJ0R1AAAADojqAEAAHRGUAMAAOiMoAYAANAZQQ0AAKAzghoAAEBnBDUAAIDOCGoAAACdEdQAAAA6I6gBAAB0RlADAADojKAGAADQmSUFtaq6tKruqqodVfWGQyxzSVVtr6rbq+o/DlsmAADA9Fh/uAWqaibJu5K8JMmuJDdX1Q2ttTvmLbMpyb9Ocmlr7StVdepKFQwAALDWLWWP2kVJdrTW7mmt7U9yfZLLFyzzs0k+3Fr7SpK01vYMWyYAAMD0WEpQOzPJznnTu8bz5vsvkpxcVTdV1a1V9cqhCgQAAJg2hz30MUktMq8tsp7/OsmLk2xM8umq+kxr7UtPWFHVVUmuSpKzzz77yKsFAACYAkvZo7YryZZ502cl2b3IMh9rrX2rtfZAkk8lecHCFbXWrm2tbWutbdu8efNyawYAAFjTlhLUbk5yXlWdW1UbklyR5IYFy/y7JD9cVeur6rgkFye5c9hSAQAApsNhD31srR2oqquTfDzJTJLrWmu3V9Vrx7df01q7s6o+luTzSQ4m+d3W2hdXsnAAAIC1ainnqKW1dmOSGxfMu2bB9L9M8i+HKw0AAGA6LekLrwEAAFg9ghoAAEBnBDUAAIDOCGoAAACdEdQAAAA6I6gBAAB0RlADAADojKAGAADQGUENAACgM4IaAABAZwQ1AACAzghqAAAAnRHUAAAAOiOoAQAAdEZQAwAA6IygBgAA0BlBDQAAoDOCGgAAQGcENQAAgM4IagAAAJ0R1AAAADojqAEAAHRGUAMAAOiMoAYAANAZQQ0AAKAzghoAAEBnBDUAAIDOCGoAAACdEdQAAAA6I6gBAAB0RlADAADojKAGAADQGUENAACgM4IaAABAZwQ1AACAzghqAAAAnRHUAAAAOiOoAQAAdEZQAwAA6IygBgAA0BlBDQAAoDOCGgAAQGcENQAAgM4IagAAAJ0R1AAAADojqAEAAHRGUAMAAOiMoAYAANCZJQW1qrq0qu6qqh1V9YanWO5vVdVsVf30cCUCAABMl8MGtaqaSfKuJJcluTDJy6vqwkMs978l+fjQRQIAAEyTpexRuyjJjtbaPa21/UmuT3L5Isv94yQfSrJnwPoAAACmzlKC2plJds6b3jWe9x1VdWaSf5jkmuFKAwAAmE5LCWq1yLy2YPrtSV7fWpt9yhVVXVVVt1TVLffff/9SawQAAJgq65ewzK4kW+ZNn5Vk94JltiW5vqqS5JQkL62qA621P5m/UGvt2iTXJsm2bdsWhj0AAACytKB2c5LzqurcJF9NckWSn52/QGvt3Lm/q+o9ST6yMKQBAACwNIcNaq21A1V1dUZXc5xJcl1r7faqeu34duelAQAADGgpe9TSWrsxyY0L5i0a0Fprrz76sgAAAKbXkr7wGgAAgNUjqAEAAHRGUAMAAOiMoAYAANAZQQ0AAKAzghoAAEBnBDUAAIDOCGoAAACdEdQAAAA6I6gBAAB0RlADAADojKAGAADQGUENAACgM4IaAABAZwQ1AACAzghqAAAAnRHUAAAAOiOoAQAAdEZQAwAA6IygBgAA0BlBDQAAoDOCGgAAQGcENQAAgM4IagAAAJ0R1AAAADojqAEAAHRGUAMAAOiMoAYAANAZQQ0AAKAzghoAAEBnBDUAAIDOCGoAAACdEdQAAAA6I6gBAAB0RlADAADojKAGAADQGUENAACgM4IaAABAZwQ1AACAzghqAAAAnRHUAAAAOiOoAQAAdEZQAwAA6IygBgAA0BlBDQAAoDOCGgAAQGcENQAAgM4IagAAAJ1ZUlCrqkur6q6q2lFVb1jk9p+rqs+Pf/5zVb1g+FIBAACmw2GDWlXNJHlXksuSXJjk5VV14YLF/jrJj7TWvj/JP09y7dCFAgAATIul7FG7KMmO1to9rbX9Sa5Pcvn8BVpr/7m19tB48jNJzhq2TAAAgOmxlKB2ZpKd86Z3jecdypVJPno0RQEAAEyz9UtYphaZ1xZdsOrvZRTUfugQt1+V5KokOfvss5dYIgAAwHRZyh61XUm2zJs+K8nuhQtV1fcn+d0kl7fWHlxsRa21a1tr21pr2zZv3rycegEAANa8pQS1m5OcV1XnVtWGJFckuWH+AlV1dpIPJ3lFa+1Lw5cJAAAwPQ576GNr7UBVXZ3k40lmklzXWru9ql47vv2aJG9K8qwk/7qqkuRAa23bypUNAACwdi3lHLW01m5McuOCedfM+/vnk/z8sKUBAABMpyV94TUAAACrR1ADAADojKAGAADQGUENAACgM4IaAABAZwQ1AACAzghqAAAAnRHUAAAAOiOoAQAAdEZQAwAA6IygBgAA0BlBDQAAoDOCGgAAQGcENQAAgM4IagAAAJ0R1AAAADojqAEAAHRGUAMAAOiMoAYAANAZQQ0AAKAzghoAAEBnBDUAAIDOCGoAAACdEdQAAAA6I6gBAAB0RlADAADojKAGAADQGUENAACgM4IaAABAZwQ1AACAzghqAAAAnRHUAAAAOiOoAQAAdEZQAwAA6IygBgAA0BlBDQAAoDOCGgAAQGcENQAAgM4IagAAAJ0R1AAAADqzftIFAACsNbMHW266a09u3/1wnnfGSbnk/FMzs64mXRbwNCKoAQAMaPZgyyve/dls37k3+/bPZuOGmWzdsinvv/JiYe1pTgBnNQlqABwVAxd4opvu2pPtO/fm0f2zSZJH989m+869uemuPXnxBadNuDqWSwBntQlqrHkGkfBEQ/YJAxfWiiH7xe27H86+cUibs2//bO7Y/fCyg5pt2eRNYwAf+n3nfXxkBLVl+vRfPXjYZR7e9/iSl2VlHDzY8paP3pkdex7J/gMHs2H9ujz31BPyxssuyDofDEyhofvEbfc+lFvvfSiPHTiYZDRwufXeh3LNTX+VFz775KHLhxUxdL+YqcqG9eu+0y+SZMP6dVlXtawxgW1ZHz72xa8tGsA/9sWv5bgNa29IPfT7btLv4xc951kr/hhDc9VH1rTtO/dmx55H8tiBg2lJHjtwMDv2PJLtO/dOujSYiKH7xJcf/Fb2zxuMJsn+Awfz5Qe/NUC1sDqG7hdbt2zKc089ITmwP2kHc+x4QLp1y6Yu6mN5znnW8dmw/olD5w3r1+WcZx0/oYpW1tDvO+/jIyeosab1Pog8eLDltnsfyodv25Xb7n0oBw+2rtbH2jN0n5i2gQtr09D9Yt26yhsvuyAn3PEn2fjX/yn/04+ed1R7DXrflk2LoQN474Z+3w29vmkY86y9/bRr1MGDLdt37s2XH/xWznnW8dm6ZdNR7SYeen29mhtELjz85GgGkUO9dmvtkILVNi3v4aEN3SfmBi63f+WBZGZ9jj1m/ZoeuEybadn2rMS2Yt26yoYHdyQP7sgLn/367urjyM0F8F/4xV/J7Amn5erXXrVm+0Qy/PtuyPVNy5hnSUGtqi5N8ttJZpL8bmvtrQtur/HtL03yaJJXt9ZuG7jWqWVAv3xDDyKHfO3mHwKQPPEQgOWc2zP0+lZCryH36WCo127oPrESAxf6ME3bnt7/4bAS27Jew0HSd31DBvCe+0Qy/PtuyPU9HcY8QzhsUKuqmSTvSvKSJLuS3FxVN7TW7pi32GVJzhv/XJzkd8a/GcA0DuiHMvQgcsjX7qkOAVhOOwy9vqH1HHJ7N+RrtxLBasiBy0roedA3tCGf6zRte3r/h8OQ9fUeDnqvb0g994lk+H4x5Pp6H/MMZSnnqF2UZEdr7Z7W2v4k1ye5fMEylyd5Xxv5TJJNVXX6wLVOrd6PEe7d3CBy471/kRc+++Sj+qAf8rUb+tye3s8VGvIk4pV4D/d8vuDQJ2AP2Sd6Nzfoe8cn784f37or7/jk3XnLR+9ck+cyDP1cp23b03u/GKq+3i/o0Ht9Q+q9TyTD94uh1tf7mGco1dpTf4BX1U8nubS19vPj6Vckubi1dvW8ZT6S5K2ttT8fT38iyetba7ccar3PfPYF7SVvvG6ApzCc7Z/bniTZ+oKth1324W8/fthl7r7ji0mS8y58/lHV9c1vH8hX9+7L/KaqSs7ctDEnPuPITzMcen1Da63lkcdm8+3HZ/OMY2ZywrEzGR1du3w9tkVrLV/5xr48+tjjSSq1rrLxmJmc/cyNy3q+Q69vaPd/87E88Mj+J83ffMKGnHLisUe0rqHfw3Ov3b7HZ9PaaF1DtMVQ6xvytZszVJ9YqfUNpffPuyH1vq14OrRF7/1iiPWtxOfJkNvtlahvaD2OKVZSj+/j5Yx5TnrGMct+vJX0wdf+nVtba9sWu20p74LFnu3CdLeUZVJVVyW5KklOOP05S3jo1bWUgHYkhnpDn3DsTDYeM/OkQd8Jx850sb5kuE68UmGjx7aoqpz9zI155LENeezx2Rx7lBu3odc3Z6i2fcYxM6nKkzZIxx5z5K/d0O/hRx6b/c66klGN+x6fzSOPzS5rYzn0+oZ87eYMHaiGXt9Q77tvz2uHOa0ljz2+vLaYM+TApdfnOk3bnjm994sh1jf058nQ2+2V+LxLhn2v9DimmG8a+sVKjXl6s5RP7l1JtsybPivJ7mUsk9batUmuTZJt27a1P/qFFx1RsT1Z7S+x7v3KW6/7wK8mSd70T25Y9jqS0ZfnvuOTdyc12p3d2uhb7H/y+8/s5pjjaTrfJRmubVfiwgRDtcOHb9uVP7511xNntuRF3/es/NQLz5r4+qbpnI05Q3+mzL/K2LHr1+XVf+fco/pMGaq+Ide1Es+1523PwYMtv/Dxd2f2hNPyE//Vj635z+KhrMSX3g+53V6pz7sh++yQVqKP6ReL6/ULrz/42kPftpSgdnOS86rq3CRfTXJFkp9dsMwNSa6uquszuojI37TW7ltWtSxq3brKC5998mBhZej1DeXpcHJor69d7+ZOIh5qgzRkO/R8CeJk+NeudwcPtux/1nMze8Jpue3eh47quc5dZWzhoO9oruY3ZH29P9detz1zg/lHLnxZMrM+7/jk3Wv+nxdDGfrzZOjt9kp83g3Zz4Y2ZB/TL9aewwa11tqBqro6ycczujz/da2126vqtePbr0lyY0aX5t+R0eX5X7NyJbOW+a6Yta3XkDv0APfpMGDu1dADjaEHfUPW1/tz7dncBSeyfkOS/q6W17ue/9E1dH3TFF70i7VnSQett9ZuzCiMzZ93zby/W5LXDVsa02glBrgsX8//hRzSSuztm5YB89BWYqAx5KBvyPp6f649ezocfTEtet9uT1N40S/Wnn4uKQMxwO3JNP0XMun3EK9p0/tAY8j6en+uPXP0RT96325PUz/TL9YeQY3uGOD2YZr+C0k/eh9oDFlf78+1Z73vxZk2PW+3p6mf6Rdrj6AGLGqa/gtJP3ofaAxZX+/PtWe978WhH9PUz/SLtUdQW6ZeL/E5CbMHW9affn72H39aHt1/IJecf2pmfCg87T26/0D+7y/cl0f3z35n3sYNM7n0+d/r/c+KuuE5P5Sb7tqTO3Y/nAvPOKm7z5Qh6+v9ufbuB887ZdIl8DQwbf1Mv1g7qi38dsxVsm3btnbLLbdM5LEZzuzBlle8+7P59JfuS1u3Pscde0y2btmU91958Zr+EJwGc227fefe7Ns/m40bZrQtAMCAqurW1tq2xW6zR42jctNde7J95960mdF5TI/un832nXtz01178uILTptwdRyNmXWV91958VT9FxIAoBeCGkfl9t0PZ9+8Q+OSZN/+2dyx+2FBbQ2YWVd58QWnaUsAgFW2btIF8PT2vDNOysYNM0+Yt3HDTC4846QJVQQAAE9/ghpH5ZLzT83WLZty3IaZVJLjxucxXXL+qZMuDQAAnrYc+shRcR4TAAAMT1DjqDmPCQAAhuXQRwAAgM4IagAAAJ0R1AAAADojqAEAAHRGUAMAAOiMoAYAANAZQQ0AAKAzghoAAEBnBDUAAIDOCGoAAACdqdbaZB646v4k907kwZ/aKUkemHQRJNEWPdEW/dAW/dAWfdAO/dAW/dAW/ThcWzy7tbZ5sRsmFtR6VVW3tNa2TboOtEVPtEU/tEU/tEUftEM/tEU/tEU/jqYtHPoIAADQGUENAACgM4Lak1076QL4Dm3RD23RD23RD23RB+3QD23RD23Rj2W3hXPUAAAAOmOPGgAAQGcEtXmq6tKququqdlTVGyZdzzSrqi9X1ReqantV3TLpeqZJVV1XVXuq6ovz5j2zqv6squ4e/z55kjVOg0O0w5ur6qvjfrG9ql46yRqnRVVtqar/p6rurKrbq+oXx/P1i1X2FG2hb6yyqnpGVf2/VfW5cVv8xni+frGKnqId9IkJqaqZqvrLqvrIeHrZfcKhj2NVNZPkS0lekmRXkpuTvLy1dsdEC5tSVfXlJNtaa74DZJVV1d9N8kiS97XWnj+e978n+UZr7a3jf2Kc3Fp7/STrXOsO0Q5vTvJIa+3/mGRt06aqTk9yemvttqo6McmtSV6W5NXRL1bVU7TFz0TfWFVVVUmOb609UlXHJPnzJL+Y5KeiX6yap6/BX4oAAAMLSURBVGiHS6NPTERV/c9JtiU5qbX2E0czhrJH7bsuSrKjtXZPa21/kuuTXD7hmmDVtdY+leQbC2ZfnuS947/fm9HAiBV0iHZgAlpr97XWbhv//c0kdyY5M/rFqnuKtmCVtZFHxpPHjH9a9ItV9RTtwARU1VlJfjzJ786bvew+Iah915lJds6b3hUf/pPUkvxpVd1aVVdNuhhyWmvtvmQ0UEpy6oTrmWZXV9Xnx4dGOqRolVXVOUl+IMlno19M1IK2SPSNVTc+xGt7kj1J/qy1pl9MwCHaIdEnJuHtSf5JkoPz5i27Twhq31WLzPMficn5wdbaC5NcluR148PAYNr9TpLnJNma5L4kb5tsOdOlqk5I8qEkv9Rae3jS9UyzRdpC35iA1tpsa21rkrOSXFRVz590TdPoEO2gT6yyqvqJJHtaa7cOtU5B7bt2Jdkyb/qsJLsnVMvUa63tHv/ek+TfZnRoKpPz9fG5IXPniOyZcD1TqbX29fEG+WCS/yv6xaoZn/vxoSR/0Fr78Hi2fjEBi7WFvjFZrbW9SW7K6Lwo/WJC5reDPjERP5jkH4yvs3B9kh+tqt/PUfQJQe27bk5yXlWdW1UbklyR5IYJ1zSVqur48Uniqarjk/xYki8+9b1YYTckedX471cl+XcTrGVqzX3Qj/3D6BerYnyy/ruT3Nla+1fzbtIvVtmh2kLfWH1VtbmqNo3/3pjkv0ny/0W/WFWHagd9YvW11n61tXZWa+2cjHLEJ1tr/12Ook+sH7zKp6nW2oGqujrJx5PMJLmutXb7hMuaVqcl+bej7XHWJ/lAa+1jky1pelTVHya5JMkpVbUrya8neWuSD1bVlUm+kuS/nVyF0+EQ7XBJVW3N6LDsLyf5hYkVOF1+MMkrknxhfB5Ikrwx+sUkHKotXq5vrLrTk7x3fNXsdUk+2Fr7SFV9OvrFajpUO7xfn+jGsrcVLs8PAADQGYc+AgAAdEZQAwAA6IygBgAA0BlBDQAAoDOCGgAAQGcENQAAgM4IagAAAJ0R1AAAADrz/wM6Hf3VQLOHogAAAABJRU5ErkJggg== ",
text/plain: [
"<Figure size 1080x432 with 1 Axes>"
]
},
metadata: {
needs_background: "light"
},
output_type: "display_data"
}
],
source: [
"# ACF 그래프 ",
"plot_acf(ts_log2) ",
"# PACF 그래프: p=1이 가장 적합해보임 ",
"# 2 이상인 구간에서는 PACF가 거의 0에 가까워지고 있어서 데이터 상관도가 0이라고 보면 됨. ",
"plot_pacf(ts_log2) ",
"plt.show()"
]
},
{
cell_type: "code",
execution_count: 114,
metadata: { },
outputs: [
{
name: "stdout",
output_type: "stream",
text: [
"Results of Dickey-Fuller Test: ",
"Test Statistic -31.819284 ",
"p-value 0.000000 ",
"#Lags Used 4.000000 ",
"Number of Observations Used 5153.000000 ",
"Critical Value (1%) -3.431620 ",
"Critical Value (5%) -2.862101 ",
"Critical Value (10%) -2.567069 ",
"dtype: float64 "
]
},
{
data: {
image/png: "... ",
text/plain: [
"<Figure size 1080x432 with 1 Axes>"
]
},
metadata: {
needs_background: "light"
},
output_type: "display_data"
}
],
source: [
"# d차 차분을 보고 시계열이 안정된 형태인지 확인 ",
"diff_1_2 = ts_log2.diff(periods=1).iloc[1:] ",
"diff_1_2.plot(title='Difference 1st') ",
" ",
"augmented_dickey_fuller_test(diff_1_2)"
]
},
{
cell_type: "code",
execution_count: 115,
metadata: { },
outputs: [
{
data: {
text/plain: [
"<matplotlib.legend.Legend at 0x7fccbb835410>"
]
},
execution_count: 115,
metadata: { },
output_type: "execute_result"
},
{
data: {
image/png: "... ",
text/plain: [
"<Figure size 720x432 with 1 Axes>"
]
},
metadata: {
needs_background: "light"
},
output_type: "display_data"
}
],
source: [
"# 학습 테스트 9:1 ",
"train_data2, test_data2 = ts_log2[:int(len(ts_log2)*0.9)], ts_log2[int(len(ts_log2)*0.9):] ",
"plt.figure(figsize=(10,6)) ",
"plt.grid(True) ",
"plt.plot(ts_log2, c='r', label='training dataset') # train_data를 적용하면 그래프가 끊어져 보이므로 자연스러운 연출을 위해 ts_log를 선택 ",
"plt.plot(test_data2, c='b', label='test dataset') ",
"plt.legend()"
]
},
{
cell_type: "code",
execution_count: 116,
metadata: { },
outputs: [
{
name: "stdout",
output_type: "stream",
text: [
"Date ",
"2000-02-11 12.517227 ",
"2000-02-14 12.417144 ",
"Name: Close, dtype: float64 ",
"(4643,) ",
"(516,) "
]
}
],
source: [
"print(ts_log2[:2]) ",
"print(train_data2.shape) ",
"print(test_data2.shape)"
]
},
{
cell_type: "code",
execution_count: 131,
metadata: { },
outputs: [
{
name: "stdout",
output_type: "stream",
text: [
" ARIMA Model Results ",
"============================================================================== ",
"Dep. Variable: D.Close No. Observations: 4642 ",
"Model: ARIMA(1, 1, 0) Log Likelihood 9142.540 ",
"Method: css-mle S.D. of innovations 0.034 ",
"Date: Thu, 13 Aug 2020 AIC -18279.079 ",
"Time: 17:14:07 BIC -18259.751 ",
"Sample: 1 HQIC -18272.279 ",
" ",
"================================================================================= ",
" coef std err z P>|z| [0.025 0.975] ",
"--------------------------------------------------------------------------------- ",
"const -0.0002 0.001 -0.365 0.715 -0.001 0.001 ",
"ar.L1.D.Close 0.0152 0.015 1.032 0.302 -0.014 0.044 ",
" Roots ",
"============================================================================= ",
" Real Imaginary Modulus Frequency ",
"----------------------------------------------------------------------------- ",
"AR.1 65.9771 +0.0000j 65.9771 0.0000 ",
"----------------------------------------------------------------------------- "
]
},
{
name: "stderr",
output_type: "stream",
text: [
"/home/aiffel0036/anaconda3/envs/aiffel/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:218: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting. ",
" ' ignored when e.g. forecasting.', ValueWarning) ",
"/home/aiffel0036/anaconda3/envs/aiffel/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:218: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting. ",
" ' ignored when e.g. forecasting.', ValueWarning) "
]
}
],
source: [
"# Build Model ",
"model2 = ARIMA(train_data2, order=(1, 1, 0)) ",
"fitted_m2 = model2.fit(disp=-1) ",
"print(fitted_m2.summary())"
]
},
{
cell_type: "code",
execution_count: 132,
metadata: { },
outputs: [
{
data: {
image/png: "... ",
text/plain: [
"<Figure size 1080x432 with 1 Axes>"
]
},
execution_count: 132,
metadata: { },
output_type: "execute_result"
},
{
data: {
image/png: "... ",
text/plain: [
"<Figure size 1080x432 with 1 Axes>"
]
},
metadata: {
needs_background: "light"
},
output_type: "display_data"
}
],
source: [
"fitted_m2.plot_predict()"
]
},
{
cell_type: "code",
execution_count: 133,
metadata: { },
outputs: [
{
data: {
image/png: "... ",
text/plain: [
"<Figure size 1000x500 with 1 Axes>"
]
},
metadata: {
needs_background: "light"
},
output_type: "display_data"
}
],
source: [
" ",
"# Forecast : 결과가 fc에 담깁니다. ",
"fc2, se2, conf2 = fitted_m2.forecast(len(test_data2), alpha=0.05) # 95% conf ",
" ",
"# Make as pandas series ",
"fc_series2 = pd.Series(fc2, index=test_data2.index) # 예측결과 ",
"lower_series2 = pd.Series(conf2[:, 0], index=test_data2.index) # 예측결과의 하한 바운드 ",
"upper_series2 = pd.Series(conf2[:, 1], index=test_data2.index) # 예측결과의 상한 바운드 ",
" ",
"# Plot ",
"plt.figure(figsize=(10,5), dpi=100) ",
"plt.plot(train_data2, label='training') ",
"plt.plot(test_data2, c='b', label='actual price') ",
"plt.plot(fc_series2, c='r',label='predicted price') ",
"plt.fill_between(lower_series2.index, lower_series2, upper_series2, color='k', alpha=.10) ",
"plt.legend() ",
"plt.show()"
]
},
{
cell_type: "code",
execution_count: 134,
metadata: { },
outputs: [
{
name: "stdout",
output_type: "stream",
text: [
"MSE: 5747219941.850224 ",
"MAE: 46195.752925678804 ",
"RMSE: 75810.42106366527 ",
"MAPE: 23.32% "
]
}
],
source: [
"# 최종으로 오차 계산 ",
"mse2 = mean_squared_error(np.exp(test_data2), np.exp(fc2)) ",
"print('MSE: ', mse2) ",
" ",
"mae2 = mean_absolute_error(np.exp(test_data2), np.exp(fc2)) ",
"print('MAE: ', mae2) ",
" ",
"rmse2 = math.sqrt(mean_squared_error(np.exp(test_data2), np.exp(fc2))) ",
"print('RMSE: ', rmse2) ",
" ",
"mape2 = np.mean(np.abs(np.exp(fc2) - np.exp(test_data2))/np.abs(np.exp(test_data2))) ",
"print('MAPE: {:.2f}%'.format(mape2*100))"
]
},
{
cell_type: "markdown",
metadata: { },
source: [
"# LG Display Stock Prediction"
]
},
{
cell_type: "code",
execution_count: 136,
metadata: { },
outputs: [
{
name: "stdout",
output_type: "stream",
text: [
"Date ",
"2004-07-22 14.05 ",
"2004-07-23 14.20 ",
"2004-07-26 14.09 ",
"2004-07-27 14.45 ",
"2004-07-28 14.40 ",
"Name: Close, dtype: float64 ",
"(4044,) "
]
}
],
source: [
"filepath3 = os.getenv('HOME')+'/aiffel/stock_prediction/data/LPL.csv' ",
"df3 = pd.read_csv(filepath3, index_col='Date', parse_dates=True) ",
"ts3 = df3['Close'] ",
"print(ts3.head()) ",
"print(ts3.shape)"
]
},
{
cell_type: "code",
execution_count: 137,
metadata: { },
outputs: [
{
data: {
text/plain: [
"[<matplotlib.lines.Line2D at 0x7fcc8a59aa90>]"
]
},
execution_count: 137,
metadata: { },
output_type: "execute_result"
},
{
data: {
image/png: "... ",
text/plain: [
"<Figure size 1080x432 with 1 Axes>"
]
},
metadata: {
needs_background: "light"
},
output_type: "display_data"
}
],
source: [
"# 데이터 확인 ",
"# 떨어지는 추세 ",
"rcParams['figure.figsize']=15,6 ",
" ",
"plt.plot(ts3)"
]
},
{
cell_type: "code",
execution_count: 138,
metadata: { },
outputs: [
{
name: "stdout",
output_type: "stream",
text: [
"Series([], Name: Close, dtype: float64) "
]
}
],
source: [
"# check the missing value ",
"print(ts3[ts3.isna()]) ",
"# interpolation ",
"ts3 = ts3.interpolate(method='time')"
]
},
{
cell_type: "code",
execution_count: 142,
metadata: { },
outputs: [
{
data: {
image/png: "... ",
text/plain: [
"<Figure size 1080x432 with 1 Axes>"
]
},
metadata: {
needs_background: "light"
},
output_type: "display_data"
}
],
source: [
"# 오르는 추세인데 딱히 규칙적 패턴은 없어보임. ",
"plot_rolling_statistics(ts3,window=12)"
]
},
{
cell_type: "code",
execution_count: 143,
metadata: { },
outputs: [
{
data: {
text/plain: [
"[<matplotlib.lines.Line2D at 0x7fcc870c3290>]"
]
},
execution_count: 143,
metadata: { },
output_type: "execute_result"
},
{
data: {
image/png: "... ",
text/plain: [
"<Figure size 1080x432 with 1 Axes>"
]
},
metadata: {
needs_background: "light"
},
output_type: "display_data"
}
],
source: [
"# 로그 함수로 변환 ",
"ts_log3 = np.log(ts3) ",
"plt.plot(ts_log3)"
]
},
{
cell_type: "code",
execution_count: 144,
metadata: { },
outputs: [
{
name: "stdout",
output_type: "stream",
text: [
"Results of Dickey-Fuller Test: ",
"Test Statistic -1.724947 ",
"p-value 0.418307 ",
"#Lags Used 20.000000 ",
"Number of Observations Used 4023.000000 ",
"Critical Value (1%) -3.431977 ",
"Critical Value (5%) -2.862259 ",
"Critical Value (10%) -2.567153 ",
"dtype: float64 "
]
}
],
source: [
"# 제대로 변환되었는지 aumented dickey-fuller test 시행 ",
"# 아직 p-value보면 덜 변환됨. ",
"augmented_dickey_fuller_test(ts_log3)"
]
},
{
cell_type: "code",
execution_count: 145,
metadata: { },
outputs: [
{
data: {
image/png: "... ",
text/plain: [
"<Figure size 1080x432 with 4 Axes>"
]
},
metadata: {
needs_background: "light"
},
output_type: "display_data"
}
],
source: [
"# Time Series Decomposition ",
"decomposition3 = seasonal_decompose(ts_log3, model='multiplicative',period=30) ",
" ",
"trend3 = decomposition2.trend ",
"seasonal3 = decomposition2.seasonal ",
"residual3 = decomposition2.resid ",
" ",
"plt.subplot(411) ",
"plt.plot(ts_log3, label='Original') ",
"plt.legend(loc='best') ",
"plt.subplot(412) ",
"plt.plot(trend3, label='Trend') ",
"plt.legend(loc='best') ",
"plt.subplot(413) ",
"plt.plot(seasonal3,label='Seasonality') ",
"plt.legend(loc='best') ",
"plt.subplot(414) ",
"# trend와 seasonality를 제거하고 난 나머지 ",
"# Trend+Seasonality+Residual = Original ",
"plt.plot(residual3, label='Residuals') ",
"plt.legend(loc='best') ",
"plt.tight_layout()"
]
},
{
cell_type: "code",
execution_count: 146,
metadata: { },
outputs: [
{
data: {
image/png: "... ",
text/plain: [
"<Figure size 1080x432 with 1 Axes>"
]
},
metadata: {
needs_background: "light"
},
output_type: "display_data"
}
],
source: [
"plot_rolling_statistics(residual3)"
]
},
{
cell_type: "code",
execution_count: 147,
metadata: { },
outputs: [
{
name: "stdout",
output_type: "stream",
text: [
"Results of Dickey-Fuller Test: ",
"Test Statistic -1.770622e+01 ",
"p-value 3.518811e-30 ",
"#Lags Used 3.300000e+01 ",
"Number of Observations Used 5.095000e+03 ",
"Critical Value (1%) -3.431634e+00 ",
"Critical Value (5%) -2.862107e+00 ",
"Critical Value (10%) -2.567072e+00 ",
"dtype: float64 "
]
}
],
source: [
"# residual check ",
"residual3.dropna(inplace=True) ",
"augmented_dickey_fuller_test(residual3)"
]
},
{
cell_type: "code",
execution_count: 151,
metadata: { },
outputs: [
{
data: {
image/png: "iVBORw0KGgoAAAANSUhEUgAAA3IAAAF1CAYAAACtTdhwAAAABHNCSVQICAgIfAhkiAAAAAlwSFlzAAALEgAACxIB0t1+/AAAADh0RVh0U29mdHdhcmUAbWF0cGxvdGxpYiB2ZXJzaW9uMy4yLjIsIGh0dHA6Ly9tYXRwbG90bGliLm9yZy+WH4yJAAAgAElEQVR4nO3dfZBsZ30f+O/vzpWEhAAZJDAIySI2YcGuIJy7YG/sRFmCLbEvSlyxA3bxVrAKW4ayU6laWNvxSzabkM3it5hYqzWKAdtgElhb9srGRLUscdZgSVgCJAWQhUBXV0hCQki6bzPd/ewf3T23Z+7MfZue6T7Tn0/V1Jxz+vRzntZRX92vfs/znGqtBQAAgO7YM+sOAAAAcHoEOQAAgI4R5AAAADpGkAMAAOgYQQ4AAKBjBDkAAICOEeQAYEqq6o1V9adbeP8fVdUbptknAHYnQQ6AHVVVn6iqb1TVOafxnlZV37Gd/dppVfXzVfVbk8daa1e11t43qz4B0B2CHAA7pqouS/L9SVqS/36mnTmJqtp7KscAYBYEOQB20uuTfCrJbyZZHUI4qtK9ZWJ/dYhiVX1ydPj2qnqyqv7B6Pj/UFV3V9WjVXVDVT1v4v3fWVUfH732YFX91Oj4OVX1y1V1YPTzy+PKYFVdUVX7q+odVfW1JP92VDX791X1W1X1eJI3VtUzquq9VfVAVd1fVf+sqpY2+rBV9StVdV9VPV5Vt1bV94+OX5nkp5L8g9Fnun39P4eq2lNVP1NVX6mqh6rq/VX1jNFrl42qlG+oqq9W1der6qe3fHcA6AxBDoCd9Pokvz36+cGqes7J3tBa+5ujzZe21s5vrf1uVf3XSf5Fkh9J8twkX0nyoSSpqqcl+Q9J/jjJ85J8R5KbRm38dJLvSXJ5kpcmeXmSn5m43LcmeWaSb0tyzejY1Un+fZILRv1+X5LeqN2XJfmBJG/Jxm4eXeuZSX4nyb+rqqe01v44yT9P8rujz/TSDd77xtHP307yV5Kcn+TX1p3zfUlelOSVSX62ql68ST8A2GUEOQB2RFV9X4YB6cOttVuT/GWSHz3D5n4syfWttc+01o4m+Z+TfO9o6OZ/m+RrrbV3t9aOtNaeaK19euJ9/7S19lBr7eEkv5DkdRPtDpL8XGvtaGvt8OjYn7XWfq+1Nkjy9CRXJfnJ1trB1tpDSX4pyWs26mRr7bdaa4+01nqttXcnOSfD4HWqn/EXW2v3tNaeHH3G16wb3vkLrbXDrbXbk9yeYTgFYAEIcgDslDck+ZPW2tdH+7+TieGVp+l5GVbhkiSjoPNIkouTXJJhSDzp+0bbz5vYf7i1dmTde+6b2P62JGcleaCqHquqx5L8H0mevdHFquofV9VdVfXN0bnPSHLhyT7cCfq6N8lkFfNrE9uHMqzaAbAATNoGYNtV1bkZDoNcGs0/S4bVqQuq6qVJDiY5b+It33qSJg9kGKrG7T81ybOS3J9h8HrtSd53x2j/0tGxsbbBeyaP3ZfkaJILW2u9E3VwNB/uHRkOe7yjtTaoqm8kqRNca6O+jl2a4ZDOB5M8/yTvBWCXU5EDYCf83ST9JC/JcM7Y5UlenOQ/Zjhv7rYkP1RV540eM/Dmde9/MMN5YmO/k+RNVXX5aLGSf57k0621e5P8YZJvraqfHC1u8rSqesXofR9M8jNVdVFVXZjkZ5OseQTAibTWHkjyJ0neXVVPHy1I8u1V9bc2OP1pGQavh5PsraqfzXBo5uRnuqyqNvtv8QeT/KOqekFVnZ9jc+pOGCABWAyCHAA74Q1J/m1r7autta+NfzJcvOPHMpxntpxhuHlfhouKTPr5JO8bDWf8kdbaTUn+SZKPJHkgybdnNE+ttfZEklcl+e8yHHr4pQwXDEmSf5bkliSfTfK5JJ8ZHTsdr09ydpI7k3wjw4VQnrvBeR9L8kdJvpjhsMgjWTtM89+Nfj9SVZ/Z4P3XJ/lAkk8m+fLo/W8/zb4CsEtVaycb2QEAAMA8UZEDAADoGEEOAACgYwQ5AACAjhHkAAAAOkaQAwAA6Ji5fiD4hRde2C677LJZdwMAAGAmbr311q+31i5af3yug9xll12WW265ZdbdAAAAmImq+spGxw2tBAAA6BhBDgAAoGMEOQAAgI4R5AAAADpGkAMAAOgYQQ4AAKBjBDkAAICOEeQAAAA6RpADAADomKkEuaq6vqoeqqrPb/J6VdWvVtXdVfXZqvruaVx3p/UHLTfd9WB+9aYv5aa7Hkx/0GbdJQAAYAHtnVI7v5nk15K8f5PXr0rywtHPK5L8+uh3Z/QHLa9776dz232P5fByP+eevZTLL7kgH3jzK7K0p864zU984aHcceDxfOfznp4rXvTsM24LAABYHFMJcq21T1bVZSc45eok72+ttSSfqqoLquq5rbUHpnH9nfCJLzyU2+57LIeW+0mSQ8v93HbfY/nEFx7KK1/8nNNuTzAEAADO1LQqcidzcZL7Jvb3j44dF+Sq6pok1yTJpZdeuiOdOxV3HHg8h0chbuzwcj93Hnj8jIKcYAgAAJypnQpyG/3tf8MJZq2165JclyT79u2bm0lo3/m8p+fcs5dWg1eSnHv2Ul7yvKefUXuLGAzH7U4zHAqbAAAsop0KcvuTXDKx//wkB3bo2lNxxYuencsvuSB/9sUH0vbszXnnnJXLL7kgV7zo2WfU3qIFw2T64VAVEgCARbVTjx+4IcnrR6tXfk+Sb3ZpflySLO2pfODNr8hFX/qDXLD/P+Vfv/ZlWwoM42BY/eWkDXLeKIRsNRhO2q5geKYmw2HL2nA4D+2Ng+HbP/gX+aWPfzFv/+Bf5HXv/fSWVyed9mqnVk8FAGAqFbmq+mCSK5JcWFX7k/xckrOSpLV2bZIbk7w6yd1JDiV50zSuu9OW9lTOe+yenPfYPWdclZps6wNvfkW+94fenOWnPjvv/pl/tKXqz7xXDJPpVw1VIVUhAQAW1bRWrXztSV5vSX58GtfaTRYpGCbTD4fzPjw1mX44NBdS2AQASHZujhw7YJ6DYTL9cKgKufX2VCEFQwCgmwQ5NjXNYDhub5rhUBVy6+2pQgqGAEA3CXLsqO0Ih6qQqpCnowvBcNyu4akAwGYEOZigCqkKeboMTxUMAWAWBDnYZqqQu7sKaXiqKiQAzIIgBwtOFXK+gmGyeMNTE1VIADhdghwwdYtUhTQ8dUgV8vQJhgBsxZ5ZdwBgp42D4QX3fyqvfPFztvSX53EwvOhLf5AL9v+n/OvXvmzLQwzH4bD6y0kb5LxRaNjq8NRptTcOhpO2swo5D+1NBsOWtcHwTIyD4ds/+Bf5pY9/MW//4F/kde/9dPqDdkbtTbZ7010P5ldv+lJuuuvBuWsPgOlRkQPYIsNTVSFPl+GpqpAAW6UiBzCHplk1nHZ7qpBD2xUMz9S0q4aLWoUE6AoVOQBOmyqkRXJOVxeqkOM2rZ4KdIEgB8BcsEiO4amna94XtelC0AS6y9BKADgFhqfO1/DUZP4XtTE8FdhOKnIAMAOGp85fFdLw1PmsGqpCwsYEOQDYJRZpeGoy/3MX5z1oJos5PHXcrnBI1xlaCQDsiO1YjXWaQ1QNTx3azcNTk+0Zomp4KrOgIgcAdNY8Vw0NTx2ap+GpyfxXIcdtTrtiqAq5+whyAAAjhqfu7uGpyfzPXdyuYNiFsMnpMbQSAKAjDE/dWnvJ9Ieozvvw1O1o0/DU+aAiBwCwwOa5ariIVcjtGE66qFXI3V4xFOQAAJiaeR6eOm5vnucubsdw0nkPm10IhvPI0EoAABbKNIeozvvw1O1ocxGHp84jQQ4AALZgnoPhdrS5aMFwXglyAAAwR6a9qM2021y0YDivBDkAAOC0LFIwnFcWOwEAAGZqnldPnVeCHAAAsKtMe7XTeWRoJQAAQMcIcgAAAB0jyAEAAHSMIAcAANAxghwAAEDHCHIAAAAdI8gBAAB0jCAHAADQMYIcAABAxwhyAAAAHSPIAQAAdIwgBwAA0DFTCXJVdWVVfaGq7q6qd27w+jOq6g+q6vaquqOq3jSN6wIAACyiLQe5qlpK8p4kVyV5SZLXVtVL1p3240nubK29NMkVSd5dVWdv9doAAACLaBoVuZcnubu1dk9rbTnJh5Jcve6cluRpVVVJzk/yaJLeFK4NAACwcKYR5C5Oct/E/v7RsUm/luTFSQ4k+VySn2itDTZqrKquqapbquqWhx9+eArdAwAA2F2mEeRqg2Nt3f4PJrktyfOSXJ7k16rq6Rs11lq7rrW2r7W276KLLppC9wAAAHaXaQS5/Ukumdh/foaVt0lvSvLRNnR3ki8n+S+mcG0AAICFM40gd3OSF1bVC0YLmLwmyQ3rzvlqklcmSVU9J8mLktwzhWsDAAAsnL1bbaC11quqtyX5WJKlJNe31u6oqreOXr82yf+S5Der6nMZDsV8R2vt61u9NgAAwCLacpBLktbajUluXHfs2ontA0l+YBrXAgAAWHRTeSA4AAAAO0eQAwAA6BhBDgAAoGMEOQAAgI4R5AAAADpGkAMAAOgYQQ4AAKBjBDkAAICOEeQAAAA6RpADAADoGEEOAACgYwQ5AACAjhHkAAAAOkaQAwAA6BhBDgAAoGMEOQAAgI4R5AAAADpGkAMAAOgYQQ4AAKBjBDkAAICOEeQAAAA6RpADAADoGEEOAACgYwQ5AACAjhHkAAAAOkaQAwAA6BhBDgAAoGMEOQAAgI4R5AAAADpGkAMAAOgYQQ4AAKBjBDkAAICOEeQAAAA6RpADAADoGEEOAACgYwQ5AACAjhHkAAAAOkaQAwAA6JipBLmqurKqvlBVd1fVOzc554qquq2q7qiq/3ca1wUAAFhEe7faQFUtJXlPklcl2Z/k5qq6obV258Q5FyT5N0mubK19taqevdXrAgAALKppVORenuTu1to9rbXlJB9KcvW6c340yUdba19NktbaQ1O4LgAAwEKaRpC7OMl9E/v7R8cm/dUk31JVn6iqW6vq9Zs1VlXXVNUtVXXLww8/PIXuAQAA7C7TCHK1wbG2bn9vkr+e5L9J8oNJ/klV/dWNGmutXdda29da23fRRRdNoXsAAAC7y5bnyGVYgbtkYv/5SQ5scM7XW2sHkxysqk8meWmSL07h+gAAAAtlGhW5m5O8sKpeUFVnJ3lNkhvWnfP7Sb6/qvZW1XlJXpHkrilcGwAAYOFsuSLXWutV1duSfCzJUpLrW2t3VNVbR69f21q7q6r+OMlnkwyS/EZr7fNbvTYAAMAimsbQyrTWbkxy47pj167b/1dJ/tU0rgcAALDIpvJAcAAAAHaOIAcAANAxghwAAEDHCHIAAAAdI8gBAAB0jCAHAADQMYIcAABAxwhyAAAAHSPIAQAAdIwgBwAA0DGCHAAAQMcIcgAAAB0jyAEAAHSMIAcAANAxghwAAEDHCHIAAAAdI8gBAAB0jCAHAADQMYIcAABAxwhyAAAAHSPIAQAAdIwgBwAA0DGCHAAAQMcIcgAAAB0jyAEAAHSMIAcAANAxghwAAEDHCHIAAAAdI8gBAAB0jCAHAADQMYIcAABAxwhyAAAAHSPIAQAAdIwgBwAA0DGCHAAAQMcIcgAAAB0jyAEAAHSMIAcAANAxUwlyVXVlVX2hqu6uqnee4Lz/sqr6VfX3p3FdAACARbTlIFdVS0nek+SqJC9J8tqqeskm5/3LJB/b6jUBAAAW2TQqci9Pcndr7Z7W2nKSDyW5eoPz3p7kI0kemsI1AQAAFtY0gtzFSe6b2N8/Oraqqi5O8veSXDuF6wEAACy0aQS52uBYW7f/y0ne0Vrrn7Sxqmuq6paquuXhhx+eQvcAAAB2l71TaGN/kksm9p+f5MC6c/Yl+VBVJcmFSV5dVb3W2u+tb6y1dl2S65Jk37596wMhAADAwptGkLs5yQur6gVJ7k/ymiQ/OnlCa+0F4+2q+s0kf7hRiAMAAODkthzkWmu9qnpbhqtRLiW5vrV2R1W9dfS6eXEAAABTNI2KXFprNya5cd2xDQNca+2N07gmAADAoprKA8EBAADYOYIcAABAxwhyAAAAHSPIAQAAdIwgBwAA0DGCHAAAQMcIcgAAAB0jyAEAAHSMIAcAANAxghwAAEDHCHIAAAAdI8gBAAB0jCAHAADQMYIcAABAxwhyAAAAHSPIAQAAdIwgBwAA0DGCHAAAQMcIcgAAAB0jyAEAAHSMIAcAANAxghwAAEDHCHIAAAAdI8gBAAB0jCAHAADQMYIcAABAxwhyAAAAHSPIAQAAdIwgBwAA0DGCHAAAQMfsnXUHAAAAdspg0NKSDFpLa8PfS3sqZy11q8YlyAEAANumP2gZtLYanI7tD0PVeHu5P0hacuCxw6vntpa0DF9vo/Oyup817a4/94kjvSTJzfc+unpuaxv38eILzs2lzzpvx/6ZTIMgBwAAC2QciMaBqrWkPw5Eg2Pbg9YyGOS47X5raa3l8HI/LckdB755fEBrbRTSTr1fR5b7SZKvPHJoap8zSXr90+hEhwhyAAAwBwaDtiZEtZY8cWRlTYA6rqo1GbTWBbT1rz1xpJeWlk/d8+hU+rvSHyRJHj/cm0p7nB5BDgAATqC1NgpHo0A1OL5CtdIfpI2GBfYHx+ZejatX/cGx905WwAYT7U4O+3tyNCzw8/c/PtXPwe4hyAEA0FnjOVb9DSpV68NSf3W75cjKcFjgFx98Ym3AGhxf3TqV/HN4ysMC4WQEOQAAts04GE3On+oP2oYBbLJaNdwevn5ouZ/WWm6/77EzClkbWe4NhwU+8uTyFD8t7BxBDgBggQ1GYahluHjFmhA1nrM1yMZhrE2EtIlhhuPVAv/sLx+ZSh97o7lYh0ZVL0CQAwCYe+NFMCZXGfzm4ZVji2OsGzo4DmDjuVnHha6JOVvjBTWS5Lb7HptKf83Fgu03lSBXVVcm+ZUkS0l+o7X2rnWv/1iSd4x2n0zyP7bWbp/GtQEA5sH6CtY4ePVG1a2HnjhybN7W4PhQdbKwNTZeBOPOA9NbBAPoni0HuapaSvKeJK9Ksj/JzVV1Q2vtzonTvpzkb7XWvlFVVyW5LskrtnptAIDTtVngGle3xqsP3v/Y4TXVrTVVsXULaYxXNNzIoaPD4PWXDx3cwU8J7HbTqMi9PMndrbV7kqSqPpTk6iSrQa619v9NnP+pJM+fwnUBgF1u/ZDCyRA1PrbcG6Ql+cojBzescK0JYqfwgOLx6oNftfogMMemEeQuTnLfxP7+nLja9uYkf7TZi1V1TZJrkuTSSy+dQvcAgJ2wYaWr304pdPUGg+MC2qmuSHhkZRi8Djx2ZHs/IMAcmUaQqw2ObfjHblX97QyD3Pdt1lhr7boMh15m3759ZsoCwDZpbTh/a321qzcYrIaxo6Ml2r/89YNrK1zrtk+l0pUIXQDTMo0gtz/JJRP7z09yYP1JVfXXkvxGkqtaa9NZixYAFsxw8YzBhhWv/qCtBq97v34wvQ2CV29w6kMMk+ToKHh97ZuCF8A8mUaQuznJC6vqBUnuT/KaJD86eUJVXZrko0le11r74hSuCQCdMRm+Jn+PF9XY/41DE8MKB6vzu8bDFCerXiczDl4PCF4Au9qWg1xrrVdVb0vysQwfP3B9a+2Oqnrr6PVrk/xskmcl+TdVlSS91tq+rV4bALbLYF2I6g1aev2WlpYHHz+yWtmaHJrY6x+rep1K+BovqnHfo4d36mMBsEtM5TlyrbUbk9y47ti1E9tvSfKWaVwLAE5kfQBrSR49uLy2wrU6HHFwXOg60XyvQ8vDZeTvedgy8gDM1lSCHABMQ8twAY5Dy701lbDJyld/cHzFqzcYTFTEjrV3cPT8ri987YnZfCAA2CaCHABTMQ5U/Q0C19owNtg0lD1xeCVJcvt935zxpwGA+SbIAbBmGfreYDj0cLwEfX80N2xtMBvk4NFeWpKb7330lJ/3BQBMhyAHsAtsGsQmwteRlX5akv/8tcePC2anshrieuP39PoSHADsNEEOYE5MDk2cXCHxgW8e3rAiNnneqQSx5dHzxb5xcGW7PwoAsM0EOYAp2ihkre73184RW7PfP36VxPEKifd+/dAMPgkAMM8EOYAN9PqjsDVox7ZHoWwcwMbb/UHLE0d6aWn58y8/OuuuAwALQJADdrVxCFvpD6tiK6MwdrQ3SGstX3rwibUhbfzssdOc9tWs9AEA7CBBDuiE8fPFDh7tpddvWRmMg9nxFbLJKtpm+eroSj9J8vUnl3fuQwAATIkgB+y41QA2Clwro+GJ64/1BqPf/cHq88U+u9/zxQAABDlgS1o7NlxxpTeslPX6bc3QxfWh7AxWugcAYIIgB6wxGLQsr6uMjYcvjkPaSn8wUT3bOJUZuggAsH0EOdjlxhWzcRgbhrRj1bOV/iAHj/bSkvz5lx89owdDAwCwswQ56KDxwh9PHFlZHa643B+s2R5Xzk604MfYOLwJcQAA3SDIwZxora2GsZXesFK2PDG0cXl0bKXfVhf++Pz9j8+41wAAzIIgB9tspT/IoLUMWvLQE0fWzDFb7h3bXtlkrhkAAKwnyMEZWh3O2Gs52u9npd9Wq2bLvfFrwxUanzzSS5L85UMHZ9xrAAB2A0EO1hnPP/vm4ZU1wWylP8jRieGN5pMBADArghwLZTAYPt9suTfI0X5/WDkbVc+Orgx/j+ef3XnA/DMAAOaTIMeuMX7+2dGV40PaeNs8NAAAdgNBjs7oD1qOrPRztDfI0V5/GNh6w2egDVry6S8/OusuAgDAjhDkmBubBbWjveGx3ibVNHPVAABYNIIcO6a1liMrw0A2aC1feeRgjvYGq+Fts6AGAACsJcgxVa211XB2ZGX4+/BKfzWstZYcWh4uxX/gsSMz7i0AAHSTIMcZOdrr58jyIEd6/YmwNsjRlX6MdAQAgO0lyLGpwWD4rLRBa/nqI4dypNfP4eVhcBPWAABgdgQ5kiRHVvo5tNzPoeXe6PcwsB08OhwGef9jh2fcQwAAYEyQWzD9QcvB5V4OLw9D2qHl4bBIC40AAEB3CHK71HjRkXFYO7Tcz8HlXo6uDGbdNQAAYIsEuV1gpT/IoaP9HFrp5fBKP4NBy833fsPz1QAAYJcS5DqkP2g5vNLPoYkq2+GVXpZ7xwLbSm+wei4AALA7CXJzaPzg7MmFRw4t91afwwYAACw2QW7GBq1l0JIDjx1eDW6Hly3vDwAAbE6Q2yG9/iCHVoYh7dBoxcjDK/08eWS4vP9XHjk04x4CAABdIchtk+XeIF/75pEcWunl4NF+lntWiwQAAKZDkNsmR3p9D9EGAAC2xZ5ZdwAAAIDTI8gBAAB0zFSCXFVdWVVfqKq7q+qdG7xeVfWro9c/W1XfPY3rAgAALKItB7mqWkryniRXJXlJktdW1UvWnXZVkheOfq5J8utbvS4AAMCimkZF7uVJ7m6t3dNaW07yoSRXrzvn6iTvb0OfSnJBVT13CtcGAABYONXa1p48XVV/P8mVrbW3jPZfl+QVrbW3TZzzh0ne1Vr709H+TUne0Vq75URtP/PbXtxe9VPXb6l/03bb7bclSS5/6eUnPK8/aDm43Dtpe1+68/NJkhe+5Lu23rkOtLcdbc57e9vR5ry3tx1tLlp729HmvLe3HW0uWnvb0ea8t7cdbS5ae9vR5ry3tx1tznt729HmbmrvnL1LOWfvfC4f8uG3/le3ttb2rT8+jSD3w0l+cF2Qe3lr7e0T5/zfSf7FuiD3P7XWbt2gvWsyHH6Z85/77X/91T/3gS31b1ZONcgBAACz1cUgN43nyO1PcsnE/vOTHDiDc5IkrbXrklyXJPv27Wu/+w+/dwpd3HmPH1nJHfc/PutuAAAAJ3HxBefm0medN+tubOjDb934+DRi581JXlhVL6iqs5O8JskN6865IcnrR6tXfk+Sb7bWHpjCtQEAABbOlityrbVeVb0tyceSLCW5vrV2R1W9dfT6tUluTPLqJHcnOZTkTVu9LgAAwKKaxtDKtNZuzDCsTR67dmK7JfnxaVwLAABg0c3njD4AAAA2JcgBAAB0zFSGVnK8885aync8+/wcXu7n4HIvh5b7We4NZt0tAABgFxDktsnepT256GnnrDnW6w9yaKU/DHdHh+Hu8Eo/vf7WnuUHAAAsFkFuB+1d2pOnL+3J059y1prjR3ujcLfcz+FR9e7wcj8D+Q4AANiAIDcHhk+SX8oFE88gbK3lyMogB5d7Obzcz6Hlfg4t93JkxfBMAABYdILcnKqqnHv2Us49e2nN8f6g5dCacNfP4ZVelnvKdwAAsCgEuY5Z2lN52lPOytPWDc9c7g2G4W5lODTz0NFhBc/wTAAA2H0EuV3i7L17cvbePXlGjgW89cMzx6tnHjU8EwAAOk2Q28U2G545Xj1zXLUbD9HsK98BAEAnCHILaLPVM4+sDAPdwaO9HF4Z/j7aG6TJdwAAMFcEOVY95aylPOWspTzzqWevHusPWg6Pnn13ZGX8M8iRnuffAQDArAhynNDSnsr55+zN+ecc/6/Kcm8Y6I6s9HNkeWJ7ZWCYJgAAbCNBjjM2XmBl/RDNZPiQ8yMrgxxd6efwuIo3qujJeAAAsDWCHNti/JDznLs25LXWcrQ3GP0MV9BcDX29QVb65uQBAMDJCHLsqKpanYuXHF/JWw16o4A3DnyCHgAAHCPIMVfONOiNq3wrvYGhmwAA7HqCHJ1ysqCXDBdhWe4Phr9HP+Owt9wX9gAA6D5Bjl1nvAhLztn8nPVh72ivP/p97LghnAAAzCtBjoV0qmFvZRTqVvrH5uit9NtqEDRnDwCAWRDkYBPjsPfUE4S91tow2I2GbK5W+fprQ+ByT9oDAGB6BDnYgqrK2XvrpNW91kZhr9+yMg534/2JwLfSbx6mDgDASQlysAOqavRsvZww8CVJf9BGoW7joLcaAi3aAgCwsAQ5mDNLeypLe8Yrc55Yrz9IbzCs9vX6xwfA8XZv9BsAgN1BkIMO27u0J3uXckqhbzyfb2UU+sbz+MbbvcFkGDTEEwBgnglysCDWzOc7BYNBy8q6cLc+7K2MKoIqfgAAO0uQAza0Z0/lnD2jeX2nYFzx6w2ODefsDY5VAI8dPxYQVf0AAM6MIAdMxWrFL6dW8UvWVnijpToAAAlFSURBVP0mA15vIPwBAJyIIAfMzOlW/ZJh+OsNjoW8/nho56Clv0kY7PWbFT4BgF1FkAM6Zc+eytl7Tq/ylxx7rMMw+A0D3+SxcShcPUcABADmmCAHLITxYx1O13j452TY660u8jIKfKPt/sS5PYu/AADbSJADOIHx8M/T1doo9I1++pOhb/2+EAgAnCZBDmAbVFX2LlX2nn4G3DQEjo8d+z1ZGRztGw4KAAtBkAOYM1sJgcmxBWHGc/2OC4AnCoaCIAB0giAHsMuMF4QZOrN5gRsFwcEJKoL9QUt/XEk0NBQAtp0gB8AaWw2CSdLrD44Fu9UqYMugHV8V7LdjYXD1HGEQAE5IkANg6vYu7dnyf2DWh8HJimB/TWXwWPVwMMixgDgwTBSA3UuQA2AuTSMMDgbrKoHrf9qwOjgMjIP0R0FwHAjH1UGBEIB5I8gBsGutHSZ65tr6ILgmAG4QDk9wrAmFAEzBloJcVT0zye8muSzJvUl+pLX2jXXnXJLk/Um+NckgyXWttV/ZynUBYCdVVc5aqpx1hiuJTjpR6BtXDgcTQ0cH64OhUAhAtl6Re2eSm1pr76qqd47237HunF6Sf9xa+0xVPS3JrVX18dbanVu8NgB0ztKeytIUqoTJ2qGjk3MJ+23tnMLx65NDTAeDrBlS2jd+FKBTthrkrk5yxWj7fUk+kXVBrrX2QJIHRttPVNVdSS5OIsgBwBZMa+jo2OSiMePK3/qwePJgeOx1FUOA7bPVIPecUVBLa+2Bqnr2iU6uqsuSvCzJp09wzjVJrkmSSy+9dIvdAwBO1bBaOIXxoyPrg9+4UnhsO8fNN9wsMKoaAqx10iBXVf8hw/lt6/306Vyoqs5P8pEkP9lae3yz81pr1yW5Lkn27dvnT2wA6KhpDiMdO9VwuPbYRFBUOQR2iZMGudba39nstap6sKqeO6rGPTfJQ5ucd1aGIe63W2sfPePeAgALbTvC4Zqgty7wDU4hCI7DoIAI7KStDq28Ickbkrxr9Pv3159QVZXkvUnuaq394havBwAwVXv2VPZkOquSTlofEActq/MOx5XCQdu4grh6btv4OMBWg9y7kny4qt6c5KtJfjhJqup5SX6jtfbqJH8jyeuSfK6qbhu976daazdu8doAAHNruwJisnaI6cmqiG0U/jYLicfaSAZNJRG6YktBrrX2SJJXbnD8QJJXj7b/NMl0x0AAACyw7RhiOnZcCFzdPlYdXA2HGwwrnQyWg6aaCNtlqxU5AAB2ke2sJCanFxQHbeNK4zgkHqsqxtxEFo4gBwDAjtnuoNjWhbsNK4QTYXAyUA7a8cNP14fF8eswa4IcAAC7RlVl79L2zupp7fiwN96fDJJtNThmFAA3Dpltg7YERk5GkAMAgNNQVVmqbNs8xbHNAuP6yuHqeavHN3jtBG1Y5KabBDkAAJhDOxUYk6wJgOuHlJ6syrhRMBxXGsfDWduacLntH2chCHIAALDgxnMXd8L6KuGawDdIWo6FxkEb7reW1fNahiEyyWqQHJ+b1fdmTdvjcyfbaKv7SXVwjX1BDgAA2DE7WWnczfbMugMAAACcHkEOAACgYwQ5AACAjhHkAAAAOkaQAwAA6BhBDgAAoGMEOQAAgI4R5AAAADpGkAMAAOgYQQ4AAKBjBDkAAICOEeQAAAA6RpADAADoGEEOAACgY6q1Nus+bKqqHk7ylVn3YwMXJvn6rDvBGu7JfHJf5o97Mn/ck/nkvswf92Q+uS/b79taaxetPzjXQW5eVdUtrbV9s+4Hx7gn88l9mT/uyfxxT+aT+zJ/3JP55L7MjqGVAAAAHSPIAQAAdIwgd2aum3UHOI57Mp/cl/njnswf92Q+uS/zxz2ZT+7LjJgjBwAA0DEqcgAAAB0jyJ2Gqrqyqr5QVXdX1Ttn3R+GqureqvpcVd1WVbfMuj+LqKqur6qHqurzE8eeWVUfr6ovjX5/yyz7uIg2uS8/X1X3j74vt1XVq2fZx0VTVZdU1f9TVXdV1R1V9ROj474vM3KCe+K7MkNV9ZSq+vOqun10X35hdNx3ZUZOcE98V2bE0MpTVFVLSb6Y5FVJ9ie5OclrW2t3zrRjpKruTbKvteYZJjNSVX8zyZNJ3t9a+67Rsf8tyaOttXeN/sfHt7TW3jHLfi6aTe7Lzyd5srX2v8+yb4uqqp6b5Lmttc9U1dOS3Jrk7yZ5Y3xfZuIE9+RH4rsyM1VVSZ7aWnuyqs5K8qdJfiLJD8V3ZSZOcE+ujO/KTKjInbqXJ7m7tXZPa205yYeSXD3jPsFcaK19Msmj6w5fneR9o+33ZfgXI3bQJveFGWqtPdBa+8xo+4kkdyW5OL4vM3OCe8IMtaEnR7tnjX5afFdm5gT3hBkR5E7dxUnum9jfH3/Qz4uW5E+q6taqumbWnWHVc1prDyTDvyglefaM+8Mxb6uqz46GXhqWNCNVdVmSlyX5dHxf5sK6e5L4rsxUVS1V1W1JHkry8daa78qMbXJPEt+VmRDkTl1tcMz/hZgPf6O19t1Jrkry46PhZMDGfj3Jtye5PMkDSd492+4spqo6P8lHkvxka+3xWfeHDe+J78qMtdb6rbXLkzw/ycur6rtm3adFt8k98V2ZEUHu1O1PcsnE/vOTHJhRX5jQWjsw+v1Qkv8rw2GwzN6Do7kn4zkoD824PyRprT04+g/xIMn/Gd+XHTeaW/KRJL/dWvvo6LDvywxtdE98V+ZHa+2xJJ/IcC6W78ocmLwnviuzI8idupuTvLCqXlBVZyd5TZIbZtynhVdVTx1NTk9VPTXJDyT5/InfxQ65IckbRttvSPL7M+wLI+O/AI38vfi+7KjRYgHvTXJXa+0XJ17yfZmRze6J78psVdVFVXXBaPvcJH8nyX+O78rMbHZPfFdmx6qVp2G0nOovJ1lKcn1r7X+dcZcWXlX9lQyrcEmyN8nvuC87r6o+mOSKJBcmeTDJzyX5vSQfTnJpkq8m+eHWmoU3dtAm9+WKDIe/tCT3JvmH4/kmbL+q+r4k/zHJ55IMRod/KsM5Wb4vM3CCe/La+K7MTFX9tQwXM1nKsPDw4dbaP62qZ8V3ZSZOcE8+EN+VmRDkAAAAOsbQSgAAgI4R5AAAADpGkAMAAOgYQQ4AAKBjBDkAAICOEeQAAAA6RpADAADoGEEOAACgY/5/991QYYWkwzIAAAAASUVORK5CYII= ",
text/plain: [
"<Figure size 1080x432 with 1 Axes>"
]
},
metadata: {
needs_background: "light"
},
output_type: "display_data"
},
{
data: {
image/png: "iVBORw0KGgoAAAANSUhEUgAAA2oAAAF1CAYAAACZNBlsAAAABHNCSVQICAgIfAhkiAAAAAlwSFlzAAALEgAACxIB0t1+/AAAADh0RVh0U29mdHdhcmUAbWF0cGxvdGxpYiB2ZXJzaW9uMy4yLjIsIGh0dHA6Ly9tYXRwbG90bGliLm9yZy+WH4yJAAAgAElEQVR4nO3dfbRd5X0f+O9PVwiLt4BBEAMyEJthwF6xQlWoJ0lD40kC7rR2vdIUO8tviwz1jEmTrKwZu64ncdLESafjxvHYDXVjYofEdt3Y49IU20nJUCdt4iKo/AIMRsHGkoWRwCgYkBG6euaPcy6+ulxJF2lfnefqfD5raenuffbZ57fPfv3u/ex9qrUWAAAA+rFq0gUAAACwP0ENAACgM4IaAABAZwQ1AACAzghqAAAAnRHUAAAAOiOoATC4qnqsqr5nCcOdX1WtqlYfjbp6VVVvqKo/O4L3f6qqXj9kTQBMlqAGMIWq6qtVtXscqB6sqt+pqpMOc1y3VtVPze/XWjuptXbfMNU+/RmPVNXxz/J9rapeOFQdPaiqd1TV783v11q7qrX2oUnVBMDwBDWA6fV3WmsnJbk0yV9P8vZn8+YaWfb9SFWdn+QHk7Qkf3e5P+9ILXZ1cNqvGALw7AlqAFOutfb1JJ9K8uKqOq2q/rCqdo6vYP1hVZ07N+z4ytavVtV/TvJEkhszClHvHV+de+94uKevZFXV366q/1ZVj1bV1qp6x7Ms8XVJ/iLJB5Ps17xv4dW8+U0Iq+qz496fH9f2D8b9/+eq2lJV36yqm6rq7Hnvf1FV/fH4tQer6m3j/sdX1buravv437vnru5V1RVVta2q3lJV30jyO+OrXn9QVb9XVY8meUNVfVdVfaCqHqiqr1fVr1TVzGITXFW/Of6uHq2q26vqB8f9r0zytiT/YDxNn1/4PVTVqqp6e1XdX1U7qup3q+q7xq/NNTV9fVV9raoeqqp/8iznBwBHgaAGMOWqan2Slyf5bxntF34nyXlJnp9kd5L3LnjLa5Ncm+TkJG9I8qdJrhs3d7xukY94PKOwdWqSv53kf6mqVz6LEl+X5PfH/36sqs5ayptaa39z/OdLxrX9m6r64SS/luQnkjwvyf1JPpokVXVykv+Y5NNJzk7ywiS3jMfxT5L8jSQbkrwkyWXZ/wrkdyd5bkbf27Xjfq9I8gfj6f79JB9Ksnc83u9L8qNJ9msyOs9t4896bpIPJ/m3VfWc1tqnk7wzyb8ZT9NLFnnvG8b//laS70lyUp45D38gyUVJXpbkF6rq4gPUAcCECGoA0+uTVbUryZ8l+U9J3tlae7i19vHW2hOttW8l+dUkP7TgfR9srd3ZWtvbWnvqUB/SWru1tfbF1tq+1toXknxkkXEuqqp+IKPw87HW2u1J/jLJa5Y+ic/wk0luaK3d0Vp7Msk/TvLScfPK/ynJN1pr72qtfbu19q3W2ufmve+XW2s7Wms7k/xSRoF1zr4kv9hae7K1tnvc789ba59sre1LckqSq5L8bGvt8dbajiS/keTqxYpsrf3eeF7sba29K8nxGQWrpU7jv2it3ddae2w8jVcvaH75S6213a21zyf5fEbhE4COaDMPML1e2Vr7j/N7VNUJGQWIK5OcNu59clXNtNZmx91bn82HVNXlSX49yYuTrMkodPzbJb799Un+qLX20Lj7w+N+v/Fsapjn7CR3zHW01h6rqoeTnJNkfUZB8EDvu39e9/3jfnN2tta+veA987+n85Icl+SBqprrtyoH+C6r6uczutp2dkb35p2S5IwDTtWha12dZP6VyG/M+/uJjK66AdARV9QAmO/nM7pyc3lr7ZQkc80Ha94wbcF7FnYv9OEkNyVZ31r7riTXLxjfoqpqbUZNFH+oqr4xvv/r55K8pKrmrgA9nuSEeW/77kOMdntGoWnuM05McnqSr2cUml6wlPdl1Cx0+7zuxb6D+f22JnkyyRmttVPH/05prb1o4ZvG96O9JaNpP621dmqSv8p3vrNDfd+L1bo3yYOHeB8AHRHUAJjv5IzuS9tVVc9N8otLeM+DGd0LdbBxfrO19u2quixLb7r4yiSzSS7J6H6tDUkuzuieuNeNh9mc5FVVdcL44SXXHKK2Dyd5Y1VtGD8M5J1JPtda+2qSP0zy3VX1s+OHh5w8vhqYjJprvr2q1lXVGUl+Icl+j8g/mNbaA0n+KMm7quqU8QM/XlBVizUBPTmjYLUzyeqq+oWMrqjNn6bzD/LEzY8k+bmquqBGP7kwd0/b3qXWC8DkCWoAzPfuJGuTPJTRkxY/vYT3/GaSHx8/JfI9i7z+vyb55ar6VkYB52NLrOX1SX6ntfa11to35v5l9GCMnxzfc/UbSfZkFF4+lNFDO+Z7R5IPVdWuqvqJ1totSf6PJB9P8kBGV9CuTpLxPXk/kuTvZNQ08N6MHsiRJL+SZFOSLyT5YkbNJ39lidMx53UZNf28K8kjGT1o5HmLDPeZjJ7C+eWMmi1+O/s3kZxrNvpwVd2RZ7oho6dxfjbJV8bv/+lnWSsAE1atHaoFBQAAAEeTK2oAAACdEdQAAAA6I6gBAAB0RlADAADojKAGAADQmdWT+uAzzjijnX/++ZP6eAAAgIm6/fbbH2qtrVvstYkFtfPPPz+bNm2a1McDAABMVFXdf6DXNH0EAADojKAGAADQGUENAACgM4IaAABAZwQ1AACAzghqAAAAnRHUAAAAOiOoAQAAdEZQAwAA6Mwhg1pV3VBVO6rqSwd4varqPVW1paq+UFWXDl/m8pvd13LL3Q/mPbfcm1vufjCz+9qkSwIAAKbU6iUM88Ek703yuwd4/aokF47/XZ7kt8b/rxiz+1pe+4HPZfPWXdm9ZzZr18xkw/pTc+M1l2dmVR3ReG+9Z0fu3P5oXnT2KbniojOPaHwAAMB0OGRQa619tqrOP8ggr0jyu621luQvqurUqnpea+2BgWpcdrfesyObt+7KE3tmkyRP7JnN5q27cus9O/Kyi886rHEuV/gDAACOfUPco3ZOkq3zureN+z1DVV1bVZuqatPOnTsH+Ohh3Ln90eweh7Q5u/fM5q7tjx72OOeHv5b9wx8AAMDBDBHUFrs8tOgNXq2197fWNrbWNq5bt26Ajx7Gi84+JWvXzOzXb+2amVxy9imHPc7lCH8AAMB0GCKobUuyfl73uUm2DzDeo+aKi87MhvWnpmb3JG1fThg3U7ziojMPe5zLEf4AAIDpMERQuynJ68ZPf/wbSf5qJd2fliQzqyo3XnN51t3773Pqtv+c//vV33fE95ItR/gDAACmwyEfJlJVH0lyRZIzqmpbkl9MclyStNauT3Jzkpcn2ZLkiSRvXK5il9PMqsoJu+7LCbvuO+wHiCwc343XXJ6Xvuqa7DnxzLzr7T/nqY8AAMCSLOWpj68+xOstyZsHq+gYMnT4AwAApsMQTR8BAAAYkKAGAADQGUENAACgM4IaAABAZwQ1AACAzghqAAAAnRHUAAAAOiOoAQAAdEZQAwAA6IygBgAA0BlBDQAAoDOCGgAAQGcENQAAgM4IagAAAJ0R1AAAADojqAEAAHRGUAMAAOiMoAYAANAZQQ0AAKAzghoAAEBnBDUAAIDOCGoAAACdEdQAAAA6I6gBAAB0RlADAADojKAGAADQGUENAACgM4IaAABAZwQ1AACAzghqAAAAnRHUAAAAOiOoAQAAdEZQAwAA6IygBgAA0BlBDQAAoDOCGgAAQGcENQAAgM4IagAAAJ0R1AAAADojqAEAAHRGUAMAAOiMoAYAANAZQQ0AAKAzghoAAEBnBDUAAIDOCGoAAACdEdQAAAA6I6gBAAB0ZklBraqurKp7qmpLVb11kde/q6r+fVV9vqrurKo3Dl8qAADAdDhkUKuqmSTvS3JVkkuSvLqqLlkw2JuT3NVae0mSK5K8q6rWDFwrAADAVFjKFbXLkmxprd3XWtuT5KNJXrFgmJbk5KqqJCcl+WaSvYNWCgAAMCWWEtTOSbJ1Xve2cb/53pvk4iTbk3wxyc+01vYNUiEAAMCUWUpQq0X6tQXdP5Zkc5Kzk2xI8t6qOuUZI6q6tqo2VdWmnTt3PutiAQAApsFSgtq2JOvndZ+b0ZWz+d6Y5BNtZEuSryT57xeOqLX2/tbaxtbaxnXr1h1uzQAAAMe0pQS125JcWFUXjB8QcnWSmxYM87UkL0uSqjoryUVJ7huyUAAAgGmx+lADtNb2VtV1ST6TZCbJDa21O6vqTePXr0/yT5N8sKq+mFFTybe01h5axroBAACOWYcMaknSWrs5yc0L+l0/7+/tSX502NIAAACm05J+8BoAAICjR1ADAADojKAGAADQGUENAACgM4IaAABAZwQ1AACAzghqAAAAnRHUAAAAOiOoAQAAdEZQAwAA6IygBgAA0BlBDQAAoDOCGgAAQGcENQAAgM4IagAAAJ0R1AAAADojqAEAAHRGUAMAAOiMoAYAANAZQQ0AAKAzghoAAEBnBDUAAIDOCGoAAACdEdQAAAA6I6gBAAB0RlADAADojKAGAADQGUENAACgM4IaAABAZwQ1AACAzghqAAAAnRHUAAAAOiOoAQAAdEZQAwAA6IygBgAA0BlBDQAAoDOCGgAAQGcENQAAgM4IagAAAJ0R1AAAADojqAEAAHRGUAMAAOiMoAYAANAZQQ0AAKAzghoAAEBnBDUAAIDOCGoAAACdEdQAAAA6s6SgVlVXVtU9VbWlqt56gGGuqKrNVXVnVf2nYcsEAACYHqsPNUBVzSR5X5IfSbItyW1VdVNr7a55w5ya5F8mubK19rWqOnO5CgYAADjWLeWK2mVJtrTW7mut7Uny0SSvWDDMa5J8orX2tSRpre0YtkwAAIDpsZSgdk6SrfO6t437zfffJTmtqm6tqtur6nVDFQgAADBtDtn0MUkt0q8tMp6/luRlSdYm+fOq+ovW2pf3G1HVtUmuTZLnP//5z75aAACAKbCUK2rbkqyf131uku2LDPPp1trjrbWHknw2yUsWjqi19v7W2sbW2sZ169Ydbs0AAADHtKUEtduSXFhVF1TVmiRXJ7lpwTD/LskPVtXqqjohyeVJ7h62VAAAgOlwyKaPrbW9VXVdks8kmUlyQ2vtzqp60/j161trd1fVp5N8Icm+JL/dWvvSchYOAABwrFrKPWpprd2c5OYF/a5f0P3Pk/zz4UoDAACYTkv6wWsAAACOHkENAACgM4IaAABAZwQ1AACAzghqAAAAnRHUAAAAOiOoAQAAdEZQAwAA6IygBgAA0BlBDQAAoDOCGgAAQGcENQAAgM4IagAAAJ0R1AAAADojqAEAAHRGUAMAAOiMoAYAANAZQQ0AAKAzghoAAEBnBDUAAIDOCGoAAACdEdQAAAA6I6gBAAB0RlADAADojKAGAADQGUENAACgM4IaAABAZwQ1AACAzghqAAAAnRHUAAAAOiOoAQAAdEZQAwAA6IygBgAA0BlBDQAAoDOCGgAAQGcENQAAgM4IagAAAJ0R1AAAADojqAEAAHRGUAMAAOiMoAYAANAZQQ0AAKAzghoAAEBnBDUAAIDOCGoAAACdEdQAAAA6I6gBAAB0RlADAADozJKCWlVdWVX3VNWWqnrrQYb761U1W1U/PlyJAAAA0+WQQa2qZpK8L8lVSS5J8uqquuQAw/2zJJ8ZukgAAIBpspQrapcl2dJau6+1tifJR5O8YpHhfjrJx5PsGLA+AACAqbOUoHZOkq3zureN+z2tqs5J8veSXD9caQAAANNpKUGtFunXFnS/O8lbWmuzBx1R1bVVtamqNu3cuXOpNQIAAEyV1UsYZluS9fO6z02yfcEwG5N8tKqS5IwkL6+qva21T84fqLX2/iTvT5KNGzcuDHsAAABkaUHttiQXVtUFSb6e5Ookr5k/QGvtgrm/q+qDSf5wYUgDAABgaQ4Z1Fpre6vquoye5jiT5IbW2p1V9abx6+5LAwAAGNBSrqiltXZzkpsX9Fs0oLXW3nDkZQEAAEyvJf3gNQAAAEePoAYAANAZQQ0AAKAzghoAAEBnBDUAAIDOCGoAAACdEdQAAAA6I6gBAAB0RlADAADojKAGAADQGUENAACgM4IaAABAZwQ1AACAzghqAAAAnRHUAAAAOiOoAQAAdEZQAwAA6IygBgAA0BlBDQAAoDOCGgAAQGcENQAAgM4IagAAAJ0R1AAAADojqAEAAHRGUAMAAOiMoAYAANAZQQ0AAKAzghoAAEBnBDUAAIDOCGoAAACdEdQAAAA6I6gBAAB0RlADAADojKAGAADQGUENAACgM4IaAABAZwQ1AACAzghqAAAAnRHUAAAAOiOoAQAAdEZQAwAA6IygBgAA0BlBDQAAoDOCGgAAQGcENQAAgM4IagAAAJ0R1AAAADojqAEAAHRGUAMAAOjMkoJaVV1ZVfdU1Zaqeusir/9kVX1h/O+/VNVLhi8VAABgOhwyqFXVTJL3JbkqySVJXl1VlywY7CtJfqi19r1J/mmS9w9dKAAAwLRYyhW1y5Jsaa3d11rbk+SjSV4xf4DW2n9prT0y7vyLJOcOWyYAAMD0WEpQOyfJ1nnd28b9DuSaJJ9a7IWquraqNlXVpp07dy69SgAAgCmylKBWi/Rriw5Y9bcyCmpvWez11tr7W2sbW2sb161bt/QqAQAApsjqJQyzLcn6ed3nJtm+cKCq+t4kv53kqtbaw8OUBwAAMH2WckXttiQXVtUFVbUmydVJbpo/QFU9P8knkry2tfbl4csEAACYHoe8otZa21tV1yX5TJKZJDe01u6sqjeNX78+yS8kOT3Jv6yqJNnbWtu4fGUDAAAcu5bS9DGttZuT3Lyg3/Xz/v6pJD81bGkAAADTaUk/eA0AAMDRI6gBAAB0RlADAADojKAGAADQGUENAACgM4IaAABAZwQ1AACAzghqAAAAnRHUAAAAOiOoAQAAdEZQAwAA6IygBgAA0BlBDQAAoDOCGgAAQGcENQAAgM4IagAAAJ0R1AAAADojqAEAAHRGUAMAAOiMoAYAANAZQQ0AAKAzghoAAEBnBDUAAIDOCGoAAACdEdQAAAA6I6gBAAB0RlADAADojKAGAADQGUENAACgM4IaAABAZwQ1AACAzghqAAAAnRHUAAAAOiOoAQAAdEZQAwAA6MzqSRcABzO7r+XWe3bkzu2P5kVnn5IrLjozM6tq0mUBAMCyEtTo1uy+ltd+4HPZvHVXdu+Zzdo1M9mw/tTceM3lwhoAAMc0TR9XkNl9Lbfc/WDec8u9ueXuBzO7r026pGV16z07snnrrjyxZzYtyRN7ZrN5667ces+OSZcGAADLyhW1FWIary7duf3R7N4zu1+/3Xtmc9f2R/Oyi8+aUFUwHTQ7BoDJEtRWiPlXl5L9ry4dq6HlRWefkrVrZp6e5iRZu2Yml5x9ygSrgmPfNJ4YAoDeaPq4Qhzs6tKx6oqLzsyG9aemZvckbV9OGB8sXnHRmZMuDY5pmh0DLK9pu52Fw+OK2goxjVeXZlZVbrzm8rz0Vddkz4ln5l1v/znNr+Ao0OwYYPlotcBSCWqH6c//8uElDffo7qee1fAH8pzVM7ngjBNz59ceSmZW5/jjVueCM07Mc1bPHPG4e7f3gXuyKvfkhDX/W/7rV7456XLgmDdTlTWrV+XJvfue7rdm9aqsqjrmtzcAy+2O+x/J7fc/8vQ29ok9s7n9/kdy/a1/mUvPO23C1R27XvqC0yddwrOm6eMKsWpV5W1XXZyT7vpk1n7lT/OPfvjCvO2qi7PKmZdnZd++ljvufySfuGNb7rj/kezT1ACeYcP6U/PCM09K9o6aHR+/elVeeOZJ2bD+1EmXtqxsH4Cj4asPP549806EJcmevfvy1Ycfn1BF9MoVtRVk1arKmoe3JA9vyaXnvWXS5aw4+/a1vPNTd2fLjseyZ+++rBkffB5J4N23r2Xz1l356sOP5/zTT8yG9ace8+F5Gqd52sydGPqHP/PzmT3prFz3pmuP+fm8HNsHOBy2sce+808/cdFWC+effuIEq6JHghpTY/PWXdmy47GnN4xP7t2XLTsey+atuw6rqcFKObAbcqe/UqaZI9f7iaGhD2aH3j6sFEJBX2xjh9H7cj3XamH+7SzT0GqBZ09QY2ocrKnB4RyIrYQDu6F3+ithmjn2LcfB7NDbh5VgJYSC3g+4h2Ybe+RWwnI9ja0WkuHX52nYPghqU2waFvD5hm5qsBIO7Ibe6a+EaU6mb9lOpmual+NgdhqbIvUeClbCAffQVso2tme9L9dzem+1MLSh1+dp2T4sKahV1ZVJfjPJTJLfbq39+oLXa/z6y5M8keQNrbU7Bq6VAU3LAj7f0E0NVsKB3dA7/ZUwzdO4bE/bNC/Hwew0NkXqPRQs1wF3zyc1VsI2tne9L9fTauj1eaUE8iN1yKc+VtVMkvcluSrJJUleXVWXLBjsqiQXjv9dm+S3Bq6Tgc1fwFv2X8CPVUM/OXMlPBlvbqc/35Hs9FfCNE/jsj1t0zz0cp1M55N1l+N7HPLJmcvxZLy5kxrv+ZN78we3b8t7/uTevPNTd3fzhM+VsI1dDkMuN8uxXK8EvT+1duj1eVqenFmtHXxGVtVLk7yjtfZj4+5/nCSttV+bN8y/SnJra+0j4+57klzRWnvgQON97nkXtx952w1HPgUD2vz5zUmSDS/ZcMhhH/32U0sa5713fSlJcuElLz78wpZhfDu/9WQeemzPM/qvO2lNzjj5+CMa99B6/Q6TpLWWL2+5L5lZk7PPfl5OOn4mowvMfWit5Wvf3J0nnnwqSaVWVdYeN5PnP3ftYdfZ+zSvpGV7KMs1zUOve0NZjuV6Tq/TvByG/h7nxrf7qdm0llTliMb3rW/vzdd37c78w5Sq5JxT1+bk5xzenRvLMc6hDb2Nba3lsSdn8+2nZvOc42a622YPvdws5/ZhaENtb4b+DpfD0Ove4YzvlOcc96w/52j42Jv+h9tbaxsXe20p38w5SbbO696W5PIlDHNOkv2CWlVdm9EVt5z0vBcs4aOPrqUEtGdr6J39UON7znEzqcozFvDjj5s5ovEux0FOr99hklRVLrpw+GV5qO+xqvL8567NY0+uyZNPzeb4AXbSyzHNQy43K2XZXgnTPPS61/NyPafXaV6O8Q39PT725OzTB4rJaHnc/dRsHnty9rAOxE46fiZrj5t5xsHnSccf/nL97Xn1zWktefKpw6txztDzZaht7Eo4qTH0crOc24eh1+ehxjP0dzgax7AnDIZen5dj+9CjpVxR+/tJfqy19lPj7tcmuay19tPzhvkPSX6ttfZn4+5bkvzvrbXbDzTejRs3tk2bNg0wCZPx53/58KRLOCLLdU/Lm1/zd5Mk7/vwTUOVOpWm7XsccnpXyrK9EqZ5aNO2XCd9LzdD+8Qd2/IHt2/L/KOKSvLjf+3cvOrScw9rnEPfT3bH/Y/kPX9y7373gB2/elX+0Q9feET3tfQ6X5ZrepPhpnk5lpvl0ut8Hvo7nNunLLxn90j3KZN+6uNLX3D6YX/WcqqqI7qiti3J+nnd5ybZfhjD0JG5+zGGXmH2nP7CzJ50Vu64/5GubtBmekzjsr0c0wzP1nI8CGPVqsql55022MMB5u4BW3hS40juAet5+7ASHqzhASpHbujvcO6+56xek2S4B3UMvT4PPb4eLSWo3Zbkwqq6IMnXk1yd5DULhrkpyXVV9dGMmkX+1cHuT6MPQy7gc2dfHrvklcnM6rznT+7t8ow+06H3ZXs5Dux632H1fDDLMJYjBA1t6JMave/7VkIIWgnLTdL3Nmzo73AlBPxpccig1lrbW1XXJflMRo/nv6G1dmdVvWn8+vVJbs7o0fxbMno8/xuXr2R6tFxnX2DShl62ez+wWw7TOM3TaKVc2R3ypEbv+76VEIJWwnLT+zZs6O9wJQT8abGkOwxbazdnFMbm97t+3t8tyZuHLY2VxNkXjlVDL9u9H9gth2mc5uXQ8xn9Ob1f2R1a7/u+lRCCkv6Xm5WwDRvyO1wJAX9a9PEsWlY8Z184Vg29bPd+YLccpnGah9b7Gf1ptRL2fb2HoJVg2rZhKyXgT4ND/uA1LMXc2ZfjV69KJVPzI53LYe6s+e7zvr/LH62cNkMv29P4Y6zTOM1D2++Mfq065n/UfKWY1n3ftO2npnEbNhfwX3Xpubn0vNOEtAlxRY1BOPsyjGk8a957c66hl+1pbFIyjdOcDLtsT9sZ/ZViGvd907ifmtZtGJMnqB2mXn+LYdK+/8IzJl3CinbL3Q/mKw89vl87+K889Hi+vXc2L7v4rAlXN7zZfS2v/cDn8viLXpm2anXed+uWbFh/am685vLMdLbDH3LZvukFP5Bb79mRu7Y/mkvOPiVXXHRmd9M7tGmb5qGX7Sf27M1/+OIDeWLP7NP91q6ZyZUv/m77ow5M075v2vZTc6ZtG0YfBDXoyJ3bH83ueQdiSbJ7z2zu2v7oMbkDvPWeHdm8dVfazGiH/8Se2Wzeuiu33rPjmJzeOTOrKi+7+KxjehoXmrZpHnrZvuKiM7Nh/anZvHVXdu+Zzdo1M9mw/tRccdGZQ5cOBzVt+6k507YNow+CGnTkRWefkrVrZp5x1vySs0+ZYFXLZ1p3+Bz7hl62Z1ZVbrzmcmf0mbhp20/BJAlq0JFpO2tuh8+xajmWbWf06cG07adgkgQ16Mi0nTW3w+dYZdnmWDVt+ymYpBr9VvXRt3HjxrZp06aJfDbQj9l9zQ6fY5JlG4BDqarbW2sbF31NUAMAADj6DhbU/OA1AABAZwQ1AACAzghqAAAAnRHUAAAAOiOoAQAAdEZQAwAA6IygBgAA0BlBDQAAoDOCGgAAQGcENQAAgM5Ua20yH1y1M8n9E/nwgzsjyUOTLoJnMF/6Y570yXzpj3nSJ/OlP+ZJf8yT5Xdea23dYi9MLKj1qqo2tdY2TroO9me+9Mc86ZP50h/zpE/mS3/Mk/6YJ5Ol6SMAAEBnBDUAAIDOCGrP9P5JF8CizJf+mCd9Ml/6Y570yXzpj3nSH/NkgtyjBgAA0BlX1AAAADojqM1TVVdW1T1VtaWq3jrpekiq6qtV9cWq2lxVmyZdz7SqqhuqakdVfWlev+dW1R9X1b3j/yUteSMAAAOBSURBVE+bZI3T5gDz5B1V9fXx+rK5ql4+yRqnTVWtr6r/t6rurqo7q+pnxv2tKxN0kPlifZmQqnpOVf3Xqvr8eJ780ri/dWWCDjJfrCsTounjWFXNJPlykh9Jsi3JbUle3Vq7a6KFTbmq+mqSja01v+ExQVX1N5M8luR3W2svHvf7P5N8s7X26+MTG6e11t4yyTqnyQHmyTuSPNZa+78mWdu0qqrnJXlea+2Oqjo5ye1JXpnkDbGuTMxB5stPxPoyEVVVSU5srT1WVccl+bMkP5PkVbGuTMxB5suVsa5MhCtq33FZki2ttftaa3uSfDTJKyZcE3ShtfbZJN9c0PsVST40/vtDGR34cJQcYJ4wQa21B1prd4z//laSu5OcE+vKRB1kvjAhbeSxcedx438t1pWJOsh8YUIEte84J8nWed3bYkPeg5bkj6rq9qq6dtLFsJ+zWmsPJKMDoSRnTrgeRq6rqi+Mm0ZqNjQhVXV+ku9L8rlYV7qxYL4k1peJqaqZqtqcZEeSP26tWVc6cID5klhXJkJQ+45apJ+zCJP3/a21S5NcleTN4+ZewOJ+K8kLkmxI8kCSd022nOlUVScl+XiSn22tPTrpehhZZL5YXyaotTbbWtuQ5Nwkl1XViyddEwecL9aVCRHUvmNbkvXzus9Nsn1CtTDWWts+/n9Hkv8noyaq9OHB8b0fc/eA7JhwPVOvtfbgeCe7L8m/jvXlqBvf1/HxJL/fWvvEuLd1ZcIWmy/Wlz601nYluTWj+6CsK52YP1+sK5MjqH3HbUkurKoLqmpNkquT3DThmqZaVZ04vvE7VXVikh9N8qWDv4uj6KYkrx///fok/26CtZCnD2zm/L1YX46q8Y34H0hyd2vtX8x7yboyQQeaL9aXyamqdVV16vjvtUn+xyT/X6wrE3Wg+WJdmRxPfZxn/LjRdyeZSXJDa+1XJ1zSVKuq78noKlqSrE7yYfNkMqrqI0muSHJGkgeT/GKSTyb5WJLnJ/lakr/fWvNwi6PkAPPkioyaprQkX03yD+fu92D5VdUPJPnTJF9Msm/c+20Z3Q9lXZmQg8yXV8f6MhFV9b0ZPSxkJqOLBh9rrf1yVZ0e68rEHGS+3BjrykQIagAAAJ3R9BEAAKAzghoAAEBnBDUAAIDOCGoAAACdEdQAAAA6I6gBAAB0RlADAADojKAGAADQmf8f3NCEL7G3wmQAAAAASUVORK5CYII= ",
text/plain: [
"<Figure size 1080x432 with 1 Axes>"
]
},
metadata: {
needs_background: "light"
},
output_type: "display_data"
}
],
source: [
"# ARIMA ",
"# ACF 그래프 ",
"plot_acf(ts_log3) ",
"# PACF 그래프: p=1이 가장 적합해보임 ",
"# 2 이상인 구간에서는 PACF가 거의 0에 가까워지고 있어서 데이터 상관도가 0이라고 보면 됨. ",
"plot_pacf(ts_log3) ",
"plt.show()"
]
},
{
cell_type: "code",
execution_count: 152,
metadata: { },
outputs: [
{
name: "stdout",
output_type: "stream",
text: [
"Results of Dickey-Fuller Test: ",
"Test Statistic -1.371373e+01 ",
"p-value 1.223940e-25 ",
"#Lags Used 1.900000e+01 ",
"Number of Observations Used 4.023000e+03 ",
"Critical Value (1%) -3.431977e+00 ",
"Critical Value (5%) -2.862259e+00 ",
"Critical Value (10%) -2.567153e+00 ",
"dtype: float64 "
]
},
{
data: {
image/png: "... ",
text/plain: [
"<Figure size 1080x432 with 1 Axes>"
]
},
metadata: {
needs_background: "light"
},
output_type: "display_data"
}
],
source: [
"# d차 차분을 보고 시계열이 안정된 형태인지 확인 => 차분함. ",
"diff_1_3 = ts_log3.diff(periods=1).iloc[1:] ",
"diff_1_3.plot(title='Difference 1st') ",
" ",
"augmented_dickey_fuller_test(diff_1_3)"
]
},
{
cell_type: "code",
execution_count: 153,
metadata: { },
outputs: [
{
data: {
text/plain: [
"<matplotlib.legend.Legend at 0x7fcc868c49d0>"
]
},
execution_count: 153,
metadata: { },
output_type: "execute_result"
},
{
data: {
image/png: "... ",
text/plain: [
"<Figure size 720x432 with 1 Axes>"
]
},
metadata: {
needs_background: "light"
},
output_type: "display_data"
}
],
source: [
"# 학습 테스트 9:1 ",
"train_data3, test_data3 = ts_log3[:int(len(ts_log3)*0.9)], ts_log3[int(len(ts_log3)*0.9):] ",
"plt.figure(figsize=(10,6)) ",
"plt.grid(True) ",
"plt.plot(ts_log3, c='r', label='training dataset') # train_data를 적용하면 그래프가 끊어져 보이므로 자연스러운 연출을 위해 ts_log를 선택 ",
"plt.plot(test_data3, c='b', label='test dataset') ",
"plt.legend()"
]
},
{
cell_type: "code",
execution_count: 154,
metadata: { },
outputs: [
{
name: "stdout",
output_type: "stream",
text: [
"Date ",
"2004-07-22 2.642622 ",
"2004-07-23 2.653242 ",
"Name: Close, dtype: float64 ",
"(3639,) ",
"(405,) "
]
}
],
source: [
"print(ts_log3[:2]) ",
"print(train_data3.shape) ",
"print(test_data3.shape)"
]
},
{
cell_type: "code",
execution_count: 155,
metadata: { },
outputs: [
{
name: "stdout",
output_type: "stream",
text: [
" ARIMA Model Results ",
"============================================================================== ",
"Dep. Variable: D.Close No. Observations: 3638 ",
"Model: ARIMA(1, 1, 0) Log Likelihood 7791.111 ",
"Method: css-mle S.D. of innovations 0.028 ",
"Date: Thu, 13 Aug 2020 AIC -15576.222 ",
"Time: 17:29:19 BIC -15557.625 ",
"Sample: 1 HQIC -15569.598 ",
" ",
"================================================================================= ",
" coef std err z P>|z| [0.025 0.975] ",
"--------------------------------------------------------------------------------- ",
"const -0.0002 0.000 -0.357 0.721 -0.001 0.001 ",
"ar.L1.D.Close -0.0425 0.017 -2.566 0.010 -0.075 -0.010 ",
" Roots ",
"============================================================================= ",
" Real Imaginary Modulus Frequency ",
"----------------------------------------------------------------------------- ",
"AR.1 -23.5237 +0.0000j 23.5237 0.5000 ",
"----------------------------------------------------------------------------- "
]
},
{
name: "stderr",
output_type: "stream",
text: [
"/home/aiffel0036/anaconda3/envs/aiffel/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:218: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting. ",
" ' ignored when e.g. forecasting.', ValueWarning) ",
"/home/aiffel0036/anaconda3/envs/aiffel/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:218: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting. ",
" ' ignored when e.g. forecasting.', ValueWarning) "
]
},
{
data: {
image/png: "... ",
text/plain: [
"<Figure size 1080x432 with 1 Axes>"
]
},
execution_count: 155,
metadata: { },
output_type: "execute_result"
},
{
data: {
image/png: "... ",
text/plain: [
"<Figure size 1080x432 with 1 Axes>"
]
},
metadata: {
needs_background: "light"
},
output_type: "display_data"
}
],
source: [
"# Build Model ",
"model3 = ARIMA(train_data3, order=(1, 1, 0)) ",
"fitted_m3 = model3.fit(disp=-1) ",
"print(fitted_m3.summary()) ",
" ",
"fitted_m3.plot_predict()"
]
},
{
cell_type: "code",
execution_count: 158,
metadata: { },
outputs: [
{
data: {
image/png: "... ",
text/plain: [
"<Figure size 1000x500 with 1 Axes>"
]
},
metadata: {
needs_background: "light"
},
output_type: "display_data"
}
],
source: [
"# Forecast : 결과가 fc에 담깁니다. ",
"fc3, se3, conf3 = fitted_m3.forecast(len(test_data3), alpha=0.05) # 95% conf ",
" ",
"# Make as pandas series ",
"fc_series3 = pd.Series(fc3, index=test_data3.index) # 예측결과 ",
"lower_series3 = pd.Series(conf3[:, 0], index=test_data3.index) # 예측결과의 하한 바운드 ",
"upper_series3 = pd.Series(conf3[:, 1], index=test_data3.index) # 예측결과의 상한 바운드 ",
" ",
"# Plot ",
"plt.figure(figsize=(10,5), dpi=100) ",
"plt.plot(train_data3, label='training') ",
"plt.plot(test_data3, c='b', label='actual price') ",
"plt.plot(fc_series3, c='r',label='predicted price') ",
"plt.fill_between(lower_series3.index, lower_series3, upper_series3, color='k', alpha=.10) ",
"plt.legend() ",
"plt.show()"
]
},
{
cell_type: "code",
execution_count: 159,
metadata: { },
outputs: [
{
name: "stdout",
output_type: "stream",
text: [
"MSE: 3.1423045093055464 ",
"MAE: 1.5487763422038086 ",
"RMSE: 1.7726546503212481 ",
"MAPE: 27.87% "
]
}
],
source: [
"# 최종으로 오차 계산 ",
"mse3 = mean_squared_error(np.exp(test_data3), np.exp(fc3)) ",
"print('MSE: ', mse3) ",
" ",
"mae3 = mean_absolute_error(np.exp(test_data3), np.exp(fc3)) ",
"print('MAE: ', mae3) ",
" ",
"rmse3 = math.sqrt(mean_squared_error(np.exp(test_data3), np.exp(fc3))) ",
"print('RMSE: ', rmse3) ",
" ",
"mape3 = np.mean(np.abs(np.exp(fc3) - np.exp(test_data3))/np.abs(np.exp(test_data3))) ",
"print('MAPE: {:.2f}%'.format(mape3*100))"
]
},
{
cell_type: "markdown",
metadata: { },
source: [
"# Conclusion ",
"- 코로나라는 예측하지 못한 변수를 만나면서 주식 예측력이 떨어진 것 같음. ",
"- 삼성 같은 경우에는 대형주이고 다른 주식들에 비해 큰 영향을 받지 않아 예측력이 10% 미만으로 나온 것 같다. ",
"- 카카오나 lg 디스플레이는 코로나, 중국 등의 영향을 받아서 너무 올라가거나 너무 떨어져서 예측하기 힘들었던 것 같다. ",
"- 주식은 외부 요인에 큰 영향을 받기 때문에 외부요인을 적용하여 실시간으로 예측할 수 있는 프로그램을 만들면 예측력이 더 좋아질 것 같다."
]
}
],
metadata: {
kernelspec: {
display_name: "aiffel",
language: "python",
name: "aiffel"
},
language_info: {
codemirror_mode: {
name: "ipython",
version: 3
},
file_extension: ".py",
mimetype: "text/x-python",
name: "python",
nbconvert_exporter: "python",
pygments_lexer: "ipython3",
version: "3.7.6"
},
toc: {
base_numbering: 1,
nav_menu: { },
number_sections: true,
sideBar: true,
skip_h1_title: false,
title_cell: "Table of Contents",
title_sidebar: "Contents",
toc_cell: false,
toc_position: { },
toc_section_display: true,
toc_window_display: false
},
varInspector: {
cols: {
lenName: 16,
lenType: 16,
lenVar: 40
},
kernels_config: {
python: {
delete_cmd_postfix: "",
delete_cmd_prefix: "del ",
library: "var_list.py",
varRefreshCmd: "print(var_dic_list())"
},
r: {
delete_cmd_postfix: ") ",
delete_cmd_prefix: "rm(",
library: "var_list.r",
varRefreshCmd: "cat(var_dic_list()) "
}
},
types_to_exclude: [
"module",
"function",
"builtin_function_or_method",
"instance",
"_Feature"
],
window_display: false
}
},
nbformat: 4,
nbformat_minor: 4
}